In [ ]:
#-----------THESIS Final Model---------------
#DistilBERT Tokenization --> DistilBERT --> Embedding extraction --> Stack embedding (transform into numerical matrix) --> LSA (document term matrix) --> LSA (SVD)

#SUMMARY-----------------------------------------------------------------------------------------------------------------------------------------------------------

#Imports --> Initialize HuggingFace Transformer Model --> Load the training and testing data 
#(Distilbert) : Define the function to extract embeddings using DistilBERT 
#(Embedding Extraction) : Convert tokenized questions to the required format -> Extract the embeddings for training and testing data using DistilBERT
#(Stack Embedding) : Detach the tensors from the computation graph and convert to NumPy arrays -> Stack Embedding -> Fit the SVD model to the training embeddings
#LSA (Document Term Matrix) : Define the LSA model to be used -> Fit the LSA model to the transformed training embeddings
#LSA (SVD) : Convert token IDs to text for testing data -> Save the text-based train, validation, and test sets to JSON files
#Parameters
#Create the QuestionAnsweringModel
#Load the text-based training data ->  Convert the text-based training data to DataFrame format ->  Convert the DataFrame to the required format for training -> # Convert context and answer to string if they are lists
#Traning
#Prediction
#F1 & EM : Initialize variables to keep track of F1 score and EM ->  Define a dictionary to map IDs to true answers -> Calculate F1 and EM
#Converse with the model 

In [ ]:
#-----------Imports---------------
import os
import json
import torch
import pandas as pd
import numpy as np
import random
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn.decomposition import TruncatedSVD
from sentence_transformers import SentenceTransformer
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
#-----------Initialize HuggingFace Transformer Model---------------
model_name = 'distilbert-base-uncased-distilled-squad'
distilbert_model = DistilBertModel.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

In [ ]:
# Load the training and testing data
with open("train.json", "r") as f:
    train_data = json.load(f)
with open("test.json", "r") as f:
    test_data = json.load(f)

In [ ]:
# -----------DistilBERT---------------
# Define the function to extract embeddings using DistilBERT
def get_distilbert_embeddings(inputs):
    # Tokenize the inputs using the tokenizer
    input_ids = inputs["input_ids"]
    encoded_inputs = [tokenizer.convert_tokens_to_string(ids) for ids in input_ids]
    encoded_inputs = tokenizer.encode_plus(encoded_inputs, padding=True, truncation=True, return_tensors="pt")
    
    # Forward pass through DistilBERT
    outputs = distilbert_model(**encoded_inputs)
    # Extract the embeddings
    embeddings = outputs.last_hidden_state
    return embeddings

In [ ]:
# -----------Embedding Extraction---------------
# Convert tokenized questions to the required format
train_questions = []
for example in train_data:
    question_tokens = example["question"]
    question_text = tokenizer.convert_tokens_to_string([str(token) for token in question_tokens])
    train_questions.append(question_text)

train_inputs = {"input_ids": train_questions}

test_questions = []
for example in test_data:
    question_tokens = example["question"]
    question_text = tokenizer.convert_tokens_to_string([str(token) for token in question_tokens])
    test_questions.append(question_text)

test_inputs = {"input_ids": test_questions}


# Extract the embeddings for training and testing data using DistilBERT
train_embeddings = get_distilbert_embeddings(train_inputs)
test_embeddings = get_distilbert_embeddings(test_inputs)

# Check the shape of the embeddings
print("Shape of train_embeddings:", train_embeddings.shape)
print("Shape of test_embeddings:", test_embeddings.shape)

In [ ]:
# -----------Stack Embedding---------------
# Flatten the embeddings
train_embeddings_flat = train_embeddings.reshape(train_embeddings.shape[0] * train_embeddings.shape[1], -1)
test_embeddings_flat = test_embeddings.reshape(test_embeddings.shape[0] * test_embeddings.shape[1], -1)

# Detach the tensors from the computation graph and convert to NumPy arrays
train_embeddings_flat_np = train_embeddings_flat.detach().numpy()
test_embeddings_flat_np = test_embeddings_flat.detach().numpy()

# Stack Embedding
num_features = 100
svd = TruncatedSVD(n_components=num_features)

# Fit the SVD model to the training embeddings
train_embeddings_transformed = svd.fit_transform(train_embeddings_flat_np)
test_embeddings_transformed = svd.transform(test_embeddings_flat_np)


In [ ]:
#-----------LSA (Document Term Matrix)---------------
# Define the LSA model to be used
lsa = TruncatedSVD(n_components=6)

# Fit the LSA model to the transformed training embeddings
train_embeddings_lsa = lsa.fit_transform(train_embeddings_transformed)
test_embeddings_lsa = lsa.transform(test_embeddings_transformed)

In [ ]:
#-----------LSA (SVD)---------------
train_data_text = []
for example in train_data:
    context_text = example["context"]
    question_text = example["question"]
    answer_text = context_text[example["start_position"]:example["end_position"]+1]  # Extract the answer text dynamically
    train_data_text.append({
        "context": context_text,
        "question": question_text,
        "answer": answer_text
    })

# Convert token IDs to text for testing data
test_data_text = []
for example, embedding in zip(test_data, test_embeddings_lsa):
    context_text = example["context"]
    question_text = example["question"]
    answer_text = context_text[example["start_position"]:example["end_position"]+1]  # Extract the answer text dynamically
    test_data_text.append({
        "context": context_text,
        "question": question_text,
        "answer": answer_text
    })


In [ ]:
# Save the text-based train, validation, and test sets to JSON files
with open("train_text.json", "w") as f:
    json.dump(train_data_text, f)
with open("test_text.json", "w") as f:
    json.dump(test_data_text, f)

In [ ]:
#-----------Parameters---------------
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 32
model_args.n_best_size = 20
model_args.num_train_epochs = 7
model_args.weight_decay = 0.5
model_args.dropout = 0.1

train_args = {
  "reprocess_input_data": False,
  "overwrite_output_dir": False,
  "use_cached_eval_features": False,
  "evaluate_during_training": True,
  "max_seq_length": 512,
  "num_train_epochs": 7,
  "evaluate_during_training_steps": 1000, 
  "wandb_project": "DISTILBERT_CHAT",
  "wandb_kwargs": {"name": f"updated_{model_name}"},
  "save_model_every_epoch": True,
  "save_eval_checkpoints": True,
  "n_best_size": 20,
  "initial_learning_rate": 0.00005,  
  "weight_decay": 0.05, 
  "train_batch_size": 32,  
  "eval_batch_size": 32, 
  "gradient_accumulation_steps": 2,
  "max_grad_norm": 1.0,
  "data_augmentation_techniques": ["random_masking", "random_permutation", "back_translation"],
}

# Create the learning rate scheduler
num_training_steps = len(train_data) * model_args.num_train_epochs // model_args.train_batch_size
warmup_proportion = 0.1
warmup_steps = int(num_training_steps * warmup_proportion)
decay_steps = num_training_steps // 5
decay_rate = 0.01
optimizer = AdamW(distilbert_model.parameters(), lr=train_args["initial_learning_rate"], weight_decay=train_args["weight_decay"])
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)

In [ ]:
#-----------Create the QuestionAnsweringModel---------------
model = QuestionAnsweringModel(
    model_type='distilbert',
    model_name=model_name,
    args=model_args,
    use_cuda=True,
)

In [ ]:
# Load the text-based training data
with open("train_text.json", "r") as f:
    train_text_data = json.load(f)

# Convert the text-based training data to DataFrame format
train_df = pd.DataFrame(train_text_data)

# Convert the DataFrame to the required format for training
train_input = []
for i, row in train_df.iterrows():
    context = row['context']
    question = row['question']
    answer = row['answer']
    
    # Convert context and answer to string if they are lists
    if isinstance(context, list):
        context = ' '.join(str(item) for item in context)
    if isinstance(answer, list):
        answer = ' '.join(str(item) for item in answer)
    
    train_input.append({
        'context': context,
        'qas': [
            {
                'id': str(i),
                'question': question,
                'answers': [
                    {
                        'text': answer,
                        'answer_start': context.find(answer),
                    }
                ],
                'is_impossible': False
            }
        ]
    })


In [ ]:
# Load the text-based test data
with open("test_text.json", "r") as f:
    test_text_data = json.load(f)

# Convert the text-based test data to DataFrame format
test_df = pd.DataFrame(test_text_data)

# Convert the DataFrame to the required format for prediction
test_input = []
for i, row in test_df.iterrows():
    context = row['context']
    question = row['question']
    answer = row['answer']
    
    # Convert context and answer to string if they are lists
    if isinstance(context, list):
        context = ' '.join(str(item) for item in context)
    if isinstance(answer, list):
        answer = ' '.join(str(item) for item in answer)
    
    test_input.append({
        'context': context,
        'qas': [
            {
                'id': str(i),
                'question': question,
                'answers': [
                    {
                        'text': answer,
                        'answer_start': context.find(answer),
                    }
                ]
            }
        ]
    })


In [15]:
#-----------Traning---------------
model.train_model(
    train_input,
    output_dir='MODEL-RESULTS-3', 
    optimizer=optimizer,
    scheduler=scheduler,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
)

add example index and unique id: 100%|██████████| 490/490 [00:00<00:00, 145316.34it/s]


Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Running Epoch 0 of 7:   0%|          | 0/16 [00:00<?, ?it/s]

c:\Users\Jude\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Running Epoch 1 of 7:   0%|          | 0/16 [00:00<?, ?it/s]

...

Running Epoch 3 of 7:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 4 of 7:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 5 of 7:   0%|          | 0/16 [00:00<?, ?it/s]

Running Epoch 6 of 7:   0%|          | 0/16 [00:00<?, ?it/s]

(112, 0.3895272864231499)

In [ ]:
#-----------Prediction---------------
predictions = model.predict(test_input)

In [ ]:
to_predict = [
    {
        "context": "OXGN is a fashion brand designed for all. From cool styles, affordable essentials and one of a kind collaboration pieces, our customers are sure to find that signature OXGN brand of style. With timeless classics from our GENERATIONS collection, the leveled-up sports inspired pieces from PREMIUM THREADS and our inclusive co-gender line COED, our brand serves to make looking cool as easy as breathing.",
        "qas": [
            {
                "question": "What is OXGN?",
                "id": "0"
            }
        ]
    },
    {
        "context": "GOLDEN ABC, Inc. (GABC) is a multi-awarded international fashion enterprise that is home to top proprietary brands shaping the retail industry today. Produced, marketed, and retailed under a fast-growing, dynamic family of well-differentiated, proprietary brands: PENSHOPPE, OXYGEN, FORME, MEMO, REGATTA, and BOCU. Originating in the Philippines in 1986, GABC now has more than 1000 strategically located sites and presence in different countries around Asia.",
        "qas": [
            {
                "question": "What is Golden ABC INC.?",
                "id": "1"
            }
        ]
    },
    {
        "context": "The OXGN VIP Loyalty Card is not applicable online and it has been already been discontinued in our physical stores last December 31, 2021.",
        "qas": [
            {
                "question": "Can my Oxygen Customer Loyalty Card be applied to my online order?",
                "id": "2"
            }
        ]
    },
    {
        "context": "A confirmation e-mail/ SMS will be sent for all successful orders made. This contains your order number, the order amount, item(s) ordered, your billing and delivery address, including the payment method used and how you will be receiving order updates. Receipt of this email means that your order has been confirmed and will be prepared by our team within 1-5 working days. You'll get another e-mail/SMS once your package has been handed over to our shipping partner or is now ready for pick-up in your chosen OXGN branch.",
        "qas": [
            {
                "question": "How to confirm order?",
                "id": "3"
            }
        ]
    },
    {
        "context": "You have the option to cancel your order within two (2) hours from the time it was created. You can see this on your order confirmation email or on your online store order history. Once the time frame has passed, the option to cancel will no longer appear on your end. If in case you still need to cancel your order but you no longer have the option to do so, you can send an email to shop@oxgnfashion.com and our Customer Care Team will get in touch with you.",
        "qas": [
            {
                "question": "Can I cancel my order?",
                "id": "4"
            }
        ]
    },
    {
        "context": "We cannot include paper bags for Online Orders shipped to their chosen delivery address but we can definitely provide for those Online orders that were checked-out on a Store Pick-up option.",
        "qas": [
            {
                "question": "Can I request for a paper bag?",
                "id": "5"
            }
        ]
    },
    {
        "context": "After you tap on REVIEW MY CART, you will see the option to send your order as a Gift. Sending it as a gift means that a gift receipt will be used instead of a regular sales invoice. A gift receipt shows proof of purchase but leaves out the amount spent. This doesn't mean that your order will be gift wrapped or it will come in special packaging. There is also no way to place a special message for the recipient. Orders made through COD (cash-on-delivery) cannot be sent as gifts.",
        "qas": [
            {
                "question": "Can I send order as a gift?",
                "id": "6"
            }
        ]
    },
    {
        "context": "Once the check out process has been completed, changes to the order in terms of item/size/color/quantity can no longer be made. What you can do is cancel your order completely within two (2) hours from the time you placed it so you can place a new one with the changes you want. You can see this on your order confirmation email and on your online store order history. You also have the option to accept the order upon delivery or pick-up the item in your chosen store and request for a return-exchange instead.",
        "qas": [
            {
                "question": "How Can I Make Changes to My Order?",
                "id": "7"
            }
        ]
    },
    {
        "context": "The first step is to click 'Add to Cart' once you've found something you want to save the items in your order basket. Second, click the 'Review My Cart' button to be redirected to the cart page, where you can make any final changes to your order, such as quantity, size, or style. Third, address Finder will appear on your screen. Please enter your delivery address by using the dropdown menus. If you accidentally exit out before finishing your details, click on 'Address Finder' to bring it back up. This is to prevent your orders from being held due to incorrect or incomplete shipping information.",
        "qas": [
            {
                "question": "How Do I Make an Order?",
                "id": "8"
            }
        ]
    },
    {
        "context": "Yes, you have the option to cancel your order within two (2) hours from the time it was created either it is shipped to your preferred address or pick-up in your chosen store. You can view this on your order confirmation email or on your online store order history. Once the time frame has passed, the option to cancel will no longer appear on your end. If in case you still want to cancel your order but you no longer have the option to do so, you can send an email to shop@oxgnfashion.com and our Customer Care Team will get in touch with you.",
        "qas": [
            {
                "question": "Can I Cancel My Order?",
                "id": "9"
            }
        ]
    },
    {
        "context": "All unclaimed packages beyond the scheduled pick-up date are considered canceled and will be put back to our stocks. You have the option to re-order again in our Online Store but we can no longer guarantee the availability of the same items as our items sell out quite fast.",
        "qas": [
            {
                "question": "Was not able to claim my Order. Can I re-order?",
                "id": "10",
            }
        ]
    },
    {
        "context": "You can now order online and pick up the items in your chosen OXGN branches: SM CITY NORTH EDSA, SM CITY SAN LAZARO, AYALA TRINOMA, ROBINSONS GALLERIA, SM MEGAMALL, SM CITY CEBU ,AYALA CENTER CEBU, MARKET MARKET, SM CITY FAIRVIEW, SM SEASIDE CEBU, SM LANANG, GMALL TAGUM, SM CITY DAVAO, GAISANO MALL OF DAVAO, SM CITY EAST ORTIGAS, SM MARIKINA, AYALA GLORIETTA, SM CITY MANILA, SM CITY GRAND CENTRAL, MALL OF ASIA, SM CITY SUCAT, ROBINSONS LAOAG, SM CITY BAGUIO, SM CITY PAMPANGA, SM CITY CLARK, SM CITY DASMARIÑAS, SM CALAMBA, SM CITY LUCENA, SM CITY LIPA, SM CITY ILO-ILO, SM CITY BACOLOD, SM Roxas, AYALA CENTRAL BLOC, Robinsons Tacloban, MINDPRO, ROBINSONS ILIGAN, Gaisano Mall Digos, SM CITY GENERAL SANTOS, KCC MALL MARBEL, SM CITY CAUAYAN, ISABELA, SM CITY MARILAO, TUTUBAN CENTER, Val Alabang, AYALA MALLS MANILA BAY, SM CITY LEGAZPI, ALTURAS MALL TAGBILARAN, SM PREMIER, SM BUTUAN, Mall of Alnor, SM City Tuguegarao, Ever Commonwelath, ROBINSONS BACOLOD, Fishermall. ",
        "qas": [
            {
                "question": "In which stores can I pickup my order?",
                "id": "11",
            }
        ]
    },
    {
        "context": "As much as we'd like to, this option is not yet allowed at the moment. If you are not available to pick up your order on the scheduled date, you may assign a representative and prepare the following: Representative's Valid ID, Authorization letter with their full name signed by you, Photocopy of your Valid ID, and Screenshot of Order No. from email.",
        "qas": [
            {
                "question": "Can I just book Grab/Lalamove/etc to pick up my order?",
                "id": "12",
            }
        ]
    },
    {
        "context": "This feature gives you the option to buy the items online and pick it up in our stores. The Stores included on the list are those that can cater pickup option only as this is not yet available nationwide. But watch out as it will be soon!",
        "qas": [
            {
                "question": "What is 'Check in Store' in the product page do?",
                "id": "13",
            }
        ]
    },
    {
        "context": "Oxgnfashion.com accepts the following forms of payment: GCash, Credit/Debit Card and COD (Cash-on-Delivery). During the order checkout process, you will be asked to select the payment method of your choice. If you select Credit/Debit Card, you will be redirected to our payment portal where you will be asked to enter your card details. Selecting GCash will also redirect you to a different page once you tap the 'Complete Orde' button. For COD (cash-on-delivery) orders, you will be asked to make payment once the courier has delivered the order to your shipping address. For orders to be picked-up in the store, you have the option to pay during your pick-up based on the store's available Payment option. You just have to choose COD as mode of payment upon check out to complete the transaction. But you can also pay in Credit/Debit Card, GCash, and Shopee Pay when you picked up your order.",
        "qas": [
            {
                "question": "How can I pay for my order?",
                "id": "14",
            }
        ]
    },
    {
        "context": "No, there is no option to save your credit or debit card information on your online store profile. Even if you have already used the same card, we do not store your card details to protect the security of your data. If you select Credit/Debit card as your payment option, you will be asked to enter your card details through a payment portal each time you complete an order.",
        "qas": [
            {
                "question": "Can I Save My Credit/Debit Card Information?",
                "id": "15",
            }
        ]
    },
    {
        "context": "Customers will receive an e-mail notification when their order is read for pick up. It is stated in the notification that they can pick up the order within 7 days from the receipt of e-mail. Customer will receive this update via their registered e-mail address. Failure to pick-up within seven (7) days may result to the cancellation of order. ",
        "qas": [
            {
                "question": "Until when can I pick-up my Order?",
                "id": "16",
            }
        ]
    },
    {
        "context": "No worries! You may assign a representative that may pick-up your order and have them present the following: Representative's Valid ID, Authorization letter with your Full Name and Signature, Photocopy of your Valid ID, and Screenshot of Order No. from email",
        "qas": [
            {
                "question": "What if I cannot pick-up my order within the given timeframe?",
                "id": "17",
            }
        ]
    },
    {
        "context": "If you have placed order via our online store and chose to pick it up in one of branches, please prepare the following to be checked by our store partners: Valid ID, Screenshot of Order No. from email. In case a representative will pick-up your order, please have them present the following: Representative's Valid ID, Authorization letter with their full name signed by you, Photocopy of your Valid ID, and Screenshot of Order No. from email",
        "qas": [
            {
                "question": "What are the requirements for pick-up orders?",
                "id": "18",
            }
        ]
    },
    {
        "context": "Customers with prepaid orders are allowed to request for up to 7 days extension from the first pick-up schedule. Unfortunately, for orders that were placed on COD/Cash payment basis, extension is not valid and will be automatically cancelled if not claimed on the assigned pick-up date. ",
        "qas": [
            {
                "question": "How can I extend the pick-up date of my order",
                "id": "19",
            }
        ]
    },
    {
        "context": "If your order gets returned to our warehouse, we're sorry to let you know that it is already considered as canceled from our end. This is only done after two (2) unsuccessful delivery attempts. For orders paid via GCash or Credit/Debit card, a refund will be processed via our Store Credit Voucher or the same payment method you have selected. If you still want to get the same items, you can definitely create a new order on our online store.",
        "qas": [
            {
                "question": "What If My Order Got Returned?",
                "id": "20",
            }
        ]
    },
    {
        "context": "Our delivery partners will make two (2) attempts in getting your package delivered. If they missed you on the first instance, the courier will either send you an SMS or give you a call to let you know that he came. If in case you wouldn't be around on the 2nd delivery attempt (done on the next business day), you have the option to assign an authorized representative who is present at your shipping address to receive the order. Your authorized representative will be asked to sign for the package on your behalf. After two (2) delivery attempts and still unsuccessful, the courier will return your package to us and will already be considered as canceled. For orders paid via GCash, Credit/Debit card, a refund will be processed via our Store Credit Voucher or the same payment method you have selected. If you have questions or you need help in getting your order delivered, you can send us an email at shop@oxgnfashion.com or fill up our Website Contact Us Form.",
        "qas": [
            {
                "question": "What will happen if I missed the delivery?",
                "id": "21",
            }
        ]
    },
    {
        "context": "As we believe in making the latest fashion styles accessible to everyone, we can assure you that we have wide reach across the Philippines. However, due to geographical and logistical restrictions, there might be some isolated areas we cannot serve but rest assured we'll do our best to get to your location. If your order went through but could not be delivered due to this, our Customer Care Team will reach out to you via email.  You can also send us an email at shop@oxgnfashion.com or fill up our Website Contact Us Form",
        "qas": [
            {
                "question": "What if I'm in the no delivery zones?",
                "id": "22",
            }
        ]
    },
    {
        "context": "Customers who need to contact our shipping partners can do so using the following methods: For LBC, customers can send an email to customercare@lbcexpress.com or call (02) 8 858 59 99. For Go21, customers can visit the Inquiry Page at https://booking.go21.com.ph/ or call +63 2 832 99 27. For J&T, customers can visit https://www.jtexpress.sg/contact-us or call (02) 911 1888. For WSI, customers can send an email to communicationsupport@wsi.ph or call (02) 8533 3888. For feedback on our shipping partners or delivery concerns, customers can send an email to shop@oxgnfashion.com or fill out the Website Contact Us Form.",
        "qas": [
            {
                "question": "Who are your shipping partners?",
                "id": "23",
            }
        ]
    },
    {
        "context": "Once your order has been handed over to our shipping partners, you will receive an email notification along with your order tracking number. To track your order, follow the steps indicated in our shipping partners' websites provided: For LBC orders, click https://www.lbcexpress.com/track/ and enter your tracking number on the Track & Trace search field. For Go21 orders, click https://booking.go21.com.ph/  and enter your Tracking Number on the Locate My Package search field. For J&T orders, click https://www.jtexpress.ph/index/query/gzquery.html  and enter your Tracking Number on the Track & Trace search field. For WSI orders, click http://www.worklinkdalivery.com/ and enter your Tracking Number on the Track & Trace search field. If your order does not arrive within the delivery timeline provided, you can send us an email at shop@oxgnfashion.com or fill up our Website Contact Us form.",
        "qas": [
            {
                "question": "How can I track orders?",
                "id": "24",
            }
        ]
    },
    {
        "context": "Customers who wish to return items purchased from our online store can do so by packing the item along with the sales invoice and bringing it to any Penshoppe store nationwide. The return must be made within 30 days from the date of purchase, with all tags attached and the item unused and unwashed. Evaluation of the item's condition will be conducted. Please note that undergarments, face masks, swimsuits, jewelries, and cosmetics are not valid for return due to sanitary reasons and availability of the item may vary per boutique. Customers may also choose to exchange the item for a different item of the same value or pay the excess amount for a higher value item, with the excess amount being forfeited if of lower value. Our store partners will assist with the return process. For any further concerns, customers can visit the link https://www.oxgnfashion.com/pages/contact-us.",
        "qas": [
            {
                "question": "What is the Online Store Returns Policy?",
                "id": "25",
            }
        ]
    },
    {
        "context": "If the same item is not available, you do not have to worry as you can choose any item inside the store. If the new item (different from what was originally purchased) chosen for exchange is of higher value than what was originally purchased, the price difference must be paid at the time of the exchange. If of lower value, the price difference is forfeited. Please don't forget to bring your online store receipt when you drop by the boutique to process your return-exchange transaction.",
        "qas": [
            {
                "question": "What If the Item I Want For Exchange is Out of Stock (exchange due to size only)?",
                "id": "26",
            }
        ]
    },
    {
        "context": "If you receive a gift item from our online store and want to have it exchanged for a different size/color/style, you have the option to do so following our Returns Policy. Just bring your gift receipt to any of our store branches and our boutique staff can assist you with this.Online returns and exchanges cannot be facilitated.",
        "qas": [
            {
                "question": "Can I Exchange Gifts Given To Me?",
                "id": "27",
            }
        ]
    },
    {
        "context": "If you need to process a return or exchange transaction and have lost your online store receipt, please reach out to us with the following information: your order number, the item you will be returning, the reason for the return, the boutique you will be going to, and the date. We require this information as we will coordinate your request with our store partners. Additionally, we will relay your order invoice number so that your return or exchange transaction can be processed. You can send us an email at shop@oxgnfashion.com or fill out our website's contact us form.",
        "qas": [
            {
                "question": "How Can I Process an Exchange If I lost my Online Store Receipt?",
                "id": "28",
            }
        ]
    },
    {
        "context": "No, our warehouse does not accept returns from customers but our physical stores do. If you'll send an item to our warehouse, we will send it back to you. ",
        "qas": [
            {
                "question": "Does Your Warehouse Accept Returns?",
                "id": "29",
            }
        ]
    },
    {
        "context": "In the event that you received a damaged or incorrect item, it is important to reach out to us immediately so that we can investigate the situation. To help us with this, please provide us with the following information: the order number and a description of the damaged or incorrect item received, along with a picture of the item including its tag. You can contact us by sending an email to shop@oxgnfashion.com or by filling out our website's contact us form.",
        "qas": [
            {
                "question": "What to do in case there is a damaged/incorrect item received?",
                "id": "30",
            }
        ]
    },
    {
        "context": "Refund requests are always subject to validation. It is important to note that a change in size, style, or color is not eligible for a refund. For online purchases, if payment was made in cash, refunds will be processed through a store credit voucher that is a one-time use and has no expiry date. On the other hand, if payment was made through a credit/debit card or GCash, it may take 20-30 business days before the refunded amount will reflect on your account. For in-store purchases, the refund will be processed through the original mode of payment and may take 1-30 days, depending on the payment type.",
        "qas": [
            {
                "question": "What if I want to request for a Refund?",
                "id": "31",
            }
        ]
    },
    {
        "context": "Creating an account on the OXGN Online Store is easy and allows you to keep track of your orders through the Order History feature. To create an account, follow these simple steps. First, tap on the 'Account icon' located on the top right corner of the screen. For first-time users, fill out the required information in the Create Account fields. Once complete, tap on the 'CREATE' button and you will be directed to the Account Page, indicating that you have successfully logged in to the OXGN Online Store.",
        "qas": [
            {
                "question": "How Do I Create an Account?",
                "id": "32",
            }
        ]
    },
    {
        "context": "Yes, you can still shop without making an online store account! Just place the items you want in your cart and proceed to check out. Once you reach the customer information page, you will be asked for your contact information to successfully place your order. ",
        "qas": [
            {
                "question": "What If I Don't Have an Account?",
                "id": "33",
            }
        ]
    },
    {
        "context": "If you have forgotten your password for your online store account, don't worry as it is an easy process to reset it. Simply tap on the 'Forgot your password' link on the login screen and it will take you to the reset password page. Enter the email address you used to register for the account and click on the 'SUBMIT' button. Within seconds, you will receive an email with instructions on how to complete the password reset process.",
        "qas": [
            {
                "question": "Forgot the Password?",
                "id": "34",
            }
        ]
    },
    {
        "context": "First time to sign in? Tap on the account icon so you can start adding your account details. Tap on the View Addresses link and enter the information needed on the Add Address fields. You can add multiple addresses and you can also choose your default address. Please ensure that you input your mobile number on the Phone field as that's needed by our shipping partners. ",
        "qas": [
            {
                "question": "How Can I Update My Account Information?",
                "id": "35",
            }
        ]
    },
    {
        "context": "Reach out to +639399101047 or complete the form below and we’ll gladly assist you. Our Customer Care Team is available from Monday to Friday, 9am to 6pm.",
        "qas": [
            {
                "question": "How to contact the store?",
                "id": "36",
            }
        ]
    },
    {
        "context": "These Terms and Conditions ('Terms', 'Terms and Conditions') govern your relationship with OXGN Website (the 'Service') operated by Golden ABC, Incorporated ('us', 'we', or 'our'). Please read these Terms and Conditions carefully before using our OXGN Website(the 'Service'). Your access to and use of the Service is conditioned on your acceptance of and compliance with these Terms. These Terms apply to all visitors, users and others who access or use the Service. By accessing or using the Service you agree to be bound by these Terms. If you disagree with any part of the terms then you may not access the Service. PURCHASES, If you wish to purchase any product or service made available through the Service ('Purchase'), you may be asked to supply certain information relevant to your Purchase including, without limitation, your credit card number, the expiration date of your credit card, your billing address, and your shipping information. You represent and warrant that: (i) you have the legal right to use any credit card(s) or other payment method(s) in connection with any Purchase; and that (ii) the information you supply to us is true, correct and complete. By submitting such information, you grant us the right to provide the information to third parties for purposes of facilitating the completion of Purchases. We reserve the right to refuse or cancel your order at any time for certain reasons including but not limited to: product or service availability, errors in the description or price of the product or service, error in your order or other reasons. We reserve the right to refuse or cancel your order if fraud or an unauthorised or illegal transaction is suspected. AVAILABILITY, ERRORS AND INACCURACIES, we are constantly updating our offerings of products and services on the Service. The products or services available on our Service may be mispriced, described inaccurately, or unavailable, and we may experience delays in updating information on the Service and in our advertising on other web sites. We cannot and do not guarantee the accuracy or completeness of any information, including prices, product images, specifications, availability, and services. We reserve the right to change or update information and to correct errors, inaccuracies, or omissions at any time without prior notice. CONTESTS, SWEEPSTAKES AND PROMOTIONS,Any contests, sweepstakes or other promotions (collectively, 'Promotions') made available through the Service may be governed by rules that are separate from these Terms. If you participate in any Promotions, please review the applicable rules as well as our Privacy Policy. If the rules for a Promotion conflict with these Terms, the Promotion rules will apply. ACCOUNTS, When you create an account with us, you must provide us information that is accurate, complete, and current at all times. Failure to do so constitutes a breach of the Terms, which may result in immediate termination of your account on our Service. You are responsible for safeguarding the password that you use to access the Service and for any activities or actions under your password, whether your password is with our Service or a third-party service. You agree not to disclose your password to any third party. You must notify us immediately upon becoming aware of any breach of security or unauthorized use of your account. COPYRIGHT POLICY, We respect the intellectual property rights of others. It is our policy to respond to any claim that Content posted on the Service infringes the copyright or other intellectual property infringement ('Infringement') of any person. If you are a copyright owner, or authorized on behalf of one, and you believe that the copyrighted work has been copied in a way that constitutes copyright infringement that is taking place through the Service, you must submit your notice in writing to the attention of 'Copyright Infringement' of legal@goldenabc.com and include in your notice a detailed description of the alleged Infringement. You may be held accountable for damages (including costs and attorneys' fees) for misrepresenting that any Content is infringing your copyright. INTELLECTUAL PROPERTY, The Service and its original content, features and functionality are and will remain the exclusive property of Golden ABC, Incorporated and its licensors. The Service is protected by copyright, trademark, and other laws of both the Philippines and foreign countries. Our trademarks and trade dress may not be used in connection with any product or service without the prior written consent of Golden ABC, Incorporated. LINKS TO OTHER WEB SITES, Our Service may contain links to third-party web sites or services that are not owned or controlled by Golden ABC, Incorporated. Golden ABC, Incorporated has no control over, and assumes no responsibility for, the content, privacy policies, or practices of any third party web sites or services. You further acknowledge and agree that Golden ABC, Incorporated shall not be responsible or liable, directly or indirectly, for any damage or loss caused or alleged to be caused by or in connection with use of or reliance on any such content, goods or services available on or through any such web sites or services. We strongly advise you to read the terms and conditions and privacy policies of any third-party web sites or services that you visit. TERMINATION, We may terminate or suspend your account immediately, without prior notice or liability, for any reason whatsoever, including without limitation if you breach the Terms. Upon termination, your right to use the Service will immediately cease. If you wish to terminate your account, you may simply discontinue using the Service. LIMITATION OF LIABILITY, In no event shall Golden ABC, Incorporated, nor its directors, employees, partners, agents, suppliers, or affiliates, be liable for any indirect, incidental, special, consequential or punitive damages, including without limitation, loss of profits, data, use, goodwill, or other intangible losses, resulting from (i) your access to or use of or inability to access or use the Service; (ii) any conduct or content of any third party on the Service; (iii) any content obtained from the Service; and (iv) unauthorized access, use or alteration of your transmissions or content, whether based on warranty, contract, tort (including negligence) or any other legal theory, whether or not we have been informed of the possibility of such damage, and even if a remedy set forth herein is found to have failed of its essential purpose. DISCLAIMER, Your use of the Service is at your sole risk. The Service is provided on an 'AS IS' and'AS AVAILABLE' basis. The Service is provided without warranties of any kind, whether express or implied, including, but not limited to, implied warranties of merchantability, fitness for a particular purpose, non-infringement or course of performance. Golden ABC, Incorporated its subsidiaries, affiliates, and its licensors do not warrant that a) the Service will function uninterrupted, secure or available at any particular time or location; b) any errors or defects will be corrected; c) the Service is free of viruses or other harmful components; or d) the results of using the Service will meet your requirements. GOVERNING LAW, these Terms shall be governed and construed in accordance with the laws of Philippines, without regard to its conflict of law provisions. Our failure to enforce any right or provision of these Terms will not be considered a waiver of those rights. If any provision of these Terms is held to be invalid or unenforceable by a court, the remaining provisions of these Terms will remain in effect. These Terms constitute the entire agreement between us regarding our Service, and supersede and replace any prior agreements we might have between us regarding the Service. CHANGES, We reserve the right, at our sole discretion, to modify or replace these Terms at any time. If a revision is material we will try to provide at least 30 days notice prior to any new terms taking effect. What constitutes a material change will be determined at our sole discretion. By continuing to access or use our Service after those revisions become effective, you agree to be bound by the revised terms. If you do not agree to the new terms, please stop using the Service. CONTACT US,If you have any questions about these Terms, please contact us.",
        "qas": [
            {
                "id": "37",
                "question": "What is your terms and conditions?",
            }
        ]
    },
    {
        "context": "There are ten available variations of tops for men: Regular Fit Shirt, Resort Fit Shirt, Long Sleeve, Hoodie, Relax set-in Pullover, Easy Fit, Jersy, Slim Fit, Oversized, and Tank Top.",
        "qas": [
            {
                "question": "What are available men's tops items?",
                "id": "38",
            }
        ]
    },
    {
        "context": "There are five available variations of bottoms for men: Skinny, Tapered (Slim Fit, Woven, Knit), and Jogger Pants",
        "qas": [
            {
                "question": "What are available men's bottoms items?",
                "id": "39",
            }
        ]
    },
    {
        "context": "There are two available variations of loungewear for men: Robe and Lounge Set.",
        "qas": [
            {
                "question": "What are available men's loungewear items?",
                "id": "40",
            }
        ]
    },
    {
        "context": "There are six available variations of footwear for men: Sliders, Single Band, Velcro Sliders, Regular Fliptops, Lace-up, and Lace-up Runner.",
        "qas": [
            {
                "question": "What are available men's footwear items?",
                "id": "41",
            }
        ]
    },
    {
        "context": "There are seven available variations of tops for ladies: Regular, Boxy, Slim fit Tee, Shirt (Short Sleeve), Sports Bra, Hoodie, and Regular Pullover",
        "qas": [
            {
                "question": "What are available ladie's top's items?",
                "id": "42",
            }
        ]
    },
    {
        "context": "There are two available variations of dress for ladies: Oversized Dress, Long Sleeve Regular Fit.",
        "qas": [
            {
                "question": "What are available ladie's dress items?",
                "id": "43",
            }
        ]
    },
    {
        "context": "There are ten available variations of bottoms for ladies: Skinny Jeans (Numerical,  Alpha), Wide Leg Jeans, Shorts (Numerical, Alpha), Biker Shorts (Alpha), Leggings (With Pockets, Without Pockets), Track Pants, and Urban Tapered Trousers.",
        "qas": [
            {
                "question": "What are available ladie's bottoms items?",
                "id": "44",
            }
        ]
    },
    {
        "context": "There are four available variations of footwear for ladies: Sliders, Single Band Sliders, Velcro Sliders, and Lace-up Sneakers.",
        "qas": [
            {
                "question": "What are available ladies' footwear items?",
                "id": "45",
            }
        ]
    },
    {
        "context": "There are three available categories of accessories in total: Bags, Caps and Hats, and Wallet and Purses.",
        "qas": [
            {
                "question": "What are available accessory items?",
                "id": "46",
            }
        ]
    },
    {
        "context": "There are four available variations of Bags in accessories: Drawstring Backpack, Bum Bag, Sling Bag, and Tote Bag.",
        "qas": [
            {
                "question": "What are the available bags?",
                "id": "47",
            }
        ]
    },
    {
        "context": "There are two available variations of Caps/Hat in accessories: Trucker Hat and Bucket Hat.",
        "qas": [
            {
                "question": "What are the available hats or caps?",
                "id": "48",
            }
        ]
    },
    {
        "context": "There are three available variations of walltes/purses in accessories: Coin Purse (Horizontal & Vertical) and Midsized Bifold Wallet.",
        "qas": [
            {
                "question": "What are the available wallets or purses",
                "id": "49",
            }
        ]
    },
    {
        "context": "For men's top regular fit shirt in double extra small, Chest: 37, Body Length: 26, Sleeve Length: 7.75, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top regular fit shirt in double extra small?",
                "id": "50",
            }
        ]
    },
    {
        "context": "For men's top regular fit shirt in extra small, Chest: 39, Body Length: 27, Sleeve Length: 8, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top regular fit shirt in extra small?",
                "id": "51",
            }
        ]
    },
    {
        "context": "For men's top regular fit shirt in small, Chest: 41, Body Length: 28, Sleeve Length: 8.25, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top regular fit shirt in small?",
                "id": "52",
            }
        ]
    },
    {
        "context": "For men's top regular fit shirt in medium, Chest: 43, Body Length: 29, Sleeve Length: 8.5, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top regular fit shirt in medium?",
                "id": "53",
            }
        ]
    },
    {
        "context": "For men's top regular fit shirt in large, Chest: 45, Body Length: 30, Sleeve Length: 8.75, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top regular fit shirt in large?",
                "id": "54",
            }
        ]
    },
    {
        "context": "For men's top regular fit shirt in extra large, Chest: 47, Body Length: 31, Sleeve Length: 9, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top regular fit shirt in extra large?",
                "id": "55",
            }
        ]
    },
    {
        "context": "For men's top regular fit shirt in double extra large, Chest: 49, Body Length: 32, Sleeve Length: 9.25, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top regular fit shirt in double extra large?",
                "id": "56",
            }
        ]
    },
    {
        "context": "For men's top resort fit shirt in double extra small, Chest: 39, Body Length: 27.5, Sleeve Length: 9.25, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top resort fit shirt in double extra small?",
                "id": "57",
            }
        ]
    },
    {
        "context": "For men's top resort fit shirt in extra small, Chest: 41, Body Length: 28, Sleeve Length: 9.5, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top resort fit shirt in extra small?",
                "id": "58",
            }
        ]
    },
    {
        "context": "For men's top resort fit shirt in small, Chest: 43, Body Length: 28.5, Sleeve Length: 9.75, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "id": "59",
                "question": "What are the size and dimension possibilities for men's top resort fit shirt in small?",
            }
        ]
    },
    {
        "context": "For men's top resort fit shirt in medium, Chest: 45, Body Length: 29, Sleeve Length: 10, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top resort fit shirt in medium?",
                "id": "60",
            }
        ]
    },
    {
        "context": "For men's top resort fit shirt in large, Chest: 47, Body Length: 29.5, Sleeve Length: 10.25, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top resort fit shirt in large?",
                "id": "61",
            }
        ]
    },
    {
        "context": "For men's top resort fit shirt in extra large, Chest: 49, Body Length: 30, Sleeve Length: 10.5, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top resort fit shirt in extra large?",
                "id": "62",
            }
        ]
    },
    {
        "context": "For men's top resort fit shirt in double extra large, Chest: 51, Body Length: 30.5, Sleeve Length: 10.75, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top resort fit shirt in double extra large?",
                "id": "63",
            }
        ]
    },
    {
        "context": "For men's top long sleeve shirt in double extra small, Chest: 36, Body Length: 26, Sleeve Length: 24.75, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top long sleeve shirt in double extra small?",
                "id": "64",
            }
        ]
    },
    {
        "context": "For men's top long sleeve shirt in extra small, Chest: 38, Body Length: 27, Sleeve Length: 25, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top long sleeve shirt in extra small?",
                "id": "65",
            }
        ]
    },
    {
        "context": "For men's top long sleeve shirt in small, Chest: 40, Body Length: 28, Sleeve Length: 25.25, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top long sleeve shirt in small?",
                "id": "66",
            }
        ]
    },
    {
        "context": "For men's top long sleeve shirt in medium, Chest: 42, Body Length: 29, Sleeve Length: 25.5, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top long sleeve shirt in medium?",
                "id": "67",
            }
        ]
    },
    {
        "context": "For men's top long sleeve shirt in large, Chest: 44, Body Length: 30, Sleeve Length: 25.75, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top long sleeve shirt in large?",
                "id": "68",
            }
        ]
    },
    {
        "context": "For men's top long sleeve shirt in extra large, Chest: 46, Body Length: 31, Sleeve Length: 26, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top long sleeve shirt in extra large?",
                "id": "69",
            }
        ]
    },
    {
        "context": "For men's top long sleeve shirt in double extra large, Chest: 48, Body Length: 32, Sleeve Length: 26.25, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top long sleeve shirt in double extra large?",
                "id": "70",
            }
        ]
    },
    {
        "context": "For men's top long sleeve shirt in triple extra large, Chest: 50, Body Length: 33, Sleeve Length: 26.5, USA: 24, EU: 58, and UK: 48.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top long sleeve shirt in triple extra large?",
                "id": "71",
            }
        ]
    },
    {
        "context": "For men's top hoodie dropped shoulder shirt in double extra small, Chest: 44, Body Length: 25.5, Sleeve Length: 17.25, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie dropped shoulder shirt in double extra small?",
                "id": "72",
            }
        ]
    },
    {
        "context": "For men's top hoodie dropped shoulder shirt in extra small, Chest: 46, Body Length: 26, Sleeve Length: 17.5, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie dropped shoulder shirt in extra small?",
                "id": "73",
            }
        ]
    },
    {
        "context": "For men's top hoodie dropped shoulder shirt in small, Chest: 48, Body Length: 26.5, Sleeve Length: 17.75, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie dropped shoulder shirt in small?",
                "id": "74",
            }
        ]
    },
    {
        "context": "For men's top hoodie dropped shoulder shirt in medium, Chest: 50, Body Length: 27, Sleeve Length: 18, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie dropped shoulder shirt in medium?",
                "id": "75",
            }
        ]
    },
    {
        "context": "For men's top hoodie dropped shoulder shirt in large, Chest: 52, Body Length: 27.5, Sleeve Length: 18.25, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie dropped shoulder shirt in large?",
                "id": "76",
            }
        ]
    },
    {
        "context": "For men's top hoodie dropped shoulder shirt in extra large, Chest: 54, Body Length: 28, Sleeve Length: 18.5, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie dropped shoulder shirt in extra large?",
                "id": "77",
            }
        ]
    },
    {
        "context": "For men's top hoodie dropped shoulder shirt in double extra large, Chest: 56, Body Length: 28.5, Sleeve Length: 18.75, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie dropped shoulder shirt in double extra large?",
                "id": "78",
            }
        ]
    },
    {
        "context": "For men's top hoodie dropped shoulder shirt in triple extra large, Chest: 58, Body Length: 29, Sleeve Length: 19, USA: 24, EU: 58, and UK: 48.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie dropped shoulder shirt in triple extra large?",
                "id": "79",
            }
        ]
    },
    {
        "context": "For men's top hoodie set-in shoulder shirt in double extra small, Chest: 38, Body Length: 25, Sleeve Length: 24.25, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie set-in shoulder shirt in double extra small?",
                "id": "80",
            }
        ]
    },
    {
        "context": "For men's top hoodie set-in shoulder shirt in extra small, Chest: 40, Body Length: 26, Sleeve Length: 24.5, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie set-in shoulder shirt in extra small?",
                "id": "81",
            }
        ]
    },
    {
        "context": "For men's top hoodie set-in shoulder shirt in small, Chest: 42, Body Length: 27, Sleeve Length: 24.75, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie set-in shoulder shirt in small?",
                "id": "82",
            }
        ]
    },
    {
        "context": "For men's top hoodie set-in shoulder shirt in medium, Chest: 44, Body Length: 28, Sleeve Length: 25, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie set-in shoulder shirt in medium?",
                "id": "83",
            }
        ]
    },
    {
        "context": "For men's top hoodie set-in shoulder shirt in large, Chest: 46, Body Length: 29, Sleeve Length: 25.25, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie set-in shoulder shirt in large?",
                "id": "84",
            }
        ]
    },
    {
        "context": "For men's top hoodie set-in shoulder shirt in extra large, Chest: 48, Body Length: 30, Sleeve Length: 25.5, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie set-in shoulder shirt in extra large?",
                "id": "85",
            }
        ]
    },
    {
        "context": "For men's top hoodie set-in shoulder shirt in double extra large, Chest: 50, Body Length: 31, Sleeve Length: 25.75, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie set-in shoulder shirt in double extra large?",
                "id": "86",
            }
        ]
    },
    {
        "context": "For men's top hoodie set-in shoulder shirt in triple extra large, Chest: 52, Body Length: 32, Sleeve Length: 26, USA: 24, EU: 58, and UK: 48.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top hoodie set-in shoulder shirt in triple extra large?",
                "id": "87",
            }
        ]
    },
    {
        "context": "For men's top relax set-in pullover shirt in double extra small, Chest: 19, Body Length: 26, Sleeve Length: 24.25, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top relax set-in pullover shirt in double extra small?",
                "id": "88",
            }
        ]
    },
    {
        "context": "For men's top relax set-in pullover shirt in extra small, Chest: 20, Body Length: 27, Sleeve Length: 24.5, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top relax set-in pullover shirt in extra small?",
                "id": "89",
            }
        ]
    },
    {
        "context": "For men's top relax set-in pullover shirt in small, Chest: 21, Body Length: 27, Sleeve Length: 24.75, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top relax set-in pullover shirt in small?",
                "id": "90",
            }
        ]
    },
    {
        "context": "For men's top relax set-in pullover shirt in medium, Chest: 22, Body Length: 28, Sleeve Length: 25, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top relax set-in pullover shirt in medium?",
                "id": "91",
            }
        ]
    },
    {
        "context": "For men's top relax set-in pullover shirt in large, Chest: 23, Body Length: 29, Sleeve Length: 25.25, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top relax set-in pullover shirt in large?",
                "id": "92",
            }
        ]
    },
    {
        "context": "For men's top relax set-in pullover shirt in extra large, Chest: 24, Body Length: 30, Sleeve Length: 25.5, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top relax set-in pullover shirt in extra large?",
                "id": "93",
            }
        ]
    },
    {
        "context": "For men's top relax set-in pullover shirt in double extra large, Chest: 25, Body Length: 31, Sleeve Length: 25.75, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top relax set-in pullover shirt in double extra large?",
                "id": "94",
            }
        ]
    },
    {
        "context": "For men's tee easy fit shirt in double extra small, Chest: 18, Body Length: 26, Sleeve Length: 7.25, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee easy fit shirt in double extra small?",
                "id": "95",
            }
        ]
    },
    {
        "context": "For men's tee easy fit shirt in extra small, Chest: 19, Body Length: 26, Sleeve Length: 7.5, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee easy fit shirt in extra small?",
                "id": "96",
            }
        ]
    },
    {
        "context": "For men's tee easy fit shirt in small, Chest: 20, Body Length: 27, Sleeve Length: 7.75, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "id": "97",
                "question": "What are the size and dimension possibilities for men's tee easy fit shirt in small?",
            }
        ]
    },
    {
        "context": "For men's tee easy fit shirt in medium, Chest: 21, Body Length: 28, Sleeve Length: 8, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "id": "98",
                "question": "What are the size and dimension possibilities for men's tee easy fit shirt in medium?",
            }
        ]
    },
    {
        "context": "For men's tee easy fit shirt in large, Chest: 22, Body Length: 29, Sleeve Length: 8.25, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee easy fit shirt in large?",
                "id": "99",
            }
        ]
    },
    {
        "context": "For men's tee easy fit shirt in extra large, Chest: 23, Body Length: 30, Sleeve Length: 8.5, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee easy fit shirt in extra large?",
                "id": "100",
            }
        ]
    },
    {
        "context": "For men's tee easy fit shirt in double extra large, Chest: 24, Body Length: 31, Sleeve Length: 8.75, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee easy fit shirt in double extra large?",
                "id": "101",
            }
        ]
    },
    {
        "context": "For men's top jersey with raglan sleeves shirt in double extra small, Chest: 36, Body Length: 25, Sleeve Length: 13.5, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top jersey with raglan sleeves shirt in double extra small?",
                "id": "102",
            }
        ]
    },
    {
        "context": "For men's top jersey with raglan sleeves shirt in extra small, Chest: 38, Body Length: 26, Sleeve Length: 14, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top jersey with raglan sleeves shirt in extra small?",
                "id": "103",
            }
        ]
    },
    {
        "context": "For men's top jersey with raglan sleeves shirt in small, Chest: 40, Body Length: 27, Sleeve Length: 14.5, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top jersey with raglan sleeves shirt in small?",
                "id": "104",
            }
        ]
    },
    {
        "context": "For men's top jersey with raglan sleeves shirt in medium, Chest: 42, Body Length: 28, Sleeve Length: 15, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top jersey with raglan sleeves shirt in medium?",
                "id": "105",
            }
        ]
    },
    {
        "context": "For men's top jersey with raglan sleeves shirt in large, Chest: 44, Body Length: 29, Sleeve Length: 15.5, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top jersey with raglan sleeves shirt in large?",
                "id": "106",
            }
        ]
    },
    {
        "context": "For men's top jersey with raglan sleeves shirt in extra large, Chest: 46, Body Length: 30, Sleeve Length: 16, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top jersey with raglan sleeves shirt in extra large?",
                "id": "107",
            }
        ]
    },
    {
        "context": "For men's top jersey with raglan sleeves shirt in double extra large, Chest: 48, Body Length: 31, Sleeve Length: 16.5, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top jersey with raglan sleeves shirt in double extra large?",
                "id": "108",
            }
        ]
    },
    {
        "context": "For men's top jersey with raglan sleeves shirt in triple extra large, Chest: 50, Body Length: 32, Sleeve Length: 17, USA: 24, EU: 58, and UK: 48.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top jersey with raglan sleeves shirt in triple extra large",
                "id": "109",
            }
        ]
    },
    {
        "context": "For men's tee slim fit shirt in double extra small, Chest: 17, Body Length: 25.5, Sleeve Length: 6.75, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee slim fit shirt in double extra small?",
                "id": "110",
            }
        ]
    },
    {
        "context": "For men's tee slim fit shirt in extra small, Chest: 18, Body Length: 26, Sleeve Length: 7, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee slim fit shirt in extra small?",
                "id": "111",
            }
        ]
    },
    {
        "context": "For men's tee slim fit shirt in small, Chest: 19, Body Length: 26.5, Sleeve Length: 7.25, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee slim fit shirt in small?",
                "id": "112"
            }
        ]
    },
    {
        "context": "For men's tee slim fit shirt in medium, Chest: 20, Body Length: 27, Sleeve Length: 7.5, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee slim fit shirt in medium?",
                "id": "113",
            }
        ]
    },
    {
        "context": "For men's tee slim fit shirt in large, Chest: 21, Body Length: 27.5, Sleeve Length: 7.75, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee slim fit shirt in large?",
                "id": "114",
            }
        ]
    },
    {
        "context": "For men's tee slim fit shirt in extra large, Chest: 22, Body Length: 28, Sleeve Length: 8, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee slim fit shirt in extra large?",
                "id": "115",
            }
        ]
    },
    {
        "context": "For men's tee slim fit shirt in double extra large, Chest: 23, Body Length: 28.5, Sleeve Length: 8.25, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee slim fit shirt in double extra large?",
                "id": "116",
            }
        ]
    },
    {
        "context": "For men's tee oversized fit shirt in double extra small, Chest: 19, Body Length: 27, Sleeve Length: 8.5, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee oversized fit shirt in double extra small?",
                "id": "117"
            }
        ]
    },
    {
        "context": "For men's tee oversized fit shirt in extra small, Chest: 20, Body Length: 28, Sleeve Length: 8.75, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee oversized fit shirt in extra small?",
                "id": "118",
            }
        ]
    },
    {
        "context": "For men's tee oversized fit shirt in small, Chest: 21, Body Length: 29, Sleeve Length: 9, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee oversized fit shirt in small?",
                "id": "119",
            }
        ]
    },
    {
        "context": "For men's tee oversized fit shirt in medium, Chest: 22, Body Length: 30, Sleeve Length: 9.25, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee oversized fit shirt in medium?",
                "id": "120",
            }
        ]
    },
    {
        "context": "For men's tee oversized fit shirt in large, Chest: 23, Body Length: 31, Sleeve Length: 9.5, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee oversized fit shirt in large?",
                "id": "121",
            }
        ]
    },
    {
        "context": "For men's tee oversized fit shirt in extra large, Chest: 24, Body Length: 32, Sleeve Length: 9.75, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee oversized fit shirt in extra large?",
                "id": "122",
            }
        ]
    },
    {
        "context": "For men's tee oversized fit shirt in double extra large, Chest: 25, Body Length: 33, Sleeve Length: 10, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tee oversized fit shirt in double extra large?",
                "id": "123",
            }
        ]
    },
    {
        "context": "For men's top tank top shirt in double extra small, Chest: 36, Body Length: 26, Sleeve Length: N/A, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top tank top shirt in double extra small?",
                "id": "124",
            }
        ]
    },
    {
        "context": "For men's top tank top shirt in extra small, Chest: 38, Body Length: 27, Sleeve Length: N/A, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top tank top shirt in extra small?",
                "id": "125",
            }
        ]
    },
    {
        "context": "For men's top tank top shirt in small, Chest: 40, Body Length: 28, Sleeve Length: N/A, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top tank top shirt in small?",
                "id": "126",
            }
        ]
    },
    {
        "context": "For men's top tank top shirt in medium, Chest: 42, Body Length: 29, Sleeve Length: N/A, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top tank top shirt in medium?",
                "id": "127",
            }
        ]
    },
    {
        "context": "For men's top tank top shirt in large, Chest: 44, Body Length: 30, Sleeve Length: N/A, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top tank top shirt in large?",
                "id": "128",
            }
        ]
    },
    {
        "context": "For men's top tank top shirt in extra large, Chest: 46, Body Length: 31, Sleeve Length: N/A, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top tank top shirt in extra large?",
                "id": "129",
            }
        ]
    },
    {
        "context": "For men's top tank top shirt in double extra large, Chest: 48, Body Length: 32, Sleeve Length: N/A, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's top tank top shirt in double extra large?",
                "id": "130",
            }
        ]
    },
    {
        "context": "For men's jeans skinny fit (regular waist band) in size 29, Waist: 28, Hips: 34, Inseam: 28, USA: 38, EU: 38, and UK: 30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans skinny fit (regular waist band) in size 29?",
                "id": "131",
            }
        ]
    },
    {
        "context": "For men's jeans skinny fit (regular waist band) in size 30, Waist: 29, Hips: 35, Inseam: 28, USA: 39, EU: 39, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans skinny fit (regular waist band) in size 30?",
                "id": "132",
            }
        ]
    },
    {
        "context": "For men's jeans skinny fit (regular waist band) in size 31, Waist: 30, Hips: 36, Inseam: 28, USA: 40, EU: 40, and UK: 31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans skinny fit (regular waist band) in size 31?",
                "id": "133",
            }
        ]
    },
    {
        "context": "For men's jeans skinny fit (regular waist band) in size 32, Waist: 31, Hips: 37, Inseam: 29, USA: 42, EU: 42, and UK: 32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans skinny fit (regular waist band) in size 32?",
                "id": "134",
            }
        ]
    },
    {
        "context": "For men's jeans skinny fit (regular waist band) in size 34, Waist: 31, Hips: 37, Inseam: 29, USA: 42, EU: 42, and UK: 32.",
        "qas": [
            {
                "id": "135",
                "question": "What are the size and dimension possibilities for men's jeans skinny fit (regular waist band) in size 34?",
            }
        ]
    },
    {
        "context": "For men's jeans slim fit (regular waist band) in size 29, Waist: 27, Hips: 36, Inseam: 28, USA: 38, EU: 38, and UK: 30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans slim fit (regular waist band) in size 29?",
                "id": "136",
            }
        ]
    },
    {
        "context": "For men's jeans slim fit (regular waist band) in size 30, Waist: 28, Hips: 37, Inseam: 28, USA: 39, EU: 39, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans slim fit (regular waist band) in size 30?",
                "id": "137",
            }
        ]
    },
    {
        "context": "For men's jeans slim fit (regular waist band) in size 31, Waist: 29, Hips: 38, Inseam: 28, USA: 40, EU: 40, and UK: 31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans slim fit (regular waist band) in size 31?",
                "id": "138",
            }
        ]
    },
    {
        "context": "For men's jeans slim fit (regular waist band) in size 32, Waist: 30, Hips: 39, Inseam: 29, USA: 42, EU: 42, and UK: 32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans slim fit (regular waist band) in size 32?",
                "id": "139",
            }
        ]
    },
    {
        "context": "For men's jeans slim fit (regular waist band) in size 34, Waist: 32, Hips: 41, Inseam: 29, USA: 44, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans slim fit (regular waist band) in size 34?",
                "id": "140",
            }
        ]
    },
    {
        "context": "For men's jeans jogger pants (garterized waist band) in size extra small, Waist: 28, Hips: 38, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans jogger pants (garterized waist band) in size extra small?",
                "id": "141",
            }
        ]
    },
    {
        "context": "For men's jeans jogger pants (garterized waist band) in size small, Waist: 30, Hips: 40, Inseam: 25, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans jogger pants (garterized waist band) in size small?",
                "id": "142",
            }
        ]
    },
    {
        "context": "For men's jeans jogger pants (garterized waist band) in size medium, Waist: 32, Hips: 42, Inseam: 25, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans jogger pants (garterized waist band) in size medium?",
                "id": "143",
            }
        ]
    },
    {
        "context": "For men's jeans jogger pants (garterized waist band) in size large, Waist: 34, Hips: 44, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans jogger pants (garterized waist band) in size large?",
                "id": "144",
            }
        ]
    },
    {
        "context": "For men's jeans jogger pants (garterized waist band) in size extra large, Waist: 36, Hips: 46, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's jeans jogger pants (garterized waist band) in size extra large?",
                "id": "145",
            }
        ]
    },
    {
        "context": "For men's tapered pants woven (regular waist band) in size extra small, Waist: 24, Hips: 37, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants woven (regular waist band) in size extra small?",
                "id": "146",
            }
        ]
    },
    {
        "context": "For men's tapered pants woven (regular waist band) in size small, Waist: 26, Hips: 39, Inseam: 25, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants woven (regular waist band) in size small?",
                "id": "147",
            }
        ]
    },
    {
        "context": "For men's tapered pants woven (regular waist band) in size medium, Waist: 28, Hips: 41, Inseam: 25, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants woven (regular waist band) in size medium?",
                "id": "148",
            }
        ]
    },
    {
        "context": "For men's tapered pants woven (regular waist band) in size large, Waist: 30, Hips: 43, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants woven (regular waist band) in size large?",
                "id": "149",
            }
        ]
    },
    {
        "context": "For men's tapered pants woven (regular waist band) in size extra large, Waist: 32, Hips: 45, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants woven (regular waist band) in size extra large?",
                "id": "150",
            }
        ]
    },
    {
        "context": "For men's tapered pants knit (garterized waist band) in size extra small, Waist: 26, Hips: 38, Inseam: 26, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants knit (garterized waist band) in size extra small?",
                "id": "151",
            }
        ]
    },
    {
        "context": "For men's tapered pants knit (garterized waist band) in size small, Waist: 28, Hips: 40, Inseam: 26, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants knit (garterized waist band) in size small?",
                "id": "152",
            }
        ]
    },
    {
        "context": "For men's tapered pants knit (garterized waist band) in size medium, Waist: 30, Hips: 42, Inseam: 26, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants knit (garterized waist band) in size medium?",
                "id": "153",
            }
        ]
    },
    {
        "context": "For men's tapered pants knit (garterized waist band) in size large, Waist: 32, Hips: 44, Inseam: 26, USA: 43-44, EU: 43-44, and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants knit (garterized waist band) in size large?",
                "id": "154",
            }
        ]
    },
    {
        "context": "For men's tapered pants knit (garterized waist band) in size extra large, Waist: 34, Hips: 46, Inseam: 26, USA: 45-46, EU: 45-46, and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants knit (garterized waist band) in size extra large?",
                "id": "155",
            }
        ]
    },
    {
        "context": "For men's tapered pants with cuff-knit (garterized waist band) in size double extra small, Waist: 23, Hips: 36, Inseam: 25, USA: 32-34, EU: 32-34, and UK: 27-28.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants with cuff-knit (garterized waist band) in size double extra small?",
                "id": "156",
            }
        ]
    },
    {
        "context": "For men's tapered pants with cuff-knit (garterized waist band) in size extra small, Waist: 25, Hips: 38, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants with cuff-knit (garterized waist band) in size extra small?",
                "id": "157",
            }
        ]
    },
    {
        "context": "For men's tapered pants with cuff-knit (garterized waist band) in size small, Waist: 27, Hips: 40, Inseam: 25, USA: 39-40, EU: 39-40 and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants with cuff-knit (garterized waist band) in size small?",
                "id": "158",
            }
        ]
    },
    {
        "context": "For men's tapered pants with cuff-knit (garterized waist band) in size medium, Waist: 29, Hips: 42, Inseam: 25, USA: 43-44, EU: 43-44 and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants with cuff-knit (garterized waist band) in size medium?",
                "id": "159",
            }
        ]
    },
    {
        "context": "For men's tapered pants with cuff-knit (garterized waist band) in size large, Waist: 31, Hips: 44, Inseam: 25, USA: 43-44, EU: 43-44 and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants with cuff-knit (garterized waist band) in size large?",
                "id": "160",
            }
        ]
    },
    {
        "context": "For men's tapered pants with cuff-knit (garterized waist band) in size extra large, Waist: 33, Hips: 46, Inseam: 25, USA: 45-46, EU: 45-46 and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants with cuff-knit (garterized waist band) in size extra large?",
                "id": "161",
            }
        ]
    },
    {
        "context": "For men's tapered pants with cuff-knit (garterized waist band) in size double extra large, Waist: 35, Hips: 48, Inseam: 25, USA: 48-50, EU: 48-50 and UK: 37-38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's tapered pants with cuff-knit (garterized waist band) in size double extra large?",
                "id": "162",
            }
        ]
    },
    {
        "context": "For men's urban shorts - knit & woven (garterized waist band) in size extra small, Waist: 28, Hips: 40, Inseam: 5, USA: 36-38, EU: 36-38 and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's urban shorts - knit & woven (garterized waist band) in size extra small?",
                "id": "163",
            }
        ]
    },
    {
        "context": "For men's urban shorts - knit & woven (garterized waist band) in size small, Waist: 30, Hips: 42, Inseam: 5, USA: 39-40, EU: 39-40 and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's urban shorts - knit & woven (garterized waist band) in size small?",
                "id": "164",
            }
        ]
    },
    {
        "context": "For men's urban shorts - knit & woven (garterized waist band) in size medium, Waist: 32, Hips: 44, Inseam: 5, USA: 40-42, EU: 40-42 and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's urban shorts - knit & woven (garterized waist band) in size medium",
                "id": "165",
            }
        ]
    },
    {
        "context": "For men's urban shorts - knit & woven (garterized waist band) in size large, Waist: 34, Hips: 46, Inseam: 5, USA: 43-44, EU: 43-44 and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's urban shorts - knit & woven (garterized waist band) in size large?",
                "id": "166",
            }
        ]
    },
    {
        "context": "For men's urban shorts - knit & woven (garterized waist band) in size extra large, Waist: 36, Hips: 48, Inseam: 5, USA: 45-46, EU: 45-46 and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's urban shorts - knit & woven (garterized waist band) in size extra large?",
                "id": "167",
            }
        ]
    },
    {
        "context": "For men's chino shorts - woven (regular waist band) in size 28, Waist: 28, Hips: 38, Inseam: 5, USA: 36, EU: 36, and UK: 29.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's chino shorts - woven (regular waist band) in size 28?",
                "id": "168",
            }
        ]
    },
    {
        "context": "For men's chino shorts - woven (regular waist band) in size 29, Waist: 29, Hips: 39, Inseam: 5, USA: 38, EU: 38, and UK: 30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's chino shorts - woven (regular waist band) in size 29?",
                "id": "169",
            }
        ]
    },
    {
        "context": "For men's chino shorts - woven (regular waist band) in size 30, Waist: 30, Hips: 40, Inseam: 5, USA: 39, EU: 39, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's chino shorts - woven (regular waist band) in size 30?",
                "id": "170",
            }
        ]
    },
    {
        "context": "For men's chino shorts - woven (regular waist band) in size 31, Waist: 31, Hips: 41, Inseam: 5, USA: 40, EU: 40, and UK: 31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's chino shorts - woven (regular waist band) in size 31",
                "id": "171",
            }
        ]
    },
    {
        "context": "For men's chino shorts - woven (regular waist band) in size 32, Waist: 32, Hips: 42, Inseam: 5, USA: 42, EU: 42, and UK: 32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's chino shorts - woven (regular waist band) in size 32?",
                "id": "172",
            }
        ]
    },
    {
        "context": "For men's chino shorts - woven (regular waist band) in size 34, Waist: 34, Hips: 44, Inseam: 5, USA: 44, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's chino shorts - woven (regular waist band) in size 34?",
                "id": "173",
            }
        ]
    },
    {
        "context": "For men's walk/board shorts - woven (garterized waist band) in size extra small, Waist: 28, Hips: 41, Inseam: 4.5, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's walk/board shorts - woven (garterized waist band) in size extra small?",
                "id": "174",
            }
        ]
    },
    {
        "context": "For men's walk/board shorts - woven (garterized waist band) in size small, Waist: 30, Hips: 43, Inseam: 4.5, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's walk/board shorts - woven (garterized waist band) in size small?",
                "id": "175",
            }
        ]
    },
    {
        "context": "For men's walk/board shorts - woven (garterized waist band) in size medium, Waist: 32, Hips: 45, Inseam: 4.5, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's walk/board shorts - woven (garterized waist band) in size medium?",
                "id": "176",
            }
        ]
    },
    {
        "context": "For men's walk/board shorts - woven (garterized waist band) in size large, Waist: 34, Hips: 47, Inseam: 4.5, USA: 43-44, EU: 43-44, and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's walk/board shorts - woven (garterized waist band) in size large?",
                "id": "177",
            }
        ]
    },
    {
        "context": "For men's walk/board shorts - woven (garterized waist band) in size extra large, Waist: 36, Hips: 49, Inseam: 4.5, USA: 45-46, EU: 45-46, and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's walk/board shorts - woven (garterized waist band) in size extra large?",
                "id": "178",
            }
        ]
    },
    {
        "context": "For men's boxer shorts (garterized waist band) in size double extra small, Waist: 20, Hips: 35, Inseam: 3.25, USA: 32, EU: 42, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's boxer shorts (garterized waist band) in size double extra small?",
                "id": "179",
            }
        ]
    },
    {
        "context": "For men's boxer shorts (garterized waist band) in size extra small, Waist: 22, Hips: 36, Inseam: 3.25, USA: 34, EU: 44, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's boxer shorts (garterized waist band) in size extra small?",
                "id": "180",
            }
        ]
    },
    {
        "context": "For men's boxer shorts (garterized waist band) in size small, Waist: 24, Hips: 38, Inseam: 3.5, USA: 36, EU: 46, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's boxer shorts (garterized waist band) in size small?",
                "id": "181",
            }
        ]
    },
    {
        "context": "For men's boxer shorts (garterized waist band) in size medium, Waist: 26, Hips: 40, Inseam: 3.5, USA: 38, EU: 48, and UK: 14.",
        "qas": [
            {
                "id": "182",
                "question": "What are the size and dimension possibilities for men's boxer shorts (garterized waist band) in size medium?",
            }
        ]
    },
    {
        "context": "For men's boxer shorts (garterized waist band) in size large, Waist: 28, Hips: 42, Inseam: 3.75, USA: 40, EU: 50, and UK: 16.",
        "qas": [
            {
                "id": "183",
                "question": "What are the size and dimension possibilities for men's boxer shorts (garterized waist band) in size large?",
            }
        ]
    },
    {
        "context": "For men's boxer shorts (garterized waist band) in size extra large, Waist: 30, Hips: 44, Inseam: 3.75, USA: 42, EU: 52, and UK: 18.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's boxer shorts (garterized waist band) in size extra large?",
                "id": "184",
            }
        ]
    },
    {
        "context": "For men's boxer shorts (garterized waist band) in size double extra large, Waist: 32, Hips: 46, Inseam: 3.75, USA: 44, EU: 54, and UK: 20.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's boxer shorts (garterized waist band) in size double extra large?",
                "id": "185",
            }
        ]
    },
    {
        "context": "For men's lounge - robe in size small, Waist: 44, Hips: 40, Inseam: 19.5, USA: 12-16, EU: 46-50, and UK: 36-40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge - robe in size small?",
                "id": "186",
            }
        ]
    },
    {
        "context": "For men's lounge - robe in size medium, Waist: 48, Hips: 42, Inseam: 19.5, USA: 18-20, EU: 52-54, and UK: 42-44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge - robe in size medium?",
                "id": "187",
            }
        ]
    },
    {
        "context": "For men's lounge set - top slim fit in size double extra small, Waist: 34, Hips: 25.5, Inseam: 6.75, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - top slim fit in size double extra small?",
                "id": "188",
            }
        ]
    },
    {
        "context": "For men's lounge set - top slim fit in size extra small, Waist: 36, Hips: 26, Inseam: 7, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - top slim fit in size extra small?",
                "id": "189",
            }
        ]
    },
    {
        "context": "For men's lounge set - top slim fit in size small, Waist: 38, Hips: 26.5, Inseam: 7.25, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - top slim fit in size small?",
                "id": "190",
            }
        ]
    },
    {
        "context": "For men's lounge set - top slim fit in size medium, Waist: 40, Hips: 27, Inseam: 7.5, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - top slim fit in size medium?",
                "id": "191",
            }
        ]
    },
    {
        "context": "For men's lounge set - top slim fit in size large, Waist: 42, Hips: 27.5, Inseam: 7.75, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - top slim fit in size large?",
                "id": "192",
            }
        ]
    },
    {
        "context": "For men's lounge set - top slim fit in size extra large, Waist: 44, Hips: 28, Inseam: 8, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - top slim fit in size extra large?",
                "id": "193"
            }
        ]
    },
    {
        "context": "For men's lounge set - top slim fit in size double extra large, Waist: 46, Hips: 28.5, Inseam: 8.25, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - top slim fit in size double extra large?",
                "id": "194",
            }
        ]
    },
    {
        "context": "For men's lounge set - bottom boxer shorts in size double extra small, Waist: 20, Hips: 36, Inseam: 3.25, USA: 33-35, EU: 33-35, and UK: 27-28.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - bottom boxer shorts in size double extra small?",
                "id": "195",
            }
        ]
    },
    {
        "context": "For men's lounge set - bottom boxer shorts in size extra small, Waist: 22, Hips: 38, Inseam: 3.25, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - bottom boxer shorts in size extra small?",
                "id": "196",
            }
        ]
    },
    {
        "context": "For men's lounge set - bottom boxer shorts in size small, Waist: 24, Hips: 40, Inseam: 3.5, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - bottom boxer shorts in size small?",
                "id": "197",
            }
        ]
    },
    {
        "context": "For men's lounge set - bottom boxer shorts in size medium, Waist: 26, Hips: 42, Inseam: 3.5, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - bottom boxer shorts in size medium?",
                "id": "198",
            }
        ]
    },
    {
        "context": "For men's lounge set - bottom boxer shorts in size large, Waist: 28, Hips: 44, Inseam: 3.75, USA: 43-44, EU: 43-44, and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - bottom boxer shorts in size large?",
                "id": "199",
            }
        ]
    },
    {
        "context": "For men's lounge set - bottom boxer shorts in size extra large, Waist: 30, Hips: 46, Inseam: 3.75, USA: 45-46, EU: 45-46, and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - bottom boxer shorts in size extra large?",
                "id": "200",
            }
        ]
    },
    {
        "context": "For men's lounge set - bottom boxer shorts in size double extra large, Waist: 32, Hips: 48, Inseam: 3.75, USA: 47-48, EU: 47-48, and UK: 37-38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lounge set - bottom boxer shorts in size double extra large?",
                "id": "201",
            }
        ]
    }, 
    {
        "context": "For men's footwear sliders unisex in size small, Ladies' US Size: 5, Men's US Size: N/A, Footbed Length: 23.5cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's footwear sliders unisex in size small?",
                "id": "202",
            }
        ]
    }, 
    {
        "context": "For men's footwear sliders unisex in size medium, Ladies' US Size: 6-7, Men's US Size: N/A, Footbed Length: 24.9cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's footwear sliders unisex in size medium?",
                "id": "203",
            }
        ]
    },
    {
        "context": "For men's footwear sliders unisex in size large, Ladies' US Size: 8-9, Men's US Size: 6, Footbed Length: 26.3cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's footwear sliders unisex in size large?",
                "id": "204",
            }
        ]
    },
    {
        "context": "For men's footwear sliders unisex in size extra large, Ladies' US Size: N/A, Men's US Size: 7-8, Footbed Length: 27.5cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's footwear sliders unisex in size extra large?",
                "id": "205",
            }
        ]
    },
    {
        "context": "For men's footwear sliders unisex in size double extra large, Ladies' US Size: N/A, Men's US Size: 9-10, Footbed Length: 28.9cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's footwear sliders unisex in size double extra large?",
                "id": "206",
            }
        ]
    },
    {
        "context": "For men's footwear single band sliders, (US: 6, EU: 39, CM: 26.1), (US: 7, EU: 40, CM: 26.8), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28.2), (US: 10, EU: 43, CM: 28.9).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's footwear single band sliders?",
                "id": "207",
            }
        ]
    },
    {
        "context": "For men's footwear velcro sliders, (US: 6, EU: 39, CM: 26.1), (US: 7, EU: 40, CM: 26.8), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28.2), (US: 10, EU: 43, CM: 28.5).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's footwear velcro sliders?",
                "id": "208",
            }
        ]
    },
    {
        "context": "For men's regular flipflops, (US: 6, EU: 39, CM: 26.6), (US: 7, EU: 40, CM: 27.3), (US: 8, EU: 41, CM: 28), (US: 9, EU: 42, CM: 28.7), (US: 10, EU: 43, CM: 29.4), then the slippers thicknes is 15mm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's regular flipflops?",
                "id": "209",
            }
        ]
    },
    {
        "context": "For men's lace-up sneakers, (US: 6, EU: 39, CM: 26.5), (US: 7, EU: 40, CM: 27), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28), (US: 10, EU: 43, CM: 28.5).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lace-up sneakers?",
                "id": "210",
            }
        ]
    },
    {
        "context": "For men's lace-up runner shoes, (US: 6, EU: 39, CM: 26.5), (US: 7, EU: 40, CM: 27), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28), (US: 10, EU: 43, CM: 28.5).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's lace-up runner shoes?",
                "id": "211",
            }
        ]
    },
    {
        "context": "For ladies' top tee - regular fit in size extra small, Chest: 34, Body Length: 22.5, Sleeve Length: 7, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - regular fit in size extra small?",
                "id": "212",
            }
        ]
    },
    {
        "context": "For ladies' top tee - regular fit in size small, Chest: 36, Body Length: 23, Sleeve Length: 7.25, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - regular fit in size small?",
                "id": "213",
            }
        ]
    },
    {
        "context": "For ladies' top tee - regular fit in size medium, Chest: 38, Body Length: 23.5, Sleeve Length: 7.5, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - regular fit in size medium?",
                "id": "214",
            }
        ]
    },
    {
        "context": "For ladies' top tee - regular fit in size large, Chest: 40, Body Length: 24, Sleeve Length: 7.75, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - regular fit in size large?",
                "id": "215",
            }
        ]
    },
    {
        "context": "For ladies' top tee - regular fit in size extra large, Chest: 42, Body Length: 24.5, Sleeve Length: 8, USA: 10, EU: 42, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - regular fit in size extra large?",
                "id": "216",
            }
        ]
    },
    {
        "context": "For ladies' top tee - boxy fit in size extra small, Chest: 39, Body Length: 19, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - boxy fit in size extra small?",
                "id": "217",
            }
        ]
    },
    {
        "context": "For ladies' top tee - boxy fit in size small, Chest: 41, Body Length: 19.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - boxy fit in size small?",
                "id": "218",
            }
        ]
    },
    {
        "context": "For ladies' top tee - boxy fit in size medium, Chest: 43, Body Length: 20, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "id": "219",
                "question": "What are the size and dimension possibilities for ladies' top tee - boxy fit in medium?",
            }
        ]
    },
    {
        "context": "For ladies' top tee - boxy fit in size large, Chest: 45, Body Length: 20.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - boxy fit in size large?",
                "id": "220"
            }
        ]
    },
    {
        "context": "For ladies' top tee - boxy fit in size extra large, Chest: 47, Body Length: 21, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee - boxy fit in size extra large?",
                "id": "221",
            }
        ]
    },
    {
        "context": "For ladies' v-neck - slim fit in size extra small, Chest: 31, Body Length: 22.5, Sleeve Length: 7.25, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' v-neck - slim fit in size extra small?",
                "id": "222",
            }
        ]
    },
    {
        "context": "For ladies' v-neck - slim fit in size small, Chest: 33, Body Length: 23, Sleeve Length: 7.5, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' v-neck - slim fit in size small?",
                "id": "223",
            }
        ]
    },
    {
        "context": "For ladies' v-neck - slim fit in size medium, Chest: 35, Body Length: 23.5, Sleeve Length: 7.75, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' v-neck - slim fit in size medium?",
                "id": "224",
            }
        ]
    },
    {
        "context": "For ladies' v-neck - slim fit in size large, Chest: 37, Body Length: 24, Sleeve Length: 8, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' v-neck - slim fit in size large?",
                "id": "225",
            }
        ]
    },
    {
        "context": "For ladies' v-neck - slim fit in size extra large, Chest: 39, Body Length: 24.5, Sleeve Length: 8.25, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' v-neck - slim fit in size extra large?",
                "id": "226",
            }
        ]
    },
    {
        "context": "For ladies' rib slim fit tee in size extra small, Chest: 28, Body Length: 22.5, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' rib slim fit tee in size extra small?",
                "id": "227",
            }
        ]
    },
    {
        "context": "For ladies' rib slim fit tee in size small, Chest: 30, Body Length: 23, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' rib slim fit tee in size small?",
                "id": "228",
            }
        ]
    },
    {
        "context": "For ladies' rib slim fit tee in size medium, Chest: 30, Body Length: 23, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' rib slim fit tee in size medium?",
                "id": "229",
            }
        ]
    },
    {
        "context": "For ladies' rib slim fit tee in size large, Chest: 34, Body Length: 24, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' rib slim fit tee in size large?",
                "id": "230",
            }
        ]
    },
    {
        "context": "For ladies' rib slim fit tee in size extra large, Chest: 36, Body Length: 24.5, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' rib slim fit tee in size extra large?",
                "id": "231",
            }
        ]
    },
    {
        "context": "For ladies' slim fit bundle tees in size extra small, Chest: 30, Body Length: 23, Sleeve Length: 4.75, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' slim fit bundle tees in size extra small?",
                "id": "232",
            }
        ]
    },
    {
        "context": "For ladies' slim fit bundle tees in size small, Chest: 32, Body Length: 23.5, Sleeve Length: 5, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' slim fit bundle tees in size small?",
                "id": "233",
            }
        ]
    },
    {
        "context": "For ladies' slim fit bundle tees in size medium, Chest: 34, Body Length: 24, Sleeve Length: 5.25, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' slim fit bundle tees in size medium?",
                "id": "234",
            }
        ]
    },
    {
        "context": "For ladies' slim fit bundle tees in size large, Chest: 36, Body Length: 24.5, Sleeve Length: 5.5, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' slim fit bundle tees in size large?",
                "id": "235",
            }
        ]
    },
    {
        "context": "For ladies' slim fit bundle tees in size extra large, Chest: 38, Body Length: 25, Sleeve Length: 5.75, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' slim fit bundle tees in size extra large?",
                "id": "236",
            }
        ]
    },
    {
        "context": "For ladies' top tee oversized fit in size extra small, Chest: 38, Body Length: 24, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee oversized fit in size extra small?",
                "id": "237",
            }
        ]
    },
    {
        "context": "For ladies' top tee oversized fit in size small, Chest: 40, Body Length: 24.5 , Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee oversized fit in size small?",
                "id": "238",
            }
        ]
    },
    {
        "context": "For ladies' top tee oversized fit in size medium, Chest: 42, Body Length: 25 , Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee oversized fit in size medium?",
                "id": "239",
            }
        ]
    },
    {
        "context": "For ladies' top tee oversized fit in size large, Chest: 44, Body Length: 25.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee oversized fit in size large?",
                "id": "240",
            }
        ]
    },
    {
        "context": "For ladies' top tee oversized fit in size extra large, Chest: 46, Body Length: 26, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top tee oversized fit in size extra large?",
                "id": "241",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - short sleeves in size extra small, Chest: 40, Body Length: 20.5, Sleeve Length: 7.5, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - short sleeves in size extra small?",
                "id": "242",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - short sleeves in size small, Chest: 42, Body Length: 21, Sleeve Length: 7.75, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - short sleeves in size small?",
                "id": "243",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - short sleeves in size medium, Chest: 44, Body Length: 21.5, Sleeve Length: 8, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - short sleeves in size medium?",
                "id": "244",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - short sleeves in size large, Chest: 46, Body Length: 22, Sleeve Length: 8.25, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - short sleeves in size large?",
                "id": "245",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - short sleeves in size extra large, Chest: 48, Body Length: 22.5, Sleeve Length: 8.5, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - short sleeves in size extra large?",
                "id": "246",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - long sleeves in size extra small, Chest: 40, Body Length: 27.5, Sleeve Length: 20.5, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - long sleeves in size extra small?",
                "id": "247",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - long sleeves in size small, Chest: 42, Body Length: 28, Sleeve Length: 20.75, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - long sleeves in size small?",
                "id": "248",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - long sleeves in size medium, Chest: 44, Body Length: 28.5, Sleeve Length: 21, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - long sleeves in size medium?",
                "id": "249",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - long sleeves in size large, Chest: 46, Body Length: 29, Sleeve Length: 21.25, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - long sleeves in size large?",
                "id": "250",
            }
        ]
    },
    {
        "context": "For ladies' top shirt - long sleeves in size extra large, Chest: 48, Body Length: 29.5, Sleeve Length: 21.5, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top shirt - long sleeves in size extra large?",
                "id": "251",
            }
        ]
    },
    {
        "context": "For ladies' top - sports bra in size extra small, Chest: 28, Body Length: 9, Sleeve Length: N/A, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - sports bra in size extra small?",
                "id": "252",
            }
        ]
    },
    {
        "context": "For ladies' top - sports bra in size small, Chest: 30, Body Length: 9.5, Sleeve Length: N/A, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - sports bra in size small?",
                "id": "253",
            }
        ]
    },
    {
        "context": "For ladies' top - sports bra in size medium, Chest: 32, Body Length: 10, Sleeve Length: N/A, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - sports bra in size medium?",
                "id": "254",
            }
        ]
    },
    {
        "context": "For ladies' top - sports bra in size large, Chest: 34, Body Length: 10.5, Sleeve Length: N/A, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - sports bra in size large?",
                "id": "255",
            }
        ]
    },
    {
        "context": "For ladies' top - sports bra in size extra large, Chest: 36, Body Length: 11, Sleeve Length: N/A, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - sports bra in size extra large?",
                "id": "256",
            }
        ]
    },
    {
        "context": "For ladies' top - hoodie in size extra small, Chest: 44, Body Length: 24.5, Sleeve Length: 17.5, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - hoodie in size extra small?",
                "id": "257",
            }
        ]
    },
    {
        "context": "For ladies' top - hoodie in size small, Chest: 46, Body Length: 25, Sleeve Length: 17.75, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - hoodie in size small?",
                "id": "258"
            }
        ]
    },
    {
        "context": "For ladies' top - hoodie in size medium, Chest: 48, Body Length: 25.5, Sleeve Length: 18, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - hoodie in size medium?",
                "id": "259",
            }
        ]
    },
    {
        "context": "For ladies' top - hoodie in size large, Chest: 50, Body Length: 26, Sleeve Length: 18.25, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - hoodie in size large?",
                "id": "260",
            }
        ]
    },
    {
        "context": "For ladies' top - hoodie in size extra large, Chest: 52, Body Length: 26.5, Sleeve Length: 18.5, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - hoodie in size extra large?",
                "id": "261",
            }
        ]
    },
    {
        "context": "For ladies' top - crop pullover in size extra small, Chest: 36, Body Length: 17.5, Sleeve Length: 18.75, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - crop pullover in size extra small?",
                "id": "262",
            }
        ]
    },
    {
        "context": "For ladies' top - crop pullover in size small, Chest: 38, Body Length: 18, Sleeve Length: 19, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - crop pullover in size small?",
                "id": "263",
            }
        ]
    },
    {
        "context": "For ladies' top - crop pullover in size medium, Chest: 40, Body Length: 18.5, Sleeve Length: 19.25, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "id": "264",
                "question": "What are the size and dimension possibilities for ladies' top - crop pullover in size medium?",
            }
        ]
    },
    {
        "context": "For ladies' top - crop pullover in size large, Chest: 42, Body Length: 19, Sleeve Length: 19.5, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - crop pullover in size large?",
                "id": "265",
            }
        ]
    },
    {
        "context": "For ladies' top - crop pullover in size extra large, Chest: 44, Body Length: 19.5, Sleeve Length: 19.75, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - crop pullover in size extra large?",
                "id": "266",
            }
        ]
    },
    {
        "context": "For ladies' top - regular pullover in size extra small, Chest: 38, Body Length: 25.5, Sleeve Length: 20.75, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - regular pullover in size extra small?",
                "id": "267",
            }
        ]
    },
    {
        "context": "For ladies' top - regular pullover in size small, Chest: 40, Body Length: 26, Sleeve Length: 21, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - regular pullover in size small?",
                "id": "268",
            }
        ]
    },
    {
        "context": "For ladies' top - regular pullover in size medium, Chest: 42, Body Length: 26.5, Sleeve Length: 21.25, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - regular pullover in size medium?",
                "id": "269",
            }
        ]
    },
    {
        "context": "For ladies' top - regular pullover in size large, Chest: 44, Body Length: 27, Sleeve Length: 21.5, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - regular pullover in size large?",
                "id": "270",
            }
        ]
    },
    {
        "context": "For ladies' top - regular pullover in size extra large, Chest: 46, Body Length: 27.5, Sleeve Length: 21.75, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' top - regular pullover in size extra large?",
                "id": "271",
            }
        ]
    },
    {
        "context": "For ladies' dress - regular fit in size extra small, Chest: 33, Body Length: 32.5, Sleeve Length: 7, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - regular fit in size extra small?",
                "id": "272",
            }
        ]
    },
    {
        "context": "For ladies' dress - regular fit in size small, Chest: 35, Body Length: 33, Sleeve Length: 7.25, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - regular fit in size small?",
                "id": "273"
            }
        ]
    },
    {
        "context": "For ladies' dress - regular fit in size medium, Chest: 37, Body Length: 33.5, Sleeve Length: 7.5, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - regular fit in size medium?",
                "id": "274",
            }
        ]
    },
    {
        "context": "For ladies' dress - regular fit in size large, Chest: 39, Body Length: 34, Sleeve Length: 7.75, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - regular fit in size large?",
                "id": "275",
            }
        ] 
    },
    {
        "context": "For ladies' dress - regular fit in size extra large, Chest: 41, Body Length: 34.5, Sleeve Length: 8, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - regular fit in size extra large?",
                "id": "276",
            }
        ]
    },
    {
        "context": "For ladies' dress - oversized fit in size extra small, Chest: 38, Body Length: 33, Sleeve Length: 8.5, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - oversized fit in size extra small?",
                "id": "277",
            }
        ]
    },
    {
        "context": "For ladies' dress - oversized fit in size small, Chest: 40, Body Length: 33.5, Sleeve Length: 8.75, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - oversized fit in size small?",
                "id": "278",
            }
        ]
    },
    {
        "context": "For ladies' dress - oversized fit in size medium, Chest: 42, Body Length: 34, Sleeve Length: 9, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - oversized fit in size medium?",
                "id": "279",
            }
        ]
    },
    {
        "context": "For ladies' dress - oversized fit in size large, Chest: 44, Body Length: 34.5, Sleeve Length: 9.25, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - oversized fit in size large?",
                "id": "280",
            }
        ]
    },
    {
        "context": "For ladies' dress - oversized fit in size extra large, Chest: 46, Body Length: 35, Sleeve Length: 9.5, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - oversized fit in size extra large?",
                "id": "281",
            }
        ]
    },
    {
        "context": "For ladies' dress - long sleeved regular fit in size extra small, Chest: 38, Body Length: 32, Sleeve Length: 20, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - long sleeved regular fit in size extra small?",
                "id": "282",
            }
        ]
    },
    {
        "context": "For ladies' dress - long sleeved regular fit in size small, Chest: 40, Body Length: 32.5, Sleeve Length: 20.25, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - long sleeved regular fit in size small?",
                "id": "283",
            }
        ]
    },
    {
        "context": "For ladies' dress - long sleeved regular fit in size medium, Chest: 42, Body Length: 33, Sleeve Length: 20.5, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - long sleeved regular fit in size medium?",
                "id": "284",
            }
        ]
    },
    {
        "context": "For ladies' dress - long sleeved regular fit in size large, Chest: 44, Body Length: 33.5, Sleeve Length: 20.75, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - long sleeved regular fit in size large?",
                "id": "285",
            }
        ]
    },
    {
        "context": "For ladies' dress - long sleeved regular fit in size extra large, Chest: 46, Body Length: 34, Sleeve Length: 21, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - long sleeved regular fit in size extra large?",
                "id": "286",
            }
        ]
    },
    {
        "context": "For ladies' dress - baby doll dress in size extra small, Chest: 33, Body Length: 31, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - baby doll dress in size extra small?",
                "id": "287",
            }
        ]
    },
    {
        "context": "For ladies' dress - baby doll dress in size small, Chest: 35, Body Length: 31.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - baby doll dress in size small?",
                "id": "288",
            }
        ]
    },
    {
        "context": "For ladies' dress - baby doll dress in size medium, Chest: 37, Body Length: 32, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "id": "289",
                "question": "What are the size and dimension possibilities for ladies' dress - baby doll dress in size medium?",
            }
        ]
    },
    {
        "context": "For ladies' dress - baby doll dress in size large, Chest: 39, Body Length: 32.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - baby doll dress in size large?",
                "id": "290",
            }
        ]
    },
    {
        "context": "For ladies' dress - baby doll dress in size extra large, Chest: 41, Body Length: 33, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' dress - baby doll dress in size extra large?",
                "id": "291",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (numerical sizing) in size 26, Waist: 26, Hip: 32,  Inseam: 26, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (numerical sizing) in size 26?",
                "id": "292",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (numerical sizing) in size 27, Waist: 27, Hip: 33,  Inseam: 26, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (numerical sizing) in size 27?",
                "id": "293",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (numerical sizing) in size 28, Waist: 28, Hip: 34,  Inseam: 26, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "id": "294",
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (numerical sizing) in size 28?",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (numerical sizing) in size 29, Waist: 29, Hip: 35,  Inseam: 26, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "id": "295",
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (numerical sizing) in size 29?",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (numerical sizing) in size 30, Waist: 30, Hip: 36,  Inseam: 26, USA: 10, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (numerical sizing) in size 30?",
                "id": "296",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (numerical sizing) in size 32, Waist: 32, Hip: 38,  Inseam: 26, USA: 12, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (numerical sizing) in size 32?",
                "id": "297",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (alpha sizing) in size extra small, Waist: 24, Hip: 36,  Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (alpha sizing) in size extra small?",
                "id": "298",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (alpha sizing) in size small, Waist: 26, Hip: 38, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (alpha sizing) in size small?",
                "id": "299",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (alpha sizing) in size medium, Waist: 28, Hip: 40, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (alpha sizing) in size medium?",
                "id": "300",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (alpha sizing) in size large, Waist: 30, Hip: 42, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (alpha sizing) in size large?",
                "id": "301",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - skinny jeans (alpha sizing) in size extra large, Waist: 32, Hip: 44, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - skinny jeans (alpha sizing) in size extra large?",
                "id": "302",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - basic straight jeans (numerical sizing) in size 26?",
                "id": "303",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - basic straight jeans (numerical sizing) in size 27?",
                "id": "304",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - basic straight jeans (numerical sizing) in size 28?",
                "id": "305",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 29, Waist: 39, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - basic straight jeans (numerical sizing) in size 29?",
                "id": "306",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - basic straight jeans (numerical sizing) in size 30?",
                "id": "307",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - basic straight jeans (numerical sizing) in size 32?",
                "id": "308",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (alpha sizing) in size extra small?",
                "id": "309",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (alpha sizing) in size small?",
                "id": "310",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (alpha sizing) in size medium?",
                "id": "311",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (alpha sizing) in size large?",
                "id": "312",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (alpha sizing) in size extra large?",
                "id": "313",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 26, Waist: 25, Hip: 36.5, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (numeiracal sizing) in size 26?",
                "id": "314",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 27, Waist: 26, Hip: 37.5, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (numeiracal sizing) in size 27?",
                "id": "315"
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 28, Waist: 27, Hip: 38.5, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (numeiracal sizing) in size 28?",
                "id": "316",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 29, Waist: 28, Hip: 39.5, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (numeiracal sizing) in size 29?",
                "id": "317",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 30, Waist: 29, Hip: 40.5, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (numeiracal sizing) in size 30?",
                "id": "318",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 32, Waist: 31, Hip: 42.5, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - wide leg jeans (numeiracal sizing) in size 32?",
                "id": "319",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (alpha sizing) in size extra small, Waist: 26, Hip: 36, Inseam: 2.5, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (alpha sizing) in size extra small?",
                "id": "320",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (alpha sizing) in size small, Waist: 28, Hip: 38, Inseam: 2.5, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (alpha sizing) in size small?",
                "id": "321",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (alpha sizing) in size medium, Waist: 30, Hip: 40, Inseam: 2.5, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (alpha sizing) in size medium?",
                "id": "322",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (alpha sizing) in size large, Waist: 32, Hip: 42, Inseam: 2.5, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (alpha sizing) in size large?",
                "id": "323",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (alpha sizing) in size extra large, Waist: 34, Hip: 44, Inseam: 2.5, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (alpha sizing) in size extra large?",
                "id": "324",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 2.5, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (numerical sizing) in size 26?",
                "id": "325",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 2.5, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (numerical sizing) in size 27?",
                "id": "326",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 2.5, USA: 6, EU: 38, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (numerical sizing) in size 28?",
                "id": "327",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 2.5, USA: 10, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (numerical sizing) in size 29?",
                "id": "328",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 2.5, USA: 10, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (numerical sizing) in size 30?",
                "id": "329",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - shorts (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 2.5, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - shorts (numerical sizing) in size 32?",
                "id": "330",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - biker shorts (alpha sizing) in size extra small, Waist: 25, Hip: 28, Inseam: 10, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - biker shorts (alpha sizing) in size extra small?",
                "id": "331",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - biker shorts (alpha sizing) in size small, Waist: 27, Hip: 30, Inseam: 10, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - biker shorts (alpha sizing) in size small?",
                "id": "332",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - biker shorts (alpha sizing) in size medium, Waist: 29, Hip: 32, Inseam: 10, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - biker shorts (alpha sizing) in size medium?",
                "id": "333",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - biker shorts (alpha sizing) in size large, Waist: 31, Hip: 34, Inseam: 10, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - biker shorts (alpha sizing) in size large?",
                "id": "334",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - biker shorts (alpha sizing) in size extra large, Waist: 33, Hip: 36, Inseam: 10, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - biker shorts (alpha sizing) in size extra large?",
                "id": "335",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size extra small, Waist: 23, Hip: 25, Inseam: 26, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings with side pockets (alpha sizing) in size extra small?",
                "id": "336",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size small, Waist: 25, Hip: 27, Inseam: 26, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings with side pockets (alpha sizing) in size small?",
                "id": "337",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size medium, Waist: 27, Hip: 29, Inseam: 26, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings with side pockets (alpha sizing) in size medium?",
                "id": "338",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size large, Waist: 29, Hip: 31, Inseam: 26, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings with side pockets (alpha sizing) in size large?",
                "id": "339",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size extra large, Waist: 31, Hip: 33, Inseam: 26, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings with side pockets (alpha sizing) in size extra large?",
                "id": "340",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size extra small, Waist: 25, Hip: 28, Inseam: 26.5, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings without pockets (alpha sizing) in size extra small?",
                "id": "341",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size small, Waist: 25, Hip: 28, Inseam: 26.5, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings without pockets (alpha sizing) in size small?",
                "id": "342",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size medium, Waist: 29, Hip: 32, Inseam: 26.5, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings without pockets (alpha sizing) in size medium?",
                "id": "343",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size large, Waist: 31, Hip: 34, Inseam: 26.5, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings without pockets (alpha sizing) in size large?",
                "id": "344",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size extra large, Waist: 33, Hip: 36, Inseam: 26.5, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - sports leggings without pockets (alpha sizing) in size extra large?",
                "id": "345",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (mid-waist) in size extra small, Waist: 23, Hip: 35, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (mid-waist) in size extra small?",
                "id": "346",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (mid-waist) in size small, Waist: 25, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (mid-waist) in size small?",
                "id": "347",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (mid-waist) in size medium, Waist: 27, Hip: 39, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (mid-waist) in size medium?",
                "id": "348",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (mid-waist) in size large, Waist: 29, Hip: 41, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (mid-waist) in size large?",
                "id": "349",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (mid-waist) in size extra large, Waist: 31, Hip: 43, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (mid-waist) in size extra large?",
                "id": "350",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (high-waist) in size extra small, Waist: 21, Hip: 35, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (high-waist) in size extra small?",
                "id": "351",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (high-waist) in size small, Waist: 23, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (high-waist) in size small?",
                "id": "352",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (high-waist) in size medium, Waist: 25, Hip: 39, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (high-waist) in size medium?",
                "id": "353",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (high-waist) in size large, Waist: 27, Hip: 41, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (high-waist) in size large?",
                "id": "354",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants (high-waist) in size extra large, Waist: 29, Hip: 43, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants (high-waist) in size extra large?",
                "id": "355",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants with cuff in size extra small, Waist: 23, Hip: 35, Inseam: 26, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants with cuff in size extra small?",
                "id": "356",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants with cuff in size small, Waist: 25, Hip: 37, Inseam: 26, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants with cuff in size small?",
                "id": "357",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants with cuff in size medium, Waist: 27, Hip: 39, Inseam: 26, USA: 6, EU: 38, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants with cuff in size medium?",
                "id": "358",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants with cuff in size large, Waist: 29, Hip: 41, Inseam: 26, USA: 10, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants with cuff in size large?",
                "id": "359",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - track pants with cuff in size extra large, Waist: 31, Hip: 43, Inseam: 26, USA: 10, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - track pants with cuff in size extra large?",
                "id": "360",
            }
        ]
    },
{
        "context": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (alpha sizing) in size extra small?",
                "id": "361",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (alpha sizing) in size small?",
                "id": "362",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (alpha sizing) in size medium?",
                "id": "363",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (alpha sizing) in size large?",
                "id": "364",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (alpha sizing) in size extra large?",
                "id": "365",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (numerical sizing) in size 26?",
                "id": "366",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (numerical sizing) in size 27?",
                "id": "367",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (numerical sizing) in size 28?",
                "id": "368",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (numerical sizing) in size 29?",
                "id": "369",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (numerical sizing) in size 30?",
                "id": "370",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban tapered trousers (numerical sizing) in size 32?",
                "id": "371",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (alpha sizing) in size extra small?",
                "id": "372",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (alpha sizing) in size small?",
                "id": "373",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (alpha sizing) in size medium?",
                "id": "374",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (alpha sizing) in size large?",
                "id": "375",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (alpha sizing) in size extra large?",
                "id": "376",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (numerical sizing) in size 26?",
                "id": "377",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (numerical sizing) in size 27?",
                "id": "378",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (numerical sizing) in size 28?",
                "id": "379",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (numerical sizing) in size 29?",
                "id": "380",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (numerical sizing) in size 30?",
                "id": "381",
            }
        ]
    },
    {
        "context": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' bottoms - urban wide trousers (numerical sizing) in size 32?",
                "id": "382",
            }
        ]
    },
    {
        "context": "For men's loungewear robe in size small, Chest: 44, Body Length: 40, Sleeve Length: 19.5, USA: 12-16, EU: 46-50, and UK: 36-40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's loungewear robe in size small?",
                "id": "383",
            }
        ]
    },
    {
        "context": "For men's loungewear robe in size medium, Chest: 48, Body Length: 42, Sleeve Length: 19.5, USA: 18-20, EU: 52-54, and UK: 42-44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's loungewear robe in size medium?",
                "id": "384",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (oversized tee) in size extra small, Chest: 39, Body Length: 19, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (oversized tee) in size extra small?",
                "id": "385",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (oversized tee) in size small, Chest: 41, Body Length: 19.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (oversized tee) in size small?",
                "id": "386",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (oversized tee) in size medium, Chest: 43, Body Length: 20, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (oversized tee) in size medium?",
                "id": "387",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (oversized tee) in size large, Chest: 45, Body Length: 20.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (oversized tee) in size large?",
                "id": "388",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (oversized tee) in size extra large, Chest: 47, Body Length: 21, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (oversized tee) in size extra large?",
                "id": "389",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size extra small, Chest: 26, Body Length: 18, Sleeve Length: 2, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size extra small?",
                "id": "390",
            }
        ]
    },
{
        "context": "For ladies' lounge set - bottom (shorts) in size small, Chest: 28, Body Length: 19, Sleeve Length: 2, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size small?",
                "id": "391",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size medium, Chest: 30, Body Length: 20, Sleeve Length: 2, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size medium?",
                "id": "392",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size large, Chest: 32, Body Length: 21, Sleeve Length: 2, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size large?",
                "id": "393",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size extra large, Chest: 34, Body Length: 22, Sleeve Length: 2, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size extra large?",
                "id": "394",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (spaghetti) in size extra small, Chest: 31, Body Length: 16, Sleeve Length: N/A, USA: 2, EU: 30, and UK: 6.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (spaghetti) in size extra small?",
                "id": "395",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (spaghetti) in size small, Chest: 33, Body Length: 16.5, Sleeve Length: N/A, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (spaghetti) in size small?",
                "id": "396",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (spaghetti) in size medium, Chest: 35, Body Length: 17, Sleeve Length: N/A, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (spaghetti) in size medium?",
                "id": "397",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (spaghetti) in size large, Chest: 37, Body Length: 17.5, Sleeve Length: N/A, USA: 8, EU: 40, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (spaghetti) in size large?",
                "id": "398",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - top (spaghetti) in size extra large, Chest: 39, Body Length: 18, Sleeve Length: N/A, USA: 10, EU: 42, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - top (spaghetti) in size extra large?",
                "id": "399",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size extra small, Chest: 26, Body Length: 36, Sleeve Length: 2, USA: 4, EU: 32, and UK: 8.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size extra small?",
                "id": "400",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size small, Chest: 28, Body Length: 38, Sleeve Length: 2, USA: 6, EU: 36, and UK: 10.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size small?",
                "id": "401",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size medium, Chest: 30, Body Length: 40, Sleeve Length: 2, USA: 8, EU: 38, and UK: 12.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size medium?",
                "id": "402",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size large, Chest: 32, Body Length: 42, Sleeve Length: 2, USA: 10, EU: 40, and UK: 14.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size large?",
                "id": "403",
            }
        ]
    },
    {
        "context": "For ladies' lounge set - bottom (shorts) in size extra large, Chest: 34, Body Length: 44, Sleeve Length: 2, USA: 12, EU: 42, and UK: 16.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lounge set - bottom (shorts) in size extra large?",
                "id": "404",
            }
        ]
    },
    {
        "context": "For ladies' footwear single band sliders, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24.2), (US: 7, EU: 37, CM: 24.9), (US: 8, EU: 38, CM: 25.6), (US: 9, EU: 39, CM: 26.3).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' footwear single band sliders?",
                "id": "405",
            }
        ]
    },
    {
        "context": "For ladies' footwear velcro sliders, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24.2), (US: 7, EU: 37, CM: 24.9), (US: 8, EU: 38, CM: 25.6), (US: 9, EU: 39, CM: 26.3).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' footwear velcro sliders?",
                "id": "406",
            }
        ]
    },
    {
        "context": "For ladies' regular flipflops, (US: 5, EU: 35, CM: 23.2), (US: 6, EU: 36, CM: 23.9), (US: 7, EU: 37, CM: 24.6), (US: 8, EU: 38, CM: 25.3), (US: 9, EU: 39, CM: 26), then the slippers thicknes is 15mm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' regular flipflops?",
                "id": "407",
            }
        ]
    },
    {
        "context": "For ladies' lace-up sneakers, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24), (US: 7, EU: 37, CM: 24.5), (US: 8, EU: 38, CM: 25), (US: 9, EU: 39, CM: 25.5).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' lace-up sneakers?",
                "id": "408",
            }
        ]
    },
    {
        "context": "For ladies' runner shoes, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24), (US: 7, EU: 37, CM: 24.5), (US: 8, EU: 38, CM: 25), (US: 9, EU: 39, CM: 25.5).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' runner shoes?",
                "id": "409",
            }
        ]
    },
    {
        "context": "For men's coed tee in size double extra small, Chest: 36, Body Length: 27.5, Sleeve Length: 7, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed tee in size double extra small?",
                "id": "410",
            }
        ]
    },
{
        "context": "For men's coed tee in size extra small, Chest: 38, Body Length: 27.5, Sleeve Length: 7.25, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed tee in size extra small?",
                "id": "411",
            }
        ]
    },
    {
        "context": "For men's coed tee in size small, Chest: 40, Body Length: 28, Sleeve Length: 7.5, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed tee in size small?",
                "id": "412",
            }
        ]
    },
    {
        "context": "For men's coed tee in size medium, Chest: 42, Body Length: 28.5, Sleeve Length: 7.75, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed tee in size medium?",
                "id": "413",
            }
        ]
    },
    {
        "context": "For men's coed tee in size large, Chest: 44, Body Length: 29, Sleeve Length: 8, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed tee in size large?",
                "id": "414",
            }
        ]
    },
    {
        "context": "For men's coed tee in size extra large, Chest: 46, Body Length: 30, Sleeve Length: 8.25, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed tee in size extra large?",
                "id": "415",
            }
        ]
    },
    {
        "context": "For men's coed tee in size double extra large, Chest: 48, Body Length: 30.5, Sleeve Length: 8.5, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed tee in size double extra large?",
                "id": "416",
            }
        ]
    },
    {
        "context": "For men's coed tee in size triple extra large, Chest: 50, Body Length: 31, Sleeve Length: 8.75, USA: 24, EU: 58, and UK: 48.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed tee in size triple extra large?",
                "id": "417",
            }
        ]
    },
    {
        "context": "For ladies' coed tee in size double extra small, Chest: 36, Body Length: 27.5, Sleeve Length: 7, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed tee in size small?",
                "id": "418",
            }
        ]
    },
    {
        "context": "For ladies' coed tee in size medium, Chest: 38, Body Length: 27.5, Sleeve Length: 7.25, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed tee in size medium?",
                "id": "419",
            }
        ]
    },
    {
        "context": "For ladies' coed tee in size large, Chest: 40, Body Length: 28, Sleeve Length: 7.5, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed tee in size large?",
                "id": "420",
            }
        ]
    },
    {
        "context": "For ladies' coed tee in size extra large, Chest: 42, Body Length: 28.5, Sleeve Length: 7.75, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed tee in size extra large?",
                "id": "421",
            }
        ]
    },
    {
        "context": "For men's coed pullover in size double extra small, Chest: 38, Body Length: 24, Sleeve Length: 21.25, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed pullover in size double extra small?",
                "id": "422",
            }
        ]
    },
    {
        "context": "For men's coed pullover in size extra small, Chest: 40, Body Length: 25, Sleeve Length: 21.5, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed pullover in size extra small?",
                "id": "423",
            }
        ]
    },
    {
        "context": "For men's coed pullover in size small, Chest: 42, Body Length: 26, Sleeve Length: 21.75, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed pullover in size small?",
                "id": "424",
            }
        ]
    },
    {
        "context": "For men's coed pullover in size medium, Chest: 44, Body Length: 27, Sleeve Length: 22, USA: 16, EU: 50, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed pullover in size medium?",
                "id": "425",
            }
        ]
    },
    {
        "context": "For men's coed pullover in size large, Chest: 46, Body Length: 28, Sleeve Length: 22.25, USA: 18, EU: 52, and UK: 42.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed pullover in size large?",
                "id": "426",
            }
        ]
    },
    {
        "context": "For men's coed pullover in size extra large, Chest: 48, Body Length: 29, Sleeve Length: 22.5, USA: 20, EU: 54, and UK: 44.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed pullover in size extra large?",
                "id": "427",
            }
        ]
    },
    {
        "context": "For men's coed pullover in size double extra large, Chest: 50, Body Length: 30, Sleeve Length: 22.75, USA: 22, EU: 56, and UK: 46.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed pullover in size double extra large?",
                "id": "428",
            }
        ]
    },
    {
        "context": "For men's coed pullover in size triple extra large, Chest: 52, Body Length: 31, Sleeve Length: 23, USA: 24, EU: 58, and UK: 48.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed pullover in size triple extra large?",
                "id": "429",
            }
        ]
    },
    {
        "context": "For ladies' coed pullover in size small, Chest: 38, Body Length: 24, Sleeve Length: 21.25, USA: 10, EU: 44, and UK: 34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed pullover in size small?",
                "id": "430",
            }
        ]
    },
    {
        "context": "For ladies' coed pullover in size medium, Chest: 40, Body Length: 25, Sleeve Length: 21.5, USA: 12, EU: 46, and UK: 36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed pullover in size medium?",
                "id": "431",
            }
        ]
    },
    {
        "context": "For ladies' coed pullover in size large, Chest: 42, Body Length: 26, Sleeve Length: 21.75, USA: 14, EU: 48, and UK: 38.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed pullover in size large?",
                "id": "432",
            }
        ]
    },
    {
        "context": "For ladies' coed pullover in size extra large, Chest: 44, Body Length: 27, Sleeve Length: 22, USA: 16, EU: 50, and UK: 40.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed pullover in size extra large?",
                "id": "433",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (urban straight trouser) in size double extra small, Waist: 22, Hip: 34, Inseam: 23, USA: 34-36, EU: 35-36, and UK: 27-28.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (urban straight trouser) in size double extra small?",
                "id": "434",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (urban straight trouser) in size extra small, Waist: 24, Hip: 36, Inseam: 23, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (urban straight trouser) in size extra small?",
                "id": "435",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (urban straight trouser) in size small, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (urban straight trouser) in size small?",
                "id": "436",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (urban straight trouser) in size medium, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (urban straight trouser) in size medium?",
                "id": "437",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (urban straight trouser) in size large, Waist: 30, Hip: 42, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (urban straight trouser) in size large?",
                "id": "438",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (urban straight trouser) in size extra large, Waist: 32, Hip: 44, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (urban straight trouser) in size extra large?",
                "id": "439",
            }
        ]
    },
    {
        "context": "For ladies' coed woven trousers (urban straight trouser) in size small, Waist: 22, Hip: 34, Inseam: 23, USA: 34-36, EU: 35-36, and UK: 27-28.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed woven trousers (urban straight trouser) in size small?",
                "id": "440",
            }
        ]
    },
    {
        "context": "For ladies' coed woven trousers (urban straight trouser) in size medium, Waist: 24, Hip: 36, Inseam: 23, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed woven trousers (urban straight trouser) in size medium?",
                "id": "441",
            }
        ]
    },
    {
        "context": "For ladies' coed woven trousers (urban straight trouser) in size large, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed woven trousers (urban straight trouser) in size large?",
                "id": "442",
            }
        ]
    },
    {
        "context": "For ladies' coed woven trousers (urban straight trouser) in size extra large, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed woven trousers (urban straight trouser) in size extra large?",
                "id": "443",
            }
        ]
    },
    {
        "context": "For men's coed shorts (woven) in size double extra small, Waist: 27, Hip: 37, Inseam: 5, USA: 34-36, EU: 35-36, and UK: 27-28.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed shorts (woven) in size double extra small?",
                "id": "444",
            }
        ]
    },
    {
        "context": "For men's coed shorts (woven) in size extra small, Waist: 29, Hip: 39, Inseam: 5, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed shorts (woven) in size extra small?",
                "id": "445",
            }
        ]
    },
    {
        "context": "For men's coed shorts (woven) in size small, Waist: 31, Hip: 41, Inseam: 5, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed shorts (woven) in size small?",
                "id": "446",
            }
        ]
    },
    {
        "context": "For men's coed shorts (woven) in size medium, Waist: 33, Hip: 43, Inseam: 5, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed shorts (woven) in size medium?",
                "id": "447",
            }
        ]
    },
    {
        "context": "For men's coed shorts (woven) in size large, Waist: 35, Hip: 45, Inseam: 5, USA: 43-44, EU: 43-44, and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed shorts (woven) in size large?",
                "id": "448",
            }
        ]
    },
    {
        "context": "For men's coed shorts (woven) in size extra large, Waist: 37, Hip: 47, Inseam: 5, USA: 45-46, EU: 45-46, and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed shorts (woven) in size extra large?",
                "id": "449",
            }
        ]
    },
    {
        "context": "For ladies' coed shorts (woven) in size small, Waist: 27, Hip: 37, Inseam: 5, USA: 34-36, EU: 35-36, and UK: 27-28.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed shorts (woven) in size small?",
                "id": "450",
            }
        ]
    },
{
        "context": "For ladies' coed shorts (woven) in size medium, Waist: 29, Hip: 39, Inseam: 5, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed shorts (woven) in size medium?",
                "id": "451",
            }
        ]
    },
    {
        "context": "For ladies' coed shorts (woven) in size large, Waist: 31, Hip: 41, Inseam: 5, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed shorts (woven) in size large?",
                "id": "452",
            }
        ]
    },
    {
        "context": "For ladies' coed shorts (woven) in size extra large, Waist: 33, Hip: 43, Inseam: 5, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed shorts (woven) in size extra large?",
                "id": "453",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (tapered) in size double extra small, Waist: 22, Hip: 34, Inseam: 24, USA: 34-36, EU: 35-36, and UK: 27-28.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (tapered) in size double extra small?",
                "id": "454",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (tapered) in size extra small, Waist: 24, Hip: 36, Inseam: 24, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (tapered) in size extra small?",
                "id": "455",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (tapered) in size small, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (tapered) in size small?",
                "id": "456",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (tapered) in size medium, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (tapered) in size medium?",
                "id": "457",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (tapered) in size large, Waist: 30, Hip: 42, Inseam: 24, USA: 43-44, EU: 43-44, and UK: 33-34.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (tapered) in size large?",
                "id": "458",
            }
        ]
    },
    {
        "context": "For men's coed woven trousers (tapered) in size extra large, Waist: 32, Hip: 44, Inseam: 24, USA: 45-46, EU: 45-46, and UK: 35-36.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for men's coed woven trousers (tapered) in size extra large?",
                "id": "459",
            }
        ]
    },
    {
        "context": "For ladies' coed woven trousers (tapered) in size small, Waist: 22, Hip: 34, Inseam: 24, USA: 34-36, EU: 35-36, and UK: 27-28.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed woven trousers (tapered) in size small?",
                "id": "460",
            }
        ]
    },
    {
        "context": "For ladies' coed woven trousers (tapered) in size medium, Waist: 24, Hip: 36, Inseam: 24, USA: 36-38, EU: 36-38, and UK: 29-30.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed woven trousers (tapered) in size medium?",
                "id": "461",
            }
        ]
    },
    {
        "context": "For ladies' coed woven trousers (tapered) in size large, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed woven trousers (tapered) in size large?",
                "id": "462",
            }
        ]
    },
    {
        "context": "For ladies' coed woven trousers (tapered) in size extra large, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for ladies' coed woven trousers (tapered) in size extra large?",
                "id": "463",
            }
        ]
    },
    {
        "context": "For accessories - regular backpack, Length: 17 inches, Width: 11 inches, Depth: 5 inches, Strap Length: 16.5 inches, Strap Width: 2.75 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - regular backpack?",
                "id": "464",
            }
        ]
    },
    {
        "context": "For accessories - large backpack, Length: 18 inches, Width: 12 inches, Depth: 6 inches, Strap Length: 16.5 inches, Strap Width: 2.75 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - large backpack?",
                "id": "465",
            }
        ]
    },
    {
        "context": "For accessories - drawstring backpack, Length: 17 inches and Width: 13 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - drawstring backpack?",
                "id": "466",
            }
        ]
    },
    {
        "context": "For accessories - regular bum bag, Length: 4.5 inches, Width: 13.5 inches, Depth: 3 inches, Strap Length: 43 inches, Strap Width: 1.5 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - regular bum bag?",
                "id": "467",
            }
        ]
    },
    {
        "context": "For accessories - multi pockets bum bag, Length: 7.5 inches, Width: 13 inches, Depth: 3.5 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - multi pockets bum bag?",
                "id": "468",
            }
        ]
    },
    {
        "context": "For accessories - oversized bum bag, Length: 9 inches, Width: 18 inches, Depth: 4 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - oversized bum bag?",
                "id": "469",
            }
        ]
    },
    {
        "context": "For accessories - duffle bag, Length: 9.75 inches, Width: 18 inches, Depth: 9.75 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - duffle bag?",
                "id": "470",
            }
        ]
    },
    {
        "context": "For accessories - mini sling bag, Length: 7 inches, Width: 5 inches, Depth: 1.5 inches, Strap Length: 53 inches, Strap Width: 1 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - mini sling bag?",
                "id": "471",
            }
        ]
    },
    {
        "context": "For accessories - regular sling bag, Length: 9 inches, Width: 7 inches, Depth: 2 inches, Strap Length: 53 inches, Strap Width: 1 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - regular sling bag?",
                "id": "472",
            }
        ]
    },
    {
        "context": "For accessories - mid sized messenger bag, Length: 10 inches, Width: 8 inches, Depth: 2.5 inches, Strap Length: 5.3 inches, Strap Width: 1.5 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - mid sized messenger bag?",
                "id": "473",
            }
        ]
    },
    {
        "context": "For accessories - type 1 tote bag, Length: 16 inches, Width: 10 inches, Depth: 4 inches, Strap Length: 26 inches, Strap Width: 1.5 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - type 1 tote bag?",
                "id": "474",
            }
        ]
    },
    {
        "context": "For accessories - type 2 tote bag, Length: 16 inches, Width: 13.5 inches, Depth: N/A, Strap Length: 26 inches, Strap Width: 1 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - type 2 tote bag?",
                "id": "475",
            }
        ]
    },
    {
        "context": "For accessories - beanie, Circumference: 57 cm and Height: 8.89 cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - beanie?",
                "id": "476",
            }
        ]
    },
    {
        "context": "For accessories - bucket hat, Circumference: 59 cm, Height: 8.89 cm, Visor Length: 6.35 cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - bucket hat?",
                "id": "477",
            }
        ]
    },
    {
        "context": "For accessories - curved cap, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.99 cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - curved cap?",
                "id": "478",
            }
        ]
    },
    {
        "context": "For accessories - dad hat, Circumference: 57 cm, Height: 17.78 cm, Visor Length: 6.99 cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - dad hat?",
                "id": "479",
            }
        ]
    },
    {
        "context": "For accessories - snap back, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.35 cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - snap back?",
                "id": "480",
            }
        ]
    },
    {
        "context": "For accessories - trucker cap, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.99 cm.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - trucker cap?",
                "id": "481",
            }
        ]
    },
    {
        "context": "For accessories - type 1 coin purse, Length: 4 inches, Width: 3 inches, Depth: 0.75 inch, Zipper Length: 3 3/4 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - type 1 coin purse?",
                "id": "482",
            }
        ]
    },
    {
        "context": "For accessories - type 2 coin purse, Length: 3 inches, Width: 4 inches, Depth: 0.75 inch, Zipper Length: 5 1/2 inches.",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - type 2 coin purse?",
                "id": "483",
            }
        ]
    },
    {
        "context": "For accessories - bifold wallet, Folded: (Length: 3.5 inches, Width: 3 1/2 inches, Depth: 0.5 inch), Bill Compartment: (Length: 4.5 inches and Height: 3 1/4 inches).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - bifold wallet?",
                "id": "484",
            }
        ]
    },
    {
        "context": "For accessories - midsized bifold wallet, Folded: (Length: 3 1/4 inches, Width: 4 1/2 inches), Bill Compartment: (Length: 6 1/2 inches and Height: 3 3/4 inches).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - midsized bifold wallet?",
                "id": "485",
            }
        ]
    },
    {
        "context": "For accessories - trifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - trifold wallet?",
                "id": "486",
            }
        ]
    },
    {
        "context": "For accessories - zip around bifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches, Depth: 0.5 inch), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - zip around bifold wallet?",
                "id": "487",
            }
        ]
    },
    {
        "context": "For accessories - trifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - trifold wallet?",
                "id": "488",
            }
        ]
    },
    {
        "context": "For accessories - zip around bifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches, Depth: 0.5 inch), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).",
        "qas": [
            {
                "question": "What are the size and dimension possibilities for accessories - zip around bifold wallet?",
                "id": "489",
            }
        ]
    },
    {
        "context": "Yes, we offer international shipping. You can place your order online, and we'll ship it to your desired destination.",
        "qas": [
            {
                "question": "Can I place an order online and have it shipped internationally?",
                "id": "490",
            }
        ]
    },
    {
        "context": "Sale items are eligible for returns and exchanges within 14 days of purchase. Please ensure the item is in its original condition.",
        "qas": [
            {
                "question": "What is your store's policy on returns and exchanges for sale items?",
                "id": "491",
            }
        ]
    },
    {
        "context": "You can stay updated on our upcoming sales and events by subscribing to our newsletter or following our social media channels.",
        "qas": [
            {
                "question": "Are there any upcoming sales or events at your store?",
                "id": "492",
            }
        ]
    },
    {
        "context": "Yes, we offer personal shopping and styling services. Our team can help you find the perfect outfit tailored to your style.",
        "qas": [
            {
                "question": "Do you offer personal shopping or styling services?",
                "id": "493",
            }
        ]
    },
    {
        "context": "You can apply for our store credit card, which offers benefits like exclusive discounts, rewards, and special financing options.",
        "qas": [
            {
                "question": "Can I apply for a store credit card, and what are the benefits?",
                "id": "494",
            }
        ]
    },
    {
        "context": "We welcome your feedback! You can provide feedback through our website, in-store, or by contacting our customer support.",
        "qas": [
            {
                "question": "How can I provide feedback or suggestions about my shopping experience?",
                "id": "495",
            }
        ]
    },
    {
        "context": "Yes, we can place a special order for items that are out of stock in your size. Please speak with a store associate for assistance.",
        "qas": [
            {
                "question": "Can I order clothing that is currently out of stock in my size?",
                "id": "496",
            }
        ]
    },
    {
        "context": "We offer gift wrapping services for a small fee. Your items will be beautifully packaged for a special touch.",
        "qas": [
            {
                "question": "What is your store's policy on gift wrapping and packaging?",
                "id": "497",
            }
        ]
    },
    {
        "context": "We release new collections regularly, typically with the changing seasons. Stay tuned for our latest arrivals.",
        "qas": [
            {
                "question": "How often do you release new collections or clothing lines?",
                "id": "498",
            }
        ]
    },
    {
        "context": "Yes, we offer the option to book private events or fashion shows in our store. Contact our events team for details and availability.",
        "qas": [
            {
                "question": "Can I book a private event or fashion show at your store?",
                "id": "499",
            }
        ]
    },
    {
        "context": "Yes, we offer express and same-day delivery options for online orders in select locations.",
        "qas": [
            {
                "question": "Do you offer express or same-day delivery for online orders?",
                "id": "500",
            }
        ]
    },
    {
        "context": "Absolutely! We have a loyalty program where you can earn rewards and exclusive discounts.",
        "qas": [
            {
                "question": "Is there a loyalty program for frequent shoppers?",
                "id": "501",
            }
        ]
    },
    {
        "context": "We offer price adjustments for items purchased within seven days before a sale. Bring your receipt for verification.",
        "qas": [
            {
                "question": "What is your store's policy on price adjustments for items purchased before a sale?",
                "id": "502",
            }
        ]
    },
    {
        "context": "Yes, you can request clothing alterations at the time of purchase. Our team can assist you with adjustments.",
        "qas": [
            {
                "question": "Can I request a clothing alteration at the time of purchase?",
                "id": "503",
            }
        ]
    },
    {
        "context": "Yes, we offer special discounts for students and senior citizens. Please bring a valid ID for verification.",
        "qas": [
            {
                "question": "Are there special discounts for students or senior citizens?",
                "id": "504",
            }
        ]
    },
    {
        "context": "We encourage clothing donations and recycling. You can inquire in-store about our donation programs.",
        "qas": [
            {
                "question": "What is your policy on donating or recycling old clothing?",
                "id": "505",
            }
        ]
    },
    {
        "context": "Certainly! We can assist with bulk orders for special events. Please contact our sales team for more details.",
        "qas": [
            {
                "question": "Can I purchase clothing items in bulk for a special event?",
                "id": "506",
            }
        ]
    },
    {
        "context": "Yes, we have a mobile app that makes online shopping convenient and offers exclusive deals.",
        "qas": [
            {
                "question": "Do you have a mobile app for online shopping?",
                "id": "507",
            }
        ]
    },
    {
        "context": "You can track your online order and check its status by logging into your account on our website or app.",
        "qas": [
            {
                "question": "How can I track my online order or check its status?",
                "id": "508",
            }
        ]
    },
    {
        "context": "Yes, we offer gift-wrapping services for online orders, making it a perfect gift option.",
        "qas": [
            {
                "question": "Do you offer gift-wrapping services for online orders?",
                "id": "509",
            }
        ]
    },
    {
        "context": "We offer store credit for returns within 30 days, or refunds in the original payment method.",
        "qas": [
            {
                "question": "What is your policy on store credit and refunds for returned items?",
                "id": "510",
            }
        ]
    },
    {
        "context": "Yes, email subscribers receive exclusive discounts and early access to promotions.",
        "qas": [
            {
                "question": "Are there any exclusive discounts or promotions for email subscribers?",
                "id": "511",
            }
        ]
    },
    {
        "context": "You can check the availability of items in a specific store location by contacting the store directly.",
        "qas": [
            {
                "question": "Can I check the availability of an item in a specific store location?",
                "id": "512",
            }
        ]
    },
    {
        "context": "We have robust security measures to protect customer data and adhere to strict privacy policies.",
        "qas": [
            {
                "question": "What measures are in place for customer data security and privacy?",
                "id": "513",
            }
        ]
    },
    {
        "context": "Yes, we offer a gift registry service for weddings and special occasions. Visit our website for details.",
        "qas": [
            {
                "question": "Do you offer a gift registry service for special occasions?",
                "id": "514",
            }
        ]
    },
    {
        "context": "Yes, we have a referral program where you and your friends can enjoy discounts.",
        "qas": [
            {
                "question": "Can I get a discount for referring friends to your store?",
                "id": "515",
            }
        ]
    },
    {
        "context": "You can apply for job openings by visiting our careers page on our website and submitting your application.",
        "qas": [
            {
                "question": "How can I apply for a job at your store? Are there current job openings?",
                "id": "516",
            }
        ]
    },
    {
        "context": "Yes, we offer corporate gifting solutions, including branded and customized gift options.",
        "qas": [
            {
                "question": "Do you offer corporate gifting solutions for businesses?",
                "id": "517",
            }
        ]
    },
    {
        "context": "Yes, we can arrange to transfer items from other store locations for your pickup.",
        "qas": [
            {
                "question": "Can I order a clothing item from a different store location for pickup?",
                "id": "518",
            }
        ]
    },
    {
        "context": "You can schedule a fashion consultation or appointment by contacting our store and booking a time slot.",
        "qas": [
            {
                "question": "What is the process for arranging a fashion consultation or appointment?",
                "id": "519",
            }
        ]
    },
    {
        "context": "Yes, we accept mobile payment options like Apple Pay and Google Pay for your convenience.",
        "qas": [
            {
                "question": "Is there a mobile payment option available in your store?",
                "id": "520",
            }
        ]
    },
    {
        "context": "Yes, you can exchange a gift for a different size or style within our return policy guidelines.",
        "qas": [
            {
                "question": "Can I exchange a gift if it's the wrong size or style?",
                "id": "521",
            }
        ]
    },
    {
        "context": "Yes, we have a store newsletter that provides updates, promotions, and exclusive offers to subscribers.",
        "qas": [
            {
                "question": "Do you have a store newsletter with updates and promotions?",
                "id": "522",
            }
        ]
    },
    {
        "context": "We have implemented measures to ensure a safe and comfortable shopping experience, including enhanced cleaning and safety protocols.",
        "qas": [
            {
                "question": "What steps are taken to ensure a safe and comfortable shopping environment?",
                "id": "523",
            }
        ]
    },
    {
        "context": "For custom-made clothing, we require measurements and specific details to tailor the item to your preferences.",
        "qas": [
            {
                "question": "Are there any requirements for placing an order for custom-made clothing?",
                "id": "524",
            }
        ]
    },
    {
        "context": "Yes, you can use multiple payment methods for a single purchase. Our staff can assist with split payments.",
        "qas": [
            {
                "question": "Can I use multiple payment methods for a single purchase?",
                "id": "525",
            }
        ]
    },
    {
        "context": "We will gladly replace or refund defective or damaged items. Please bring them to our attention within our return policy timeframe.",
        "qas": [
            {
                "question": "What is your policy on handling defective or damaged items?",
                "id": "526",
            }
        ]
    },
    {
        "context": "We value your feedback! You can provide feedback in our stores or through our website.",
        "qas": [
            {
                "question": "Do you have a customer feedback program or suggestion box?",
                "id": "527",
            }
        ]
    },
    {
        "context": "Yes, you can place orders for gifts and have them shipped to multiple addresses during the checkout process.",
        "qas": [
            {
                "question": "Can I order and send clothing items as gifts to different addresses?",
                "id": "528",
            }
        ]
    },
    {
        "context": "You can sign up for a store credit card in-store or online, and eligibility requirements will be reviewed during the application process.",
        "qas": [
            {
                "question": "How do I sign up for a store credit card, and what are the eligibility requirements?",
                "id": "529",
            }
        ]
    },
    {
        "context": "Good morning! How can I assist you today?",
        "qas": [
            {
                "question": "Good morning!",
                "id": "530",
            }
        ]
    },
    {
        "context": "Hello! How can I help you?",
        "qas": [
            {
                "question": "Hello there!",
                "id": "531",
            }
        ]
    },
    {
        "context": "Good afternoon! What can I do for you?",
        "qas": [
            {
                "question": "Hi, it's afternoon!",
                "id": "532",
            }
        ]
    },
    {
        "context": "Good day! How may I assist you?",
        "qas": [
            {
                "question": "Good day!",
                "id": "533",
            }
        ]
    },
    {
        "context": "Good evening! How can I be of service?",
        "qas": [
            {
                "question": "Hello, it's evening!",
                "id": "534",
            }
        ]
    },
    {
        "context": "Good night! How can I assist you at this hour?",
        "qas": [
            {
                "question": "Hi, it's nighttime!",
                "id": "535",
            }
        ]
    },
    {
        "context": "Good dawn! What brings you here so early?",
        "qas": [
            {
                "question": "Hey, it's dawn!",
                "id": "536",
            }
        ]
    },
    {
        "context": "Greetings! How can I help you as the day winds down?",
        "qas": [
            {
                "question": "Greetings, it's dusk!",
                "id": "537",
            }
        ]
    },
    {
        "context": "Good morning and sunrise to you! How can I assist you?",
        "qas": [
            {
                "question": "Hi, it's sunrise!",
                "id": "538",
            }
        ]
    },
    {
        "context": "Good evening and sunset to you! What can I do for you?",
        "qas": [
            {
                "question": "Good evening, it's sunset!",
                "id": "539",
            }
        ]
    },
    {
        "context": "Hello! How may I assist you during this beautiful twilight?",
        "qas": [
            {
                "question": "Hello, it's twilight!",
                "id": "540",
            }
        ]
    },
    {
        "context": "A tie bar, when worn properly, holds the tie in place and adds a stylish and functional element to a business outfit.",
        "qas": [
            {
                "question": "What's the role of a tie bar in business attire for men?",
                "id": "541",
            }
        ]
    },
    {
        "context": "When selecting dress shoes, consider classic styles like oxfords or derbies, and ensure they are well-polished and match the color of your belt.",
        "qas": [
            {
                "question": "Can you offer tips on choosing the right pair of dress shoes for business attire?",
                "id": "542",
            }
        ]
    },
    {
        "context": "To incorporate seasonal trends, add accessories or select clothing items in seasonal colors while maintaining a professional core look.",
        "qas": [
            {
                "question": "How can I incorporate seasonal trends into my business wardrobe?",
                "id": "543",
            }
        ]
    },
    {
        "context": "For a black-tie event, men should wear a black tuxedo, a black bow tie, and polished black dress shoes.",
        "qas": [
            {
                "question": "What's the appropriate way to accessorize a black-tie outfit in the business world?",
                "id": "544",
            }
        ]
    },
    {
        "context": "Women have various options for footwear, including pumps, loafers, ankle boots, and closed-toe heels to match their business outfits.",
        "qas": [
            {
                "question": "What are the options for women's footwear in business attire?",
                "id": "545",
            }
        ]
    },
    {
        "context": "Statement jewelry can be worn in business attire but should be used sparingly to avoid appearing too flashy or distracting.",
        "qas": [
            {
                "question": "Can I wear statement jewelry in business attire?",
                "id": "546",
            }
        ]
    },
    {
        "context": "A tailored dress shirt ensures a perfect fit and a sharp, professional appearance, as it's designed to complement your body shape.",
        "qas": [
            {
                "question": "What's the significance of a tailored dress shirt in business attire?",
                "id": "547",
            }
        ]
    },
    {
        "context": "Transition from day to night by adding a blazer, statement accessories, and a change of shoes to elevate your look for evening events.",
        "qas": [
            {
                "question": "How can I transition from a day-to-night look in business attire?",
                "id": "548",
            }
        ]
    },
    {
        "context": "Turtlenecks can be appropriate in business attire, especially in colder months, when worn with a well-fitted blazer or suit.",
        "qas": [
            {
                "question": "Can men wear turtlenecks in business attire?",
                "id": "549",
            }
        ]
    },
    {
        "context": "Incorporate colors like navy or deep blue to convey confidence, or choose power colors like red or burgundy for a bolder statement.",
        "qas": [
            {
                "question": "How can I use color psychology in my business attire to convey confidence?",
                "id": "550",
            }
        ]
    },
    {
        "context": "Good midnight! How can I assist you at this late hour?",
        "qas": [
            {
                "question": "Hi, it's midnight!",
                "id": "551",
            }
        ]
    },
    {
        "context": "Good noon! How can I help you at this point in the day?",
        "qas": [
            {
                "question": "Hey, it's noon!",
                "id": "552",
            }
        ]
    },
    {
        "context": "Good morning! I'm here to help. What can I do for you?",
        "qas": [
            {
                "question": "Good morning, how can I assist you?",
                "id": "553",
            }
        ]
    },
    {
        "context": "Hello! I'm here to assist you with anything you need.",
        "qas": [
            {
                "question": "Hi there, what brings you today?",
                "id": "554",
            }
        ]
    },
    {
        "context": "Good afternoon! Feel free to ask any questions or seek assistance.",
        "qas": [
            {
                "question": "Good afternoon, how may I help you?",
                "id": "555",
            }
        ]
    },
    {
        "context": "Hello! I'm here to assist you with anything you need. How can I help?",
        "qas": [
            {
                "question": "Hello, what can I do for you this afternoon?",
                "id": "556",
            }
        ]
    },
    {
        "context": "Good evening! I'm here to help. How can I be of service?",
        "qas": [
            {
                "question": "Good evening.",
                "id": "557",
            }
        ]
    },
    {
        "context": "Good night! I'm here to assist you at this hour. What can I do for you?",
        "qas": [
            {
                "question": "Hi, it's nighttime",
                "id": "558",
            }
        ]
    },
    {
        "context": "Good dawn! I'm here to assist you. How can I help at this early hour?",
        "qas": [
            {
                "question": "Hey, it's dawn.",
                "id": "559",
            }
        ]
    },
    {
        "context": "Greetings! I'm here to help as the day winds down. What can I do for you?",
        "qas": [
            {
                "question": "Greetings, it's dusk",
                "id": "560",
            }
        ]
    },
    {
        "context": "Good morning and sunrise to you! I'm here to help. How can I assist you today?",
        "qas": [
            {
                "question": "Hi, it's sunrise",
                "id": "561",
            }
        ]
    },
    {
        "context": "Good evening and sunset to you! I'm here to assist you. What can I do for you?",
        "qas": [
            {
                "question": "Good evening, it's sunset.",
                "id": "562",
            }
        ]
    },
    {
        "context": "Hello! I'm here to help during this lovely twilight. How may I assist you?",
        "qas": [
            {
                "question": "Hello, it's twilight.",
                "id": "563",
            }
        ]
    },
    {
        "context": "Good midnight! I'm here to assist you at this late hour. What can I do for you?",
        "qas": [
            {
                "question": "Hi, it's midnight.",
                "id": "564",
            }
        ]
    },
    {
        "context": "Good noon! I'm here to assist you. What can I do for you at this time?",
        "qas": [
            {
                "question": " Hey, it's noon.",
                "id": "565",
            }
        ]
    },
    {
        "context": "Business casual attire typically includes slacks, a button-up shirt, and closed-toe shoes for men, and a blouse, slacks or skirt, and closed-toe shoes for women.",
        "qas": [
            {
                "question": "What is business casual attire?",
                "id": "566",
            }
        ]
    },
    {
        "context": "Business formal attire is more formal than business casual and often includes a suit and tie for men, and a tailored suit or dress for women.",
        "qas": [
            {
                "question": "What's the difference between business casual and business formal attire?",
                "id": "567",
            }
        ]
    },
    {
        "context": "Essential accessories may include a classic watch, a leather belt, and tasteful jewelry.",
        "qas": [
            {
                "question": "What are some essential accessories for a professional look in the apparel industry?",
                "id": "568",
            }
        ]
    },
    {
        "context": "To prolong the life of your suits, dry clean them only when necessary, hang them properly, and store them in a garment bag.",
        "qas": [
            {
                "question": " How can I maintain and care for my business suits to ensure they last longer?",
                "id": "569",
            }
        ]
    },
    {
        "context": "Neutral colors like black, gray, navy, and white are often considered appropriate for business attire.",
        "qas": [
            {
                "question": "What colors are considered appropriate for business attire?",
                "id": "570",
            }
        ]
    },
    {
        "context": "A power tie, typically a bold and confident color, is worn to convey authority and confidence in business settings.",
        "qas": [
            {
                "question": "What's the significance of a power tie in the business world?",
                "id": "571",
            }
        ]
    },
    {
        "context": "Dress codes in the workplace help maintain a professional image and set a standard for appropriate attire.",
        "qas": [
            {
                "question": "What is the purpose of a dress code in the workplace?",
                "id": "572",
            }
        ]
    },
    {
        "context": "Choose lightweight, breathable fabrics like linen or cotton and opt for short-sleeved shirts or dresses.",
        "qas": [
            {
                "question": "How can I dress professionally in hot weather while working in the business apparel industry?",
                "id": "573",
            }
        ]
    },
    {
        "context": "In many business settings, open-toed shoes are not considered appropriate; closed-toe shoes are a safer choice.",
        "qas": [
            {
                "question": "Are open-toed shoes appropriate in a business setting?",
                "id": "574",
            }
        ]
    },
    {
        "context": "Dress in professional attire that's slightly more formal than the company's dress code to make a positive impression.",
        "qas": [
            {
                "question": "What's the best way to dress for a job interview in the business apparel industry?",
                "id": "575",
            }
        ]
    },
    {
        "context": "Denim is generally not considered business casual, but some workplaces allow it on specific days or with certain conditions.",
        "qas": [
            {
                "question": "Can you wear denim in a business casual setting?",
                "id": "576",
            }
        ]
    },
    {
        "context": "Wear solid, non-distracting colors, and make sure your top half looks professional as it's what's visible on camera.",
        "qas": [
            {
                "question": "How can I dress appropriately for a video conference in the business world?",
                "id": "577",
            }
        ]
    },
    {
        "context": "A blazer is a versatile piece that can elevate a business casual look and add a touch of formality.",
        "qas": [
            {
                "question": "What's the role of a blazer in business attire?",
                "id": "578",
            }
        ]
    },
    {
        "context": "Sneakers are typically not considered appropriate in business settings; opt for more formal footwear.",
        "qas": [
            {
                "question": "Are sneakers acceptable in the business apparel industry?",
                "id": "579",
            }
        ]
    },
    {
        "context": "Personal grooming is crucial for making a positive impression and projecting professionalism.",
        "qas": [
            {
                "question": "What's the importance of personal grooming in the business apparel industry?",
                "id": "580",
            }
        ]
    },
    {
        "context": "Look for sales, outlet stores, and second-hand shops to find high-quality attire at affordable prices.",
        "qas": [
            {
                "question": "How can I dress professionally on a budget in the business apparel industry?",
                "id": "581",
            }
        ]
    },
    {
        "context": "A well-fitted suit can enhance your confidence and convey attention to detail, which is highly regarded in the business world.",
        "qas": [
            {
                "question": "What's the significance of a well-fitted suit in business?",
                "id": "582",
            }
        ]
    },
    {
        "context": "Subtle patterns like stripes or checks are acceptable in business attire, but avoid loud or distracting prints.",
        "qas": [
            {
                "question": "Can I wear patterned shirts in a business setting?",
                "id": "583",
            }
        ]
    },
    {
        "context": "Women can accessorize with a stylish handbag, tasteful jewelry, and closed-toe heels or flats.",
        "qas": [
            {
                "question": "How should I accessorize a business suit for women?",
                "id": "584",
            }
        ]
    },
    {
        "context": "Dress formally, often in a tailored suit for men and an elegant dress for women, to show respect for the occasion.",
        "qas": [
            {
                "question": "What's the best way to dress for a corporate event in the business apparel industry?",
                "id": "585",
            }
        ]
    },
    {
        "context": "Turtlenecks can be appropriate in a business casual setting, especially in colder months, as long as they are clean and well-maintained.",
        "qas": [
            {
                "question": "Is it appropriate to wear turtlenecks in a business casual setting?",
                "id": "586",
            }
        ]
    },
    {
        "context": "A classic Windsor knot or a four-in-hand knot are suitable for a professional look in the business world.",
        "qas": [
            {
                "question": "What kind of tie knot is suitable for a professional look?",
                "id": "587",
            }
        ]
    },
    {
        "context": "Incorporate your company's branding colors subtly in your attire, such as with a tie or scarf, to show alignment with the brand.",
        "qas": [
            {
                "question": "How can I match my attire with my company's branding colors?",
                "id": "588",
            }
        ]
    },
    {
        "context": "Women can wear pantsuits in the business world, as they offer a professional and stylish alternative to dresses or skirts.",
        "qas": [
            {
                "question": "Can women wear pantsuits in the business world?",
                "id": "589",
            }
        ]
    },
    {
        "context": "Cufflinks add a touch of elegance and personal style to dress shirts with French cuffs, making them a symbol of sophistication.",
        "qas": [
            {
                "question": "What's the significance of cufflinks in business attire?",
                "id": "590",
            }
        ]
    },
    {
        "context": "For a business-casual event, consider wearing chinos, a button-down shirt, and loafers for a polished yet relaxed appearance.",
        "qas": [
            {
                "question": "How can I dress professionally for a business-casual event in the apparel industry?",
                "id": "591",
            }
        ]
    },
    {
        "context": "All-black attire can be appropriate in a business setting, but it's important to add some variety with accessories or subtle patterns.",
        "qas": [
            {
                "question": "Can you wear all-black attire in a business setting?",
                "id": "592",
            }
        ]
    },
    {
        "context": "Dress in professional attire that's slightly more formal than the company's dress code to make a positive impression.",
        "qas": [
            {
                "question": "What's the best way to dress for a job interview in the business apparel industry?",
                "id": "593",
            }
        ]
    },
    {
        "context": "Look for sales, outlet stores, and second-hand shops to find high-quality attire at affordable prices.",
        "qas": [
            {
                "question": "How can I dress professionally on a budget in the business apparel industry?",
                "id": "594",
            }
        ]
    },
    {
        "context": "Business casual attire typically includes slacks, a button-up shirt, and closed-toe shoes for men, and a blouse, slacks or skirt, and closed-toe shoes for women.",
        "qas": [
            {
                "question": "What is business casual attire?",
                "id": "595",
            }
        ]
    },
    {
        "context": "For a business dress, opt for fabrics like wool, wool blends, or high-quality synthetic materials for a professional look.",
        "qas": [
            {
                "question": "How can I choose the right fabric for a business dress?",
                "id": "596",
            }
        ]
    },
    {
        "context": "A business skirt should typically fall just above or below the knee, ensuring a professional and modest appearance.",
        "qas": [
            {
                "question": "What's the appropriate length for a business skirt?",
                "id": "597",
            }
        ]
    },
    {
        "context": "Vests or waistcoats can add a touch of sophistication to business attire and are still considered stylish.",
        "qas": [
            {
                "question": "Are vests or waistcoats still in style for business attire?",
                "id": "598",
            }
        ]
    },
    {
        "context": "A pocket square adds a touch of sophistication and personal style to a suit jacket, enhancing its overall appearance.",
        "qas": [
            {
                "question": "What's the significance of wearing a pocket square in a suit jacket?",
                "id": "599",
            }
        ]
    },
    {
        "context": "Different collar styles suit different face shapes; for example, spread collars can complement round faces, while point collars work well with oval faces.",
        "qas": [
            {
                "question": "How can I choose the right dress shirt collar style for my face shape in business attire?", 
                "id": "600",
            }
        ]
    },
    {
        "context": "When selecting a clothing store, consider factors such as location, product range, price range, and customer reviews to find the best fit for your shopping preferences.",
        "qas": [
            {
                "question": "What factors should I consider when choosing a clothing store for my shopping needs?",
                "id": "601",
            }
        ]
    },
    {
        "context": "To find the best deals, sign up for newsletters, follow the store's social media accounts, and keep an eye on sales and clearance sections.",
        "qas": [
            {
                "question": "How can I find the best deals and discounts at clothing stores?",
                "id": "602",
            }
        ]
    },
    {
        "context": "Popular clothing store chains known for quality and selection include OXGN, Zara, H&M, Nordstrom, and Macy's.",
        "qas": [
            {
                "question": "What are some popular clothing store chains known for their quality and selection?",
                "id": "603",
            }
        ]
    },
    {
        "context": "A well-organized store layout makes shopping easier, helping customers find what they need quickly and enhancing their overall shopping experience.",
        "qas": [
            {
                "question": "What's the significance of a well-organized clothing store layout?",
                "id": "604",
            }
        ]
    },
    {
        "context": "Use the store's sizing charts, and don't hesitate to ask for assistance from store staff to ensure you find the right size.",
        "qas": [
            {
                "question": "How can I determine my clothing size accurately when shopping in-store?",
                "id": "605",
            }
        ]
    },
    {
        "context": "To make the changing room experience smoother, limit the number of items you bring in, and ensure you check the store's return policy.",
        "qas": [
            {

                "question": "Can you offer tips for navigating the changing room experience in clothing stores?",
                "id": "606",
            }
        ]
    },
    {
        "context": "Stay updated on fashion trends by following fashion magazines, blogs, and influencers, and then shop for trendy items at your favorite clothing stores.",
        "qas": [
            {
                "question": "How can I stay updated on the latest fashion trends available in clothing stores?",
                "id": "607",
            }
        ]
    },
    {
        "context": "Yes, there are eco-friendly clothing stores that offer sustainable and environmentally-conscious fashion choices, promoting ethical and responsible shopping.",
        "qas": [
            {
                "question": "Are there eco-friendly clothing stores that offer sustainable fashion options?",
                "id": "608",
            }
        ]
    },
    {
        "context": "A personal shopper can assist customers in finding clothing that suits their style and needs, providing a tailored shopping experience.",
        "qas": [
            {
                "question": "What's the role of a personal shopper in a clothing store?",
                "id": "609",
            }
        ]
    },
    {
        "context": "To make the most of sales and promotions, plan your shopping trips during sale events, and prioritize items you need.",
        "qas": [
            {
                "question": "Can you provide advice on making the most of sales and promotions at clothing stores?",
                "id": "610",
            }
        ]
    },
    {
        "context": "Shopping at thrift stores can be budget-friendly, so look for well-maintained and timeless pieces to score high-quality clothing at affordable prices.",
        "qas": [
            {
                "question": "How can I shop for high-quality clothing without breaking the bank at thrift stores?",
                "id": "611",
            }
        ]
    },
    {
        "context": "Yes, many clothing stores provide personalized tailoring services to ensure your clothing fits perfectly and is tailored to your preferences.",
        "qas": [
            {
                "question": "Are there clothing stores that offer personalized tailoring services for a perfect fit?",
                "id": "612",
            }
        ]
    },
    {
        "context": "Customer reviews and ratings provide valuable insights into the store's quality, service, and customer satisfaction, helping you make informed decisions.",
        "qas": [
            {
                "question": "What's the importance of customer reviews and ratings when selecting a clothing store?",
                "id": "613",
            }
        ]
    },
    {
        "context": "To maintain the quality of your clothing, follow care instructions, store garments properly, and handle stains promptly.",
        "qas": [
            {
                "question": "How can I maintain the quality of clothing items purchased from clothing stores?",
                "id": "614",
            }
        ]
    },
    {
        "context": "Yes, some clothing stores specialize in specific fashion niches or styles, catering to unique tastes, such as vintage, bohemian, or streetwear.",
        "qas": [
            {
                "question": "Are there clothing stores that specialize in specific fashion niches or styles?",
                "id": "615",
            }
        ]
    },
    {
        "context": "Loyalty programs can offer rewards, discounts, and exclusive deals for repeat customers, enhancing the shopping experience and building brand loyalty.",
        "qas": [
            {
                "question": "What's the significance of a loyalty program offered by clothing stores?",
                "id": "616",
            }
        ]
    },
    {
        "context": "To identify counterfeit items, research authentic designer features and logos, and be cautious when the price seems too good to be true.",
        "qas": [
            {
                "question": "How can I identify counterfeit or fake designer items when shopping at second-hand clothing stores?",
                "id": "617",
            }
        ]
    },
    {
        "context": "A fashion stylist can provide style advice, help you put together outfits, and suggest clothing items that complement your personal style.",
        "qas": [
            {
                "question": "What's the role of a fashion stylist in a clothing store?",
                "id": "618",
            }
        ]
    },
    {
        "context": "To make the most of the fitting room experience, take your time, try on different sizes, and assess the comfort and fit of each item.",
        "qas": [
            {
                "question": "How can I make the most of the fitting room experience when shopping for clothing?",
                "id": "619",
            }
        ]
    },
    {
        "context": "Several clothing stores, such as Patagonia, EILEEN FISHER, and Reformation, are known for their commitment to sustainable and ethical fashion practices.",
        "qas": [
            {
                "question": "Can you recommend clothing stores that focus on sustainable and ethical fashion practices?",
                "id": "620",
            }
        ]
    },
    {
        "context": "Budget your spending by setting a limit, making a shopping list, and considering cost per wear when evaluating potential purchases.",
        "qas": [
            {
                "question": "How can I effectively budget my spending when shopping for clothing at retail stores?",
                "id": "621",
            }
        ]
    },
    {
        "context": "Yes, many clothing stores collaborate with designers or brands to offer exclusive, limited-edition releases that attract fashion enthusiasts.",
        "qas": [
            {
                "question": "Are there clothing stores that offer exclusive, limited-edition releases?",
                "id": "622",
            }
        ]
    },
    {
        "context": "When shopping online, consider checking size charts, reading product descriptions, and reviewing return policies to make informed decisions.",
        "qas": [
            {
                "question": "What should I consider when shopping for clothing online versus in physical stores?",
                "id": "623",
            }
        ]
    },
    {
        "context": "Make sustainable choices by opting for eco-friendly materials, supporting ethical brands, and recycling or upcycling clothing items.",
        "qas": [
            {
                "question": "How can I make sustainable choices when shopping at clothing stores?",
                "id": "624",
            }
        ]
    },
    {
        "context": "When searching for a special occasion dress, consider the dress code, venue, and your personal style to find the perfect outfit.",
        "qas": [
            {
                "question": "Can you offer guidance on finding the perfect dress for a special occasion at clothing stores?",
                "id": "625",
            }
        ]
    },
    {
        "context": "Avoid impulse buying by creating a shopping list, setting a budget, and taking your time to carefully consider each purchase.",
        "qas": [
            {
                "question": "How can I avoid impulse buying when shopping at clothing stores?",
                "id": "626",
            }
        ]
    },
    {
        "context": "Many clothing stores now offer inclusive sizing options including us in OXGN store, catering to a wide range of body types and promoting body positivity.",
        "qas": [
            {
                "question": "Are there clothing stores that offer inclusive sizing options for all body types?",
                "id": "627",
            }
        ]
    },
    {
        "context": "Seasonal sales and clearance events provide an opportunity to purchase clothing items at reduced prices, making it easier to update your wardrobe.",
        "qas": [
            {
                "question": "What's the role of seasonal sales and clearance events in clothing stores?",
                "id": "628",
            }
        ]
    },
    {
        "context": "To find clothing stores catering to specific age groups or demographics, read customer reviews, consult fashion magazines, and explore social media for recommendations.",
        "qas": [
            {
                "question": "How can I find clothing stores that cater to a specific age group or demographic?",
                "id": "629",
            }
        ]
    },
    {
        "context": "Clothing stores like Aerie and Target are known for their commitment to diversity and inclusivity, both in marketing and product offerings.",
        "qas": [
            {
                "question": "Can you recommend clothing stores that prioritize diversity and inclusivity in their marketing and products?",
                "id": "630",
            }
        ]
    },
    {
        "context": "Trying on clothing items allows you to assess the fit, comfort, and overall look of the garment, helping you make confident purchasing decisions.",
        "qas": [
            {
                "question": "What's the importance of trying on clothing items before purchasing them in clothing stores?",
                "id": "631",
            }
        ]
    },
    {
        "context": "After a shopping spree, organize and declutter your closet by categorizing items, donating or selling clothing you no longer need, and maximizing storage space.",
        "qas": [
            {
                "question": "How can I organize and declutter my closet after a shopping spree at clothing stores?",
                "id": "632",
            }
        ]
    },
    {
        "context": "When searching for the perfect pair of jeans, consider your body shape, preferred fit, and wash options, and consult store staff for assistance.",
        "qas": [
            {
                "question": "Can you provide tips for finding the perfect pair of jeans at clothing stores?",
                "id": "633",
            }
        ]
    },
    {
        "context": "Discover local boutiques by exploring your neighborhood, seeking recommendations from friends or online platforms, and attending local fashion events.",
        "qas": [
            {
                "question": "How can I discover local boutique clothing stores that offer unique and one-of-a-kind fashion pieces?",
                "id": "634",
            }
        ]
    },
    {
        "context": "Online customer reviews offer insights into a store's product quality, service, and customer experiences, aiding in your decision-making process.",
        "qas": [
            {
                "question": "What's the role of online customer reviews in choosing the right clothing store?",
                "id": "635",
            }
        ]
    },
    {
        "context": "Brands and stores like Everlane, People Tree, and Outerknown prioritize fair labor practices and ethical sourcing in their fashion production.",
        "qas": [
            {
                "question": "Are there clothing stores known for their commitment to fair labor practices and ethical sourcing?",
                "id": "636",
            }
        ]
    },
    {
        "context": "Look for clothing stores that offer customization or made-to-measure services to ensure your clothing fits perfectly and suits your unique style.",
        "qas": [
            {
                "question": "How can I find clothing stores that offer customizable or made-to-measure clothing?",
                "id": "637",
            }
        ]
    },
    {
        "context": "Clothing store loyalty programs reward frequent shoppers with exclusive discounts, early access to sales, and other perks, enhancing their shopping experience.",
        "qas": [
            {
                "question": "What's the significance of clothing store loyalty programs for frequent shoppers?",
                "id": "638",
            }
        ]
    },
    {
        "context": "To find vintage or retro clothing stores, explore local thrift shops, consignment stores, or online marketplaces specializing in vintage fashion.",
        "qas": [
            {
                "question": "Can you provide guidance on finding vintage or retro clothing stores for unique fashion pieces?",
                "id": "639",
            }
        ]
    },
    {
        "context": "Make environmentally conscious choices by opting for eco-friendly materials, supporting sustainable brands, and choosing clothing made with ethical practices.",
        "qas": [
            {
                "question": "How can I make environmentally conscious choices when shopping at clothing stores?",
                "id": "640",
            }
        ]
    },
    {
        "context": "Inspect clothing items for sturdy stitching, quality fabrics, no defects, and attention to detail, which indicate good quality and durability.",
        "qas": [
            {
                "question": "What should I look for when inspecting the quality of clothing items in stores?",
                "id": "641",
            }
        ]
    },
    {
        "context": "Department stores like Bloomingdale's and specialty stores like Men's Wearhouse offer a wide range of formal wear options for special events.",
        "qas": [
            {
                "question": "Can you recommend clothing stores that offer a wide range of formal wear options for special events?",
                "id": "642",
            }
        ]
    },
    {
        "context": "To align your clothing purchases with your style and needs, plan your wardrobe, create a color palette, and focus on versatile pieces that can be mixed and matched.",
        "qas": [
            {
                "question": "How can I ensure my clothing purchases align with my personal style and wardrobe needs?",
                "id": "643",
            }
        ]
    },
    {
        "context": "To align your clothing purchases with your style and needs, plan your wardrobe, create a color palette, and focus on versatile pieces that can be mixed and matched.",
        "qas": [
            {
                "question": "What are some online tools or apps that can enhance the clothing shopping experience?",
                "id": "644",
            }
        ]
    },
    {
        "context": "Brands like Aerie, Dove, and Fenty Beauty are recognized for promoting body positivity and diverse representation in their advertising campaigns.",
        "qas": [
            {
                "question": "Are there clothing stores known for their commitment to promoting body positivity and diverse representation in their advertising?",
                "id": "645",
            }
        ]
    },
    {
        "context": "A fashion consultant can offer personalized style advice, recommend clothing items, and help customers discover fashion choices that suit their preferences.",
        "qas": [
            {
                "question": "What's the role of a fashion consultant in clothing stores?",
                "id": "646",
            }
        ]
    },
    {
        "context": "A fashion consultant can offer personalized style advice, recommend clothing items, and help customers discover fashion choices that suit their preferences.",
        "qas": [
            {
                "question": "Can you offer tips for finding timeless and versatile pieces at clothing stores?",
                "id": "647",
            }
        ]
    },
    {
        "context": "Calculate the cost per wear by dividing the item's price by the number of times you anticipate wearing it, helping you make cost-effective decisions.",
        "qas": [
            {
                "question": "How can I evaluate the cost per wear of clothing items in stores to make cost-effective choices?",
                "id": "648",
            }
        ]
    },
    {
        "context": "A clothing store's return and exchange policy ensures flexibility in case a purchase doesn't meet your expectations or fit as intended.",
        "qas": [
            {
                "question": " What's the significance of a clothing store's return and exchange policy when making purchases?",
                "id": "649",
            }
        ]
    },
    {
        "context": "Look for clothing stores that actively support charitable causes or donate a portion of their proceeds to social initiatives, making a positive impact through fashion.",
        "qas": [
            {
                "question": "How can I find clothing stores that support charitable causes or donate to social initiatives?",
                "id": "650",
            }
        ]
    },
    {
        "context": "Customers often engage with clothing brands through social media, customer reviews, and feedback forms to share their opinions and preferences.",
        "qas": [
            {
                "question": "How do customers typically engage with clothing brands to voice their opinions and preferences?",
                "id": "651",
            }
        ]
    },
    {
        "context": "Customer feedback is used to identify areas for improvement, refine product designs, and create offerings that align better with customer preferences.",
        "qas": [
            {
                "question": "Can you explain how clothing retailers use customer feedback to enhance product offerings?",
                "id": "652",
            }
        ]
    },
    {
        "context": "Clothing stores employ strategies like consistent pricing, inventory availability across channels, and responsive customer service to create a seamless shopping experience.",
        "qas": [
            {
                "question": "What strategies do clothing stores implement to create a seamless online and in-store shopping experience for their customers?",
                "id": "653",
            }
        ]
    },
    {
        "context": "Businesses protect customer data by implementing robust cybersecurity measures, encryption, and complying with data protection regulations to ensure security and privacy.",
        "qas": [
            {
                "question": "How do businesses ensure data security and privacy when collecting customer information in the clothing industry?",
                "id": "654",
            }
        ]
    },
    {
        "context": "Social media platforms provide a channel for clothing brands to engage with customers, gather feedback, and address concerns in real-time.",
        "qas": [
            {
                "question": "What role does social media play in customer interactions and feedback for clothing brands?",
                "id": "655",
            }
        ]
    },
    {
        "context": "Clothing retailers can encourage feedback through surveys, incentives, and user-friendly feedback platforms, making it convenient for customers to share their insights.",
        "qas": [
            {
                "question": "How can clothing retailers encourage customers to provide valuable feedback and insights?",
                "id": "656",
            }
        ]
    },
    {
        "context": "Building trust involves delivering on promises, transparency, and addressing issues promptly, ultimately fostering strong and credible customer relationships.",
        "qas": [
            {
                "question": "What are some best practices for building trust and credibility in customer relationships within the clothing industry?",
                "id": "657",
            }
        ]
    },
    {
        "context": "Personalization in marketing campaigns tailors content to individual customer preferences, increasing engagement and conversion rates for clothing brands.",
        "qas": [
            {
                "question": "Can you describe the role of personalization in marketing campaigns for clothing brands?",
                "id": "658",
            }
        ]
    },
    {
        "context": "Effective complaint management involves prompt acknowledgment, thorough investigation, resolution, and feedback, ensuring customer satisfaction.",
        "qas": [
            {
                "question": "How do businesses manage and resolve customer complaints effectively in the clothing industry?",
                "id": "659",
            }
        ]
    },
    {
        "context": "Strategies include loyalty programs, personalized discounts, and exceptional customer service to retain loyal customers and reduce churn.",
        "qas": [
            {
                "question": "What strategies can clothing stores employ to retain loyal customers and prevent churn?",
                "id": "660",
            }
        ]
    },
    {
        "context": "Data analytics help clothing retailers analyze customer behavior and preferences, enabling data-driven decision-making and personalized experiences.",
        "qas": [
            {
                "question": "How can clothing retailers use data analytics to understand customer behavior and preferences?",
                "id": "661",
            }
        ]
    },
    {
        "context": "Businesses benefit from CRM solutions with improved customer insights, streamlined processes, and the ability to nurture leads and retain customers effectively.",
        "qas": [
            {
                "question": "What benefits do businesses gain from implementing customer relationship management (CRM) solutions?",
                "id": "662",
            }
        ]
    },
    {
        "context": "Personalization in email marketing campaigns increases engagement and conversion rates by delivering relevant content tailored to individual preferences.",
        "qas": [
            {
                "question": "What is the impact of personalization in email marketing campaigns for clothing brands?",

                "id": "663",
            }
        ]
    },
    {
        "context": "Clothing brands should regularly update CRM strategies by monitoring customer trends, preferences, and feedback to remain responsive to changing needs.",
        "qas": [
            {
                "question": "How can clothing brands align their CRM strategies with changing customer behaviors and preferences?",
                "id": "664",
            }
        ]
    },
    {
        "context": "Omni-channel customer service ensures consistency and convenience across all customer touchpoints, whether in-store, online, or via mobile.",
        "qas": [
            {
                "question": "Can you explain the role of omni-channel customer service in the clothing industry?",
                "id": "665",
            }
        ]
    },
    {
        "context": "A successful CRM strategy includes customer data management, personalization, analytics, and a customer-centric culture within the organization.",
        "qas": [
            {
                "question": "What are the key components of a successful customer relationship management (CRM) strategy for clothing businesses?",
                "id": "666",
            }
        ]
    },
    {
        "context": "Businesses safeguard data security and privacy through robust cybersecurity measures, data encryption, and compliance with privacy regulations.",
        "qas": [
            {
                "question": "How can businesses ensure data security and privacy when managing customer information within the clothing industry?",
                "id": "667",
            }
        ]
    },
    {
        "context": "CRM allows clothing retailers to segment their customer base based on demographics, purchase history, and behavior, making it easier to tailor marketing efforts.",
        "qas": [
            {
                "question": "How do clothing retailers use CRM to segment their customer base for targeted marketing?",
                "id": "668",
            }
        ]
    },
    {
        "context": "Employee training and engagement are critical for providing excellent customer service, as motivated and well-informed staff are more likely to satisfy customer needs.",
        "qas": [
            {
                "question": "What role does employee training and engagement play in delivering excellent customer service in the clothing industry?",
                "id": "669",
            }
        ]
    },
    {
        "context": "Encouraging feedback involves creating user-friendly feedback platforms, conducting surveys, and offering incentives to make it easy for customers to share their insights.",
        "qas": [
            {
                "question": "How do clothing retailers encourage customers to provide valuable feedback and insights for continuous improvement?",
                "id": "670",
            }
        ]
    },
    {
        "context": "Omni-channel customer service ensures that customers experience consistent and convenient interactions across all channels, fostering strong and loyal relationships.",
        "qas": [
            {
                "question": "Can you describe the significance of omni-channel customer service and its role in customer relationships within the clothing industry?",
                "id": "671",
            }
        ]
    },
    {
        "context": "Best practices include prompt acknowledgment, thorough investigation, resolution, and seeking customer feedback to ensure complaints are resolved effectively.",
        "qas": [
            {
                "question": "What are some best practices for clothing retailers in managing and resolving customer complaints effectively?",
                "id": "672",
            }
        ]
    },
    {
        "context": "CRM systems enable businesses to identify and nurture leads by tracking interactions and automating lead scoring to prioritize potential customers.",
        "qas": [
            {
                "question": "How do businesses use CRM to identify and nurture potential leads in the clothing industry?",
                "id": "673",
            }
        ]
    },
    {
        "context": "Strategies for retaining loyal customers include offering rewards, personalized discounts, and exceptional customer service to reduce churn.",
        "qas": [
            {
                "question": "What strategies can clothing brands employ to retain loyal customers and prevent churn?",
                "id": "674",
            }
        ]
    },
    {
        "context": "Data analytics in CRM helps businesses analyze customer behavior patterns, enabling them to make data-driven decisions and personalize interactions.",
        "qas": [
            {
                "question": "Can you elaborate on the role of data analytics in CRM for understanding customer behavior and preferences in the clothing industry?",
                "id": "675",
            }
        ]
    },
    {
        "context": "Clothing retailers measure effectiveness through metrics such as customer satisfaction, retention rates, lead conversion, and return on investment (ROI).",
        "qas": [
            {
                "question": "How do clothing retailers measure the effectiveness of their CRM initiatives and customer relationships?",
                "id": "676",
            }
        ]
    },
    {
        "context": "Automation simplifies routine tasks, enhances efficiency, and ensures timely follow-ups, improving the overall customer experience.",
        "qas": [
            {
                "question": "What role does automation play in streamlining CRM processes for clothing businesses?",
                "id": "677",
            }
        ]
    },
    {
        "context": "Personalization involves segmenting customers, tailoring content, and delivering personalized offers, promotions, and recommendations.",
        "qas": [
            {
                "question": "How do clothing brands personalize marketing campaigns and promotions to cater to individual customer preferences?",
                "id": "678",
            }
        ]
    },
    {
        "context": "Customer feedback provides valuable insights into customer preferences and areas for improvement, helping shape CRM strategies.",
        "qas": [
            {
                "question": "Can you explain the role of customer feedback in shaping a company's CRM strategy in the clothing industry?",
                "id": "679",
            }
        ]
    },
    {
        "context": "Strategies include offering loyalty programs, conducting surveys, and tracking online behavior to collect and update customer data.",
        "qas": [
            {
                "question": "What strategies can clothing retailers employ to collect and update customer data for their CRM systems?",
                "id": "680",
            }
        ]
    },
    {
        "context": "Clothing brands protect customer data through data encryption, strict access controls, and compliance with privacy regulations to ensure responsible usage.",
        "qas": [
            {
                "question": "How can clothing brands ensure that customer data, including personal information and purchase history, is protected and used responsibly?",
                "id": "681",
            }
        ]
    },
    {
        "context": "Key components include customer data management, personalization, analytics, and a customer-centric culture within the organization.",
        "qas": [
            {
                "question": "What are the key components of a successful CRM strategy in the clothing industry?",
                "id": "682",
            }
        ]
    },
    {
        "context": "Social media platforms allow clothing brands to engage with customers, gather feedback, and address concerns in real-time, fostering strong customer relationships.",
        "qas": [
            {
                "question": "How does social media engagement contribute to customer relationships for clothing brands?",
                "id": "683",
            }
        ]
    },
    {
        "context": "Loyalty programs and incentives reward customers for repeat purchases, encourage brand loyalty, and boost customer retention.",
        "qas": [
            {
                "question": "What role do loyalty programs or incentives play in customer retention for clothing retailers?",
                "id": "684",
            }
        ]
    },
    {
        "context": "CRM systems help businesses track sales leads throughout the sales funnel, monitor progress, and measure lead conversion effectively.",
        "qas": [
            {
                "question": "How do businesses use CRM to track sales leads and conversions effectively in the clothing industry?",
                "id": "685",
            }
        ]
    },
    {
        "context": "Clothing retailers adapt by monitoring trends, offering new styles, and communicating with customers to understand their preferences.",
        "qas": [
            {
                "question": "How do clothing retailers adapt to changing fashion trends while maintaining strong customer relationships?",
                "id": "686",
            }
        ]
    },
    {
        "context": "Customer service representatives play a key role in resolving issues, addressing inquiries, and ensuring customers have a positive experience with the brand.",
        "qas": [
            {
                "question": " Can you explain the role of customer service representatives in managing and strengthening customer relationships in the clothing industry?",
                "id": "687",
            }
        ]
    },
    {
        "context": "Strategies include follow-up emails, post-purchase surveys, and personalized recommendations to maintain engagement after a sale.",
        "qas": [
            {
                "question": "What strategies can clothing brands employ to personalize the post-purchase experience and engage with customers beyond the sale?",
                "id": "688",
            }
        ]
    },
    {
        "context": "Ensuring consistency involves synchronizing inventory, pricing, and providing uniform customer service across all channels.",
        "qas": [
            {
                "question": "How can businesses in the clothing industry ensure that their online and offline customer experiences align seamlessly?",
                "id": "689",
            }
        ]
    },
    {
        "context": "Factors include product quality, personalized experiences, rewards programs, and excellent customer service that inspire loyalty and advocacy.",
        "qas": [
            {
                "question": "What are the key factors that influence customer loyalty and advocacy in the clothing industry?",
                "id": "690",
            }
        ]
    },
    {
        "context": "CRM systems track and analyze customer interactions, helping brands identify and address emerging concerns promptly.",
        "qas": [
            {
                "question": "How do clothing brands utilize CRM to identify and address emerging customer pain points and concerns?",
                "id": "691",
            }
        ]
    },
    {
        "context": "Community engagement events provide opportunities for customers to connect with the brand, fostering a sense of belonging and stronger relationships.",
        "qas": [
            {
                "question": "What role does community engagement, such as organizing events or fashion shows, play in strengthening customer relationships for clothing retailers?",
                "id": "692",
            }
        ]
    },
    {
        "context": "Effective returns and exchanges involve clear policies, hassle-free processes, and a focus on customer satisfaction to maintain positive relationships.",
        "qas": [
            {
                "question": "How can clothing brands effectively manage returns and exchanges while maintaining positive customer relationships?",
                "id": "693",
            }
        ]
    },
    {
        "context": "Personalized loyalty programs offer tailored rewards, discounts, and incentives that enhance customer retention and engagement.",
        "qas": [
            {
                "question": " Can you describe the impact of personalized loyalty programs on customer retention within the clothing industry?",
                "id": "694",
            }
        ]
    },
    {
        "context": "Clothing brands invest in employee training programs to ensure staff can provide exceptional service and build strong customer relationships.",
        "qas": [
            {
                "question": "What measures do clothing brands take to ensure that their employees are well-trained in delivering exceptional customer service?",
                "id": "695",
            }
        ]
    },
    {
        "context": "CRM systems help identify high-value customers, allowing brands to offer exclusive benefits and rewards to enhance customer loyalty.",
        "qas": [
            {
                "question": "How do clothing retailers use CRM to identify and prioritize their most valuable customers and offer exclusive benefits?",
                "id": "696",
            }
        ]
    },
    {
        "context": "Data analytics help brands analyze sales trends, customer preferences, and fashion shifts, allowing them to adjust inventory accordingly.",
        "qas": [
            {
                "question": "Can you explain the role of data analytics in identifying emerging fashion trends and adjusting inventory in the clothing industry?",
                "id": "697",
            }
        ]
    },
    {
        "context": "Strategies include transparent communication, eco-friendly product lines, and participation in social and environmental initiatives to engage customers in sustainability.",
        "qas": [
            {
                "question": "What strategies can clothing brands employ to communicate and involve customers in sustainability and ethical practices within the industry?",
                "id": "698",
            }
        ]
    },
    {
        "context": "Brands can implement regular data cleansing processes and encourage customers to update their information to maintain accurate CRM data.",
        "qas": [
            {
                "question": "How can clothing brands ensure that their customer data remains accurate and up-to-date for effective CRM practices?",
                "id": "699",
            }
        ]
    },
    {
        "context": "A feedback and improvement loop allows brands to continually gather insights, make necessary improvements, and demonstrate responsiveness to customer needs, strengthening relationships.",
        "qas": [
            {
                "question": "What role does a comprehensive feedback and improvement loop play in enhancing customer relationships for clothing retailers?",
                "id": "700",
            }
        ]
    }
] 

In [ ]:
#-----------F1 & EM---------------
# Initialize variables to keep track of F1 score and EM
total_f1_score = 0.0
total_em = 0.0
total_precision = 0.0  # Initialize total precision
f1_scores = []  # List to store individual F1 scores

# Define a dictionary to map IDs to true answers
true_answers = {
    "0": "OXGN is a fashion brand designed for all. From cool styles, affordable essentials and one of a kind collaboration pieces, our customers are sure to find that signature OXGN brand of style.",
    "1": "GOLDEN ABC, Inc. (GABC) is a multi-awarded international fashion enterprise that is home to top proprietary brands shaping the retail industry today.",
    "2": "The OXGN VIP Loyalty Card is not applicable online and it has been already been discontinued in our physical stores last December 31, 2021.",
    "3": "A confirmation e-mail/ SMS will be sent for all successful orders made. This contains your order number, the order amount, item(s) ordered, your billing and delivery address, including the payment method used and how you will be receiving order updates.",
    "4": "Yes, you have the option cancel your order within two (2) hours from the time it was created. You can see this on your order confirmation email or on your online store order history. Once the time frame has passed, the option to cancel will no longer appear on your end. If in case you still need to cancel your order but you no longer have the option to do so, you can send an email to shop@oxgnfashion.com and our Customer Care Team will get in touch with you.",
    "5": "We cannot include paper bags for Online Orders shipped to their chosen delivery address but we can definitely provide for those Online orders that were checked-out on a Store Pick-up option.",
    "6": "After you tap on REVIEW MY CART, you will see the option to send your order as a Gift. Sending it as a gift means that a gift receipt will be used instead of a regular sales invoice. A gift receipt shows proof of purchase but leaves out the amount spent.",
    "7": "Once the check out process has been completed, changes to the order in terms of item/size/color/quantity can no longer be made. What you can do is cancel your order completely within two (2) hours from the time you placed it so you can place a new one with the changes you want.",
    "8": "The first step is to click 'Add to Cart' once you've found something you want to save the items in your order basket. Second, click the 'Review My Cart' button to be redirected to the cart page, where you can make any final changes to your order, such as quantity, size, or style. Third, address Finder will appear on your screen. Please enter your delivery address by using the dropdown menus. If you accidentally exit out before finishing your details, click on 'Address Finder' to bring it back up. This is to prevent your orders from being held due to incorrect or incomplete shipping information.",
    "9": "Yes, you have the option cancel your order within two (2) hours from the time it was created either it is shipped to your prefered address or pick-up in your chosen You can view this on your order confirmation email or on your online store order history. Once the time frame has passed, the option to cancel will no longer appear on your end.",
    "10": "All unclaimed packages beyond the scheduled pick-up date are considered canceled and will be put back to our stocks.",
    "11": "You can now order online and pick up the items in your chosen OXGN branches: SM CITY NORTH EDSA, SM CITY SAN LAZARO, AYALA TRINOMA, ROBINSONS GALLERIA, SM MEGAMALL, SM CITY CEBU ,AYALA CENTER CEBU, MARKET MARKET, SM CITY FAIRVIEW, SM SEASIDE CEBU, SM LANANG, GMALL TAGUM, SM CITY DAVAO, GAISANO MALL OF DAVAO, SM CITY EAST ORTIGAS, SM MARIKINA, AYALA GLORIETTA, SM CITY MANILA, SM CITY GRAND CENTRAL, MALL OF ASIA, SM CITY SUCAT, ROBINSONS LAOAG, SM CITY BAGUIO, SM CITY PAMPANGA, SM CITY CLARK, SM CITY DASMARIÑAS, SM CALAMBA, SM CITY LUCENA, SM CITY LIPA, SM CITY ILO-ILO, SM CITY BACOLOD, SM Roxas, AYALA CENTRAL BLOC, Robinsons Tacloban, MINDPRO, ROBINSONS ILIGAN, Gaisano Mall Digos, SM CITY GENERAL SANTOS, KCC MALL MARBEL, SM CITY CAUAYAN, ISABELA, SM CITY MARILAO, TUTUBAN CENTER, Val Alabang, AYALA MALLS MANILA BAY, SM CITY LEGAZPI, ALTURAS MALL TAGBILARAN, SM PREMIER, SM BUTUAN, Mall of Alnor, SM City Tuguegarao, Ever Commonwelath, ROBINSONS BACOLOD, Fishermall.",
    "12": "As much as we'd like to, this option is not yet allowed at the moment.",
    "13": "This feature gives you the option to buy the items online and pick it up in our stores.",
    "14": "Oxgnfashion.com accepts the following forms of payment: GCash, Credit/Debit Card and COD (Cash-on-Delivery). During the order checkout process, you will be asked to select the payment method of your choice. If you select Credit/Debit Card, you will be redirected to our payment portal where you will be asked to enter your card details.",
    "15": "No, there is no option to save your credit or debit card information on your online store profile. Even if you have already used the same card, we do not store your card details to protect the security of your data.",
    "16": "Customers will receive an e-mail notification when their order is read for pick up. It is stated in the notification that they can pick up the order within 7 days from the receipt of e-mail.",
    "17": "No worries! You may assign a representative that may pick-up your order and have them present the following: Representative's Valid ID, Authorization letter with your Full Name and Signature, Photocopy of your Valid ID, and Screenshot of Order No. from email.",
    "18": "If you have placed order via our online store and chose to pick it up in one of branches, please prepare the following to be checked by our store partners: Valid ID, Screenshot of Order No. from email.",
    "19": "Customers with prepaid orders are allowed to request for up to 7 days extension from the first pick-up schedule.",
    "20": "If your order gets returned to our warehouse, we're sorry to let you know that it is already considered as canceled from our end. This is only done after two (2) unsuccessful delivery attempts.",
    "21": "Our delivery partners will make two (2) attempts in getting your package delivered. If they missed you on the first instance, the courier will either send you an SMS or give you a call to let you know that he came.",
    "22": "As we believe in making the latest fashion styles accessible to everyone, we can assure you that we have wide reach across the Philippines. However, due to geographical and logistical restrictions, there might be some isolated areas we cannot serve but rest assured we'll do our best to get to your location.",
    "23": "Customers who need to contact our shipping partners can do so using the following methods: For LBC, customers can send an email to customercare@lbcexpress.com or call (02) 8 858 59 99. For Go21, customers can visit the Inquiry Page at https://booking.go21.com.ph/ or call +63 2 832 99 27. For J&T, customers can visit https://www.jtexpress.sg/contact-us or call (02) 911 1888. For WSI, customers can send an email to communicationsupport@wsi.ph or call (02) 8533 3888. For feedback on our shipping partners or delivery concerns, customers can send an email to shop@oxgnfashion.com or fill out the Website Contact Us Form.",
    "24": "Once your order has been handed over to our shipping partners, you will receive an email notification along with your order tracking number.",
    "25": "Customers who wish to return items purchased from our online store can do so by packing the item along with the sales invoice and bringing it to any Penshoppe store nationwide. The return must be made within 30 days from the date of purchase, with all tags attached and the item unused and unwashed.",
    "26": "If the same item is not available, you do not have to worry as you can choose any item inside the store. If the new item (different from what was originally purchased) chosen for exchange is of higher value than what was originally purchased, the price difference must be paid at the time of the exchange.",
    "27": "If you receive a gift item from our online store and want to have it exchanged for a different size/color/style, you have the option to do so following our Returns Policy.",
    "28": "If you need to process a return or exchange transaction and have lost your online store receipt, please reach out to us with the following information: your order number, the item you will be returning, the reason for the return, the boutique you will be going to, and the date.",
    "29": "No, our warehouse does not accept returns from customers but our physical stores do.",
    "30": "In the event that you received a damaged or incorrect item, it is important to reach out to us immediately so that we can investigate the situation.",
    "31": "Refund requests are always subject to validation. It is important to note that a change in size, style, or color is not eligible for a refund.",
    "32": "Creating an account on the OXGN Online Store is easy and allows you to keep track of your orders through the Order History feature.",
    "33": "Yes, you can still shop without making an online store account! Just place the items you want in your cart and proceed to check out.",
    "34": "If you have forgotten your password for your online store account, don't worry as it is an easy process to reset it.",
    "35": "First time to sign in? Tap on the account icon so you can start adding your account details.",
    "36": "Reach out to +639399101047 or complete the form below and we’ll gladly assist you.",
    "37": "These Terms and Conditions ('Terms', 'Terms and Conditions') govern your relationship with OXGN Website (the 'Service') operated by Golden ABC, Incorporated ('us', 'we', or 'our'). Please read these Terms and Conditions carefully before using our OXGN Website(the 'Service'). Your access to and use of the Service is conditioned on your acceptance of and compliance with these Terms. These Terms apply to all visitors, users and others who access or use the Service. By accessing or using the Service you agree to be bound by these Terms. If you disagree with any part of the terms then you may not access the Service. These Terms constitute the entire agreement between us regarding our Service, and supersede and replace any prior agreements we might have between us regarding the Service. CHANGES, We reserve the right, at our sole discretion, to modify or replace these Terms at any time. If a revision is material we will try to provide at least 30 days notice prior to any new terms taking effect. What constitutes a material change will be determined at our sole discretion. By continuing to access or use our Service after those revisions become effective, you agree to be bound by the revised terms. If you do not agree to the new terms, please stop using the Service. CONTACT US,If you have any questions about these Terms, please contact us.",
    "38": "There are ten available variations of tops for men: Regular Fit Shirt, Resort Fit Shirt, Long Sleeve, Hoodie, Relax set-in Pullover, Easy Fit, Jersy, Slim Fit, Oversized, and Tank Top.",
    "39": "There are five available variations of bottoms for men: Skinny, Tapered (Slim Fit, Woven, Knit), and Jogger Pants",
    "40": "There are two available variations of loungewear for men: Robe and Lounge Set.",
    "41": "There are six available variations of footwear for men: Sliders, Single Band, Velcro Sliders, Regular Fliptops, Lace-up, and Lace-up Runner.",
    "42": "There are seven available variations of tops for ladies: Regular, Boxy, Slim fit Tee, Shirt (Short Sleeve), Sports Bra, Hoodie, and Regular Pullover.",
    "43": "There are two available variations of dress for ladies: Oversized Dress, Long Sleeve Regular Fit.",
    "44": "There are ten available variations of bottoms for ladies: Skinny Jeans (Numerical,  Alpha), Wide Leg Jeans, Shorts (Numerical, Alpha), Biker Shorts (Alpha), Leggings (With Pockets, Without Pockets), Track Pants, and Urban Tapered Trousers.",
    "45": "There are four available variations of footwear for ladies: Sliders, Single Band Sliders, Velcro Sliders, and Lace-up Sneakers.",
    "46": "There are three available categories of accessories in total: Bags, Caps and Hats, and Wallet and Purses.",
    "47": "There are four available variations of Bags in accessories: Drawstring Backpack, Bum Bag, Sling Bag, and Tote Bag.",
    "48": "There are two available variations of Caps/Hat in accessories: Trucker Hat and Bucket Hat.",
    "49": "There are three available variations of walltes/purses in accessories: Coin Purse (Horizontal & Vertical) and Midsized Bifold Wallet.",
    "50": "For men's top regular fit shirt in double extra small, Chest: 37, Body Length: 26, Sleeve Length: 7.75, USA: 10, EU: 44, and UK: 34.",
    "51": "For men's top regular fit shirt in extra small, Chest: 39, Body Length: 27, Sleeve Length: 8, USA: 12, EU: 46, and UK: 36.",
    "52": "For men's top regular fit shirt in small, Chest: 41, Body Length: 28, Sleeve Length: 8.25, USA: 14, EU: 48, and UK: 38.",
    "53": "For men's top regular fit shirt in medium, Chest: 43, Body Length: 29, Sleeve Length: 8.5, USA: 16, EU: 50, and UK: 40.",
    "54": "For men's top regular fit shirt in large, Chest: 45, Body Length: 30, Sleeve Length: 8.75, USA: 18, EU: 52, and UK: 42.",
    "55": "For men's top regular fit shirt in extra large, Chest: 47, Body Length: 31, Sleeve Length: 9, USA: 20, EU: 54, and UK: 44.",
    "56": "For men's top regular fit shirt in double extra large, Chest: 49, Body Length: 32, Sleeve Length: 9.25, USA: 22, EU: 56, and UK: 46.",
    "57": "For men's top resort fit shirt in double extra small, Chest: 39, Body Length: 27.5, Sleeve Length: 9.25, USA: 10, EU: 44, and UK: 34.",
    "58": "For men's top resort fit shirt in extra small, Chest: 41, Body Length: 28, Sleeve Length: 9.5, USA: 12, EU: 46, and UK: 36.",
    "59": "For men's top resort fit shirt in small, Chest: 43, Body Length: 28.5, Sleeve Length: 9.75, USA: 14, EU: 48, and UK: 38.",
    "60": "For men's top resort fit shirt in medium, Chest: 45, Body Length: 29, Sleeve Length: 10, USA: 16, EU: 50, and UK: 40.",
    "61": "For men's top resort fit shirt in large, Chest: 47, Body Length: 29.5, Sleeve Length: 10.25, USA: 18, EU: 52, and UK: 42.",
    "62": "For men's top resort fit shirt in extra large, Chest: 49, Body Length: 30, Sleeve Length: 10.5, USA: 20, EU: 54, and UK: 44.",
    "63": "For men's top resort fit shirt in double extra large, Chest: 51, Body Length: 30.5, Sleeve Length: 10.75, USA: 22, EU: 56, and UK: 46.",
    "64": "For men's top long sleeve shirt in double extra small, Chest: 36, Body Length: 26, Sleeve Length: 24.75, USA: 10, EU: 44, and UK: 34.",
    "65": "For men's top long sleeve shirt in extra small, Chest: 38, Body Length: 27, Sleeve Length: 25, USA: 12, EU: 46, and UK: 36.",
    "66": "For men's top long sleeve shirt in small, Chest: 40, Body Length: 28, Sleeve Length: 25.25, USA: 14, EU: 48, and UK: 38.",
    "67": "For men's top long sleeve shirt in medium, Chest: 42, Body Length: 29, Sleeve Length: 25.5, USA: 16, EU: 50, and UK: 40.",
    "68": "For men's top long sleeve shirt in large, Chest: 44, Body Length: 30, Sleeve Length: 25.75, USA: 18, EU: 52, and UK: 42.",
    "69": "For men's top long sleeve shirt in extra large, Chest: 46, Body Length: 31, Sleeve Length: 26, USA: 20, EU: 54, and UK: 44.",
    "70": "For men's top long sleeve shirt in double extra large, Chest: 48, Body Length: 32, Sleeve Length: 26.25, USA: 22, EU: 56, and UK: 46.",
    "71": "For men's top long sleeve shirt in triple extra large, Chest: 50, Body Length: 33, Sleeve Length: 26.5, USA: 24, EU: 58, and UK: 48.",
    "72": "For men's top hoodie dropped shoulder shirt in double extra small, Chest: 44, Body Length: 25.5, Sleeve Length: 17.25, USA: 10, EU: 44, and UK: 34.",
    "73": "For men's top hoodie dropped shoulder shirt in extra small, Chest: 46, Body Length: 26, Sleeve Length: 17.5, USA: 12, EU: 46, and UK: 36.",
    "74": "For men's top hoodie dropped shoulder shirt in small, Chest: 48, Body Length: 26.5, Sleeve Length: 17.75, USA: 14, EU: 48, and UK: 38.",
    "75": "For men's top hoodie dropped shoulder shirt in medium, Chest: 50, Body Length: 27, Sleeve Length: 18, USA: 16, EU: 50, and UK: 40.",
    "76": "For men's top hoodie dropped shoulder shirt in large, Chest: 52, Body Length: 27.5, Sleeve Length: 18.25, USA: 18, EU: 52, and UK: 42.",
    "77": "For men's top hoodie dropped shoulder shirt in extra large, Chest: 54, Body Length: 28, Sleeve Length: 18.5, USA: 20, EU: 54, and UK: 44.",
    "78": "For men's top hoodie dropped shoulder shirt in double extra large, Chest: 56, Body Length: 28.5, Sleeve Length: 18.75, USA: 22, EU: 56, and UK: 46.",
    "79": "For men's top hoodie dropped shoulder shirt in triple extra large, Chest: 58, Body Length: 29, Sleeve Length: 19, USA: 24, EU: 58, and UK: 48.",
    "80": "For men's top hoodie set-in shoulder shirt in double extra small, Chest: 38, Body Length: 25, Sleeve Length: 24.25, USA: 10, EU: 44, and UK: 34.",
    "81": "For men's top hoodie set-in shoulder shirt in extra small, Chest: 40, Body Length: 26, Sleeve Length: 24.5, USA: 12, EU: 46, and UK: 36.",
    "82": "For men's top hoodie set-in shoulder shirt in small, Chest: 42, Body Length: 27, Sleeve Length: 24.75, USA: 14, EU: 48, and UK: 38.",
    "83": "For men's top hoodie set-in shoulder shirt in medium, Chest: 44, Body Length: 28, Sleeve Length: 25, USA: 16, EU: 50, and UK: 40.",
    "84": "For men's top hoodie set-in shoulder shirt in large, Chest: 46, Body Length: 29, Sleeve Length: 25.25, USA: 18, EU: 52, and UK: 42.",
    "85": "For men's top hoodie set-in shoulder shirt in extra large, Chest: 48, Body Length: 30, Sleeve Length: 25.5, USA: 20, EU: 54, and UK: 44.",
    "86": "For men's top hoodie set-in shoulder shirt in double extra large, Chest: 50, Body Length: 31, Sleeve Length: 25.75, USA: 22, EU: 56, and UK: 46.",
    "87": "For men's top hoodie set-in shoulder shirt in triple extra large, Chest: 52, Body Length: 32, Sleeve Length: 26, USA: 24, EU: 58, and UK: 48.",
    "88": "For men's top relax set-in pullover shirt in double extra small, Chest: 19, Body Length: 26, Sleeve Length: 24.25, USA: 10, EU: 44, and UK: 34.",
    "89": "For men's top relax set-in pullover shirt in extra small, Chest: 20, Body Length: 27, Sleeve Length: 24.5, USA: 12, EU: 46, and UK: 36.",
    "90": "For men's top relax set-in pullover shirt in small, Chest: 21, Body Length: 27, Sleeve Length: 24.75, USA: 14, EU: 48, and UK: 38.",
    "91": "For men's top relax set-in pullover shirt in medium, Chest: 22, Body Length: 28, Sleeve Length: 25, USA: 16, EU: 50, and UK: 40.",
    "92": "For men's top relax set-in pullover shirt in large, Chest: 23, Body Length: 29, Sleeve Length: 25.25, USA: 18, EU: 52, and UK: 42.",
    "93": "For men's top relax set-in pullover shirt in extra large, Chest: 24, Body Length: 30, Sleeve Length: 25.5, USA: 20, EU: 54, and UK: 44.",
    "94": "For men's top relax set-in pullover shirt in double extra large, Chest: 25, Body Length: 31, Sleeve Length: 25.75, USA: 22, EU: 56, and UK: 46.",
    "95": "For men's tee easy fit shirt in double extra small, Chest: 18, Body Length: 26, Sleeve Length: 7.25, USA: 10, EU: 44, and UK: 34.",
    "96": "For men's tee easy fit shirt in extra small, Chest: 19, Body Length: 26, Sleeve Length: 7.5, USA: 12, EU: 46, and UK: 36.",
    "97": "For men's tee easy fit shirt in small, Chest: 20, Body Length: 27, Sleeve Length: 7.75, USA: 14, EU: 48, and UK: 38.",
    "98": "For men's tee easy fit shirt in medium, Chest: 21, Body Length: 28, Sleeve Length: 8, USA: 16, EU: 50, and UK: 40.",
    "99": "For men's tee easy fit shirt in large, Chest: 22, Body Length: 29, Sleeve Length: 8.25, USA: 18, EU: 52, and UK: 42.",
    "100": "For men's tee easy fit shirt in extra large, Chest: 23, Body Length: 30, Sleeve Length: 8.5, USA: 20, EU: 54, and UK: 44.",
    "101": "For men's tee easy fit shirt in double extra large, Chest: 24, Body Length: 31, Sleeve Length: 8.75, USA: 22, EU: 56, and UK: 46.",
    "102": "For men's top jersey with raglan sleeves shirt in double extra small, Chest: 36, Body Length: 25, Sleeve Length: 13.5, USA: 10, EU: 44, and UK: 34.",
    "103": "For men's top jersey with raglan sleeves shirt in extra small, Chest: 38, Body Length: 26, Sleeve Length: 14, USA: 12, EU: 46, and UK: 36.",
    "104": "For men's top jersey with raglan sleeves shirt in small, Chest: 40, Body Length: 27, Sleeve Length: 14.5, USA: 14, EU: 48, and UK: 38.",
    "105": "For men's top jersey with raglan sleeves shirt in medium, Chest: 42, Body Length: 28, Sleeve Length: 15, USA: 16, EU: 50, and UK: 40.",
    "106": "For men's top jersey with raglan sleeves shirt in large, Chest: 44, Body Length: 29, Sleeve Length: 15.5, USA: 18, EU: 52, and UK: 42.",
    "107": "For men's top jersey with raglan sleeves shirt in extra large, Chest: 46, Body Length: 30, Sleeve Length: 16, USA: 20, EU: 54, and UK: 44.",
    "108": "For men's top jersey with raglan sleeves shirt in double extra large, Chest: 48, Body Length: 31, Sleeve Length: 16.5, USA: 22, EU: 56, and UK: 46.",
    "109": "For men's top jersey with raglan sleeves shirt in triple extra large, Chest: 50, Body Length: 32, Sleeve Length: 17, USA: 24, EU: 58, and UK: 48.",
    "110": "For men's tee slim fit shirt in double extra small, Chest: 17, Body Length: 25.5, Sleeve Length: 6.75, USA: 10, EU: 44, and UK: 34.",
    "111": "For men's tee slim fit shirt in extra small, Chest: 18, Body Length: 26, Sleeve Length: 7, USA: 12, EU: 46, and UK: 36.",
    "112": "For men's tee slim fit shirt in small, Chest: 19, Body Length: 26.5, Sleeve Length: 7.25, USA: 14, EU: 48, and UK: 38.",
    "113": "For men's tee slim fit shirt in medium, Chest: 20, Body Length: 27, Sleeve Length: 7.5, USA: 16, EU: 50, and UK: 40.",
    "114": "For men's tee slim fit shirt in large, Chest: 21, Body Length: 27.5, Sleeve Length: 7.75, USA: 18, EU: 52, and UK: 42.",
    "115": "For men's tee slim fit shirt in extra large, Chest: 22, Body Length: 28, Sleeve Length: 8, USA: 20, EU: 54, and UK: 44.",
    "116": "For men's tee slim fit shirt in double extra large, Chest: 23, Body Length: 28.5, Sleeve Length: 8.25, USA: 22, EU: 56, and UK: 46.",
    "117": "For men's tee oversized fit shirt in double extra small, Chest: 19, Body Length: 27, Sleeve Length: 8.5, USA: 10, EU: 44, and UK: 34.",
    "118": "For men's tee oversized fit shirt in extra small, Chest: 20, Body Length: 28, Sleeve Length: 8.75, USA: 12, EU: 46, and UK: 36.",
    "119": "For men's tee oversized fit shirt in small, Chest: 21, Body Length: 29, Sleeve Length: 9, USA: 14, EU: 48, and UK: 38.",
    "120": "For men's tee oversized fit shirt in medium, Chest: 22, Body Length: 30, Sleeve Length: 9.25, USA: 16, EU: 50, and UK: 40.",
    "121": "For men's tee oversized fit shirt in large, Chest: 23, Body Length: 31, Sleeve Length: 9.5, USA: 18, EU: 52, and UK: 42.",
    "122": "For men's tee oversized fit shirt in extra large, Chest: 24, Body Length: 32, Sleeve Length: 9.75, USA: 20, EU: 54, and UK: 44.",
    "123": "For men's tee oversized fit shirt in double extra large, Chest: 25, Body Length: 33, Sleeve Length: 10, USA: 22, EU: 56, and UK: 46.",
    "124": "For men's top tank top shirt in double extra small, Chest: 36, Body Length: 26, Sleeve Length: N/A, USA: 10, EU: 44, and UK: 34.",
    "125": "For men's top tank top shirt in extra small, Chest: 38, Body Length: 27, Sleeve Length: N/A, USA: 12, EU: 46, and UK: 36.",
    "126": "For men's top tank top shirt in small, Chest: 40, Body Length: 28, Sleeve Length: N/A, USA: 14, EU: 48, and UK: 38.",
    "127": "For men's top tank top shirt in medium, Chest: 42, Body Length: 29, Sleeve Length: N/A, USA: 16, EU: 50, and UK: 40.",
    "128": "For men's top tank top shirt in large, Chest: 44, Body Length: 30, Sleeve Length: N/A, USA: 18, EU: 52, and UK: 42.",
    "129": "For men's top tank top shirt in extra large, Chest: 46, Body Length: 31, Sleeve Length: N/A, USA: 20, EU: 54, and UK: 44.",
    "130": "For men's top tank top shirt in double extra large, Chest: 48, Body Length: 32, Sleeve Length: N/A, USA: 22, EU: 56, and UK: 46.",
    "131": "For men's jeans skinny fit (regular waist band) in size 29, Waist: 28, Hips: 34, Inseam: 28, USA: 38, EU: 38, and UK: 30.",
    "132": "For men's jeans skinny fit (regular waist band) in size 30, Waist: 29, Hips: 35, Inseam: 28, USA: 39, EU: 39, and UK: 30-31.",
    "133": "For men's jeans skinny fit (regular waist band) in size 31, Waist: 30, Hips: 36, Inseam: 28, USA: 40, EU: 40, and UK: 31.",
    "134": "For men's jeans skinny fit (regular waist band) in size 32, Waist: 31, Hips: 37, Inseam: 29, USA: 42, EU: 42, and UK: 32.",
    "135": "For men's jeans skinny fit (regular waist band) in size 34, Waist: 31, Hips: 37, Inseam: 29, USA: 42, EU: 42, and UK: 32.",
    "136": "For men's jeans slim fit (regular waist band) in size 29, Waist: 27, Hips: 36, Inseam: 28, USA: 38, EU: 38, and UK: 30.",
    "137": "For men's jeans slim fit (regular waist band) in size 30, Waist: 28, Hips: 37, Inseam: 28, USA: 39, EU: 39, and UK: 30-31.",
    "138": "For men's jeans slim fit (regular waist band) in size 31, Waist: 29, Hips: 38, Inseam: 28, USA: 40, EU: 40, and UK: 31.",
    "139": "For men's jeans slim fit (regular waist band) in size 32, Waist: 30, Hips: 39, Inseam: 29, USA: 42, EU: 42, and UK: 32.",
    "140": "For men's jeans slim fit (regular waist band) in size 34, Waist: 32, Hips: 41, Inseam: 29, USA: 44, EU: 44, and UK: 34.",
    "141": "For men's jeans jogger pants (garterized waist band) in size extra small, Waist: 28, Hips: 38, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "142": "For men's jeans jogger pants (garterized waist band) in size small, Waist: 30, Hips: 40, Inseam: 25, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "143": "For men's jeans jogger pants (garterized waist band) in size medium, Waist: 32, Hips: 42, Inseam: 25, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "144": "For men's jeans jogger pants (garterized waist band) in size large, Waist: 34, Hips: 44, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.",
    "145": "For men's jeans jogger pants (garterized waist band) in size extra large, Waist: 36, Hips: 46, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.",
    "146": "For men's tapered pants woven (regular waist band) in size extra small, Waist: 24, Hips: 37, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "147": "For men's tapered pants woven (regular waist band) in size small, Waist: 26, Hips: 39, Inseam: 25, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "148": "For men's tapered pants woven (regular waist band) in size medium, Waist: 28, Hips: 41, Inseam: 25, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "149": "For men's tapered pants woven (regular waist band) in size large, Waist: 30, Hips: 43, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.",
    "150": "For men's tapered pants woven (regular waist band) in size extra large, Waist: 32, Hips: 45, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.",
    "151": "For men's tapered pants knit (garterized waist band) in size extra small, Waist: 26, Hips: 38, Inseam: 26, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "152": "For men's tapered pants knit (garterized waist band) in size small, Waist: 28, Hips: 40, Inseam: 26, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "153": "For men's tapered pants knit (garterized waist band) in size medium, Waist: 30, Hips: 42, Inseam: 26, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "154": "For men's tapered pants knit (garterized waist band) in size large, Waist: 32, Hips: 44, Inseam: 26, USA: 43-44, EU: 43-44, and UK: 33-34.",
    "155": "For men's tapered pants knit (garterized waist band) in size extra large, Waist: 34, Hips: 46, Inseam: 26, USA: 45-46, EU: 45-46, and UK: 35-36.",
    "156": "For men's tapered pants with cuff-knit (garterized waist band) in size double extra small, Waist: 23, Hips: 36, Inseam: 25, USA: 32-34, EU: 32-34, and UK: 27-28.",
    "157": "For men's tapered pants with cuff-knit (garterized waist band) in size extra small, Waist: 25, Hips: 38, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "158": "For men's tapered pants with cuff-knit (garterized waist band) in size small, Waist: 27, Hips: 40, Inseam: 25, USA: 39-40, EU: 39-40 and UK: 30-31.",
    "159": "For men's tapered pants with cuff-knit (garterized waist band) in size medium, Waist: 29, Hips: 42, Inseam: 25, USA: 43-44, EU: 43-44 and UK: 31-32.",
    "160": "For men's tapered pants with cuff-knit (garterized waist band) in size large, Waist: 31, Hips: 44, Inseam: 25, USA: 43-44, EU: 43-44 and UK: 33-34.",
    "161": "For men's tapered pants with cuff-knit (garterized waist band) in size extra large, Waist: 33, Hips: 46, Inseam: 25, USA: 45-46, EU: 45-46 and UK: 35-36.",
    "162": "For men's tapered pants with cuff-knit (garterized waist band) in size double extra large, Waist: 35, Hips: 48, Inseam: 25, USA: 48-50, EU: 48-50 and UK: 37-38.",
    "163": "For men's urban shorts - knit & woven (garterized waist band) in size extra small, Waist: 28, Hips: 40, Inseam: 5, USA: 36-38, EU: 36-38 and UK: 29-30.",
    "164": "For men's urban shorts - knit & woven (garterized waist band) in size small, Waist: 30, Hips: 42, Inseam: 5, USA: 39-40, EU: 39-40 and UK: 30-31.",
    "165": "For men's urban shorts - knit & woven (garterized waist band) in size medium, Waist: 32, Hips: 44, Inseam: 5, USA: 40-42, EU: 40-42 and UK: 31-32.",
    "166": "For men's urban shorts - knit & woven (garterized waist band) in size large, Waist: 34, Hips: 46, Inseam: 5, USA: 43-44, EU: 43-44 and UK: 33-34.",
    "167": "For men's urban shorts - knit & woven (garterized waist band) in size extra large, Waist: 36, Hips: 48, Inseam: 5, USA: 45-46, EU: 45-46 and UK: 35-36.",
    "168": "For men's chino shorts - woven (regular waist band) in size 28, Waist: 28, Hips: 38, Inseam: 5, USA: 36, EU: 36, and UK: 29.",
    "169": "For men's chino shorts - woven (regular waist band) in size 29, Waist: 29, Hips: 39, Inseam: 5, USA: 38, EU: 38, and UK: 30.",
    "170": "For men's chino shorts - woven (regular waist band) in size 30, Waist: 30, Hips: 40, Inseam: 5, USA: 39, EU: 39, and UK: 30-31.",
    "171": "For men's chino shorts - woven (regular waist band) in size 31, Waist: 31, Hips: 41, Inseam: 5, USA: 40, EU: 40, and UK: 31.",
    "172": "For men's chino shorts - woven (regular waist band) in size 32, Waist: 32, Hips: 42, Inseam: 5, USA: 42, EU: 42, and UK: 32.",
    "173": "For men's chino shorts - woven (regular waist band) in size 34, Waist: 34, Hips: 44, Inseam: 5, USA: 44, EU: 44, and UK: 34.",
    "174": "For men's walk/board shorts - woven (garterized waist band) in size extra small, Waist: 28, Hips: 41, Inseam: 4.5, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "175": "For men's walk/board shorts - woven (garterized waist band) in size small, Waist: 30, Hips: 43, Inseam: 4.5, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "176": "For men's walk/board shorts - woven (garterized waist band) in size medium, Waist: 32, Hips: 45, Inseam: 4.5, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "177": "For men's walk/board shorts - woven (garterized waist band) in size large, Waist: 34, Hips: 47, Inseam: 4.5, USA: 43-44, EU: 43-44, and UK: 33-34.",
    "178": "For men's walk/board shorts - woven (garterized waist band) in size extra large, Waist: 36, Hips: 49, Inseam: 4.5, USA: 45-46, EU: 45-46, and UK: 35-36.",
    "179": "For men's boxer shorts (garterized waist band) in size double extra small, Waist: 20, Hips: 35, Inseam: 3.25, USA: 32, EU: 42, and UK: 8.",
    "180": "For men's boxer shorts (garterized waist band) in size extra small, Waist: 22, Hips: 36, Inseam: 3.25, USA: 34, EU: 44, and UK: 10.",
    "181": "For men's boxer shorts (garterized waist band) in size small, Waist: 24, Hips: 38, Inseam: 3.5, USA: 36, EU: 46, and UK: 12.",
    "182": "For men's boxer shorts (garterized waist band) in size medium, Waist: 26, Hips: 40, Inseam: 3.5, USA: 38, EU: 48, and UK: 14.",
    "183": "For men's boxer shorts (garterized waist band) in size large, Waist: 28, Hips: 42, Inseam: 3.75, USA: 40, EU: 50, and UK: 16.",
    "184": "For men's boxer shorts (garterized waist band) in size extra large, Waist: 30, Hips: 44, Inseam: 3.75, USA: 42, EU: 52, and UK: 18.",
    "185": "For men's boxer shorts (garterized waist band) in size double extra large, Waist: 32, Hips: 46, Inseam: 3.75, USA: 44, EU: 54, and UK: 20.",
    "186": "For men's lounge - robe in size small, Waist: 44, Hips: 40, Inseam: 19.5, USA: 12-16, EU: 46-50, and UK: 36-40.",
    "187": "For men's lounge - robe in size medium, Waist: 48, Hips: 42, Inseam: 19.5, USA: 18-20, EU: 52-54, and UK: 42-44.",
    "188": "For men's lounge set - top slim fit in size double extra small, Waist: 34, Hips: 25.5, Inseam: 6.75, USA: 10, EU: 44, and UK: 34.",
    "189": "For men's lounge set - top slim fit in size extra small, Waist: 36, Hips: 26, Inseam: 7, USA: 12, EU: 46, and UK: 36.",
    "190": "For men's lounge set - top slim fit in size small, Waist: 38, Hips: 26.5, Inseam: 7.25, USA: 14, EU: 48, and UK: 38.",
    "191": "For men's lounge set - top slim fit in size medium, Waist: 40, Hips: 27, Inseam: 7.5, USA: 16, EU: 50, and UK: 40.",
    "192": "For men's lounge set - top slim fit in size large, Waist: 42, Hips: 27.5, Inseam: 7.75, USA: 18, EU: 52, and UK: 42.",
    "193": "For men's lounge set - top slim fit in size extra large, Waist: 44, Hips: 28, Inseam: 8, USA: 20, EU: 54, and UK: 44.",
    "194": "For men's lounge set - top slim fit in size double extra large, Waist: 46, Hips: 28.5, Inseam: 8.25, USA: 22, EU: 56, and UK: 46.",
    "195": "For men's lounge set - bottom boxer shorts in size double extra small, Waist: 20, Hips: 36, Inseam: 3.25, USA: 33-35, EU: 33-35, and UK: 27-28.",
    "196": "For men's lounge set - bottom boxer shorts in size extra small, Waist: 22, Hips: 38, Inseam: 3.25, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "197": "For men's lounge set - bottom boxer shorts in size small, Waist: 24, Hips: 40, Inseam: 3.5, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "198": "For men's lounge set - bottom boxer shorts in size medium, Waist: 26, Hips: 42, Inseam: 3.5, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "199": "For men's lounge set - bottom boxer shorts in size large, Waist: 28, Hips: 44, Inseam: 3.75, USA: 43-44, EU: 43-44, and UK: 33-34.",
    "200": "For men's lounge set - bottom boxer shorts in size extra large, Waist: 30, Hips: 46, Inseam: 3.75, USA: 45-46, EU: 45-46, and UK: 35-36.",
    "201": "For men's lounge set - bottom boxer shorts in size double extra large, Waist: 32, Hips: 48, Inseam: 3.75, USA: 47-48, EU: 47-48, and UK: 37-38.",
    "202": "For men's footwear sliders unisex in size small, Ladies' US Size: 5, Men's US Size: N/A, Footbed Length: 23.5cm.",
    "203": "For men's footwear sliders unisex in size medium, Ladies' US Size: 6-7, Men's US Size: N/A, Footbed Length: 24.9cm.",
    "204": "For men's footwear sliders unisex in size large, Ladies' US Size: 8-9, Men's US Size: 6, Footbed Length: 26.3cm.",
    "205": "For men's footwear sliders unisex in size extra large, Ladies' US Size: N/A, Men's US Size: 7-8, Footbed Length: 27.5cm.",
    "206": "For men's footwear sliders unisex in size double extra large, Ladies' US Size: N/A, Men's US Size: 9-10, Footbed Length: 28.9cm.",
    "207": "For men's footwear single band sliders, (US: 6, EU: 39, CM: 26.1), (US: 7, EU: 40, CM: 26.8), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28.2), (US: 10, EU: 43, CM: 28.9).",
    "208": "For men's footwear velcro sliders, (US: 6, EU: 39, CM: 26.1), (US: 7, EU: 40, CM: 26.8), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28.2), (US: 10, EU: 43, CM: 28.5).",
    "209": "For men's regular flipflops, (US: 6, EU: 39, CM: 26.6), (US: 7, EU: 40, CM: 27.3), (US: 8, EU: 41, CM: 28), (US: 9, EU: 42, CM: 28.7), (US: 10, EU: 43, CM: 29.4), then the slippers thicknes is 15mm.",
    "210": "For men's lace-up sneakers, (US: 6, EU: 39, CM: 26.5), (US: 7, EU: 40, CM: 27), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28), (US: 10, EU: 43, CM: 28.5).",
    "211": "For men's lace-up runner shoes, (US: 6, EU: 39, CM: 26.5), (US: 7, EU: 40, CM: 27), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28), (US: 10, EU: 43, CM: 28.5).",
    "212": "For ladies' top tee - regular fit in size extra small, Chest: 34, Body Length: 22.5, Sleeve Length: 7, USA: 2, EU: 30, and UK: 6.",
    "213": "For ladies' top tee - regular fit in size small, Chest: 36, Body Length: 23, Sleeve Length: 7.25, USA: 4, EU: 32, and UK: 8.",
    "214": "For ladies' top tee - regular fit in size medium, Chest: 38, Body Length: 23.5, Sleeve Length: 7.5, USA: 6, EU: 36, and UK: 10.",
    "215": "For ladies' top tee - regular fit in size large, Chest: 40, Body Length: 24, Sleeve Length: 7.75, USA: 8, EU: 40, and UK: 12.",
    "216": "For ladies' top tee - regular fit in size extra large, Chest: 42, Body Length: 24.5, Sleeve Length: 8, USA: 10, EU: 42, and UK: 42.",
    "217": "For ladies' top tee - boxy fit in size extra small, Chest: 39, Body Length: 19, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
    "218": "For ladies' top tee - boxy fit in size small, Chest: 41, Body Length: 19.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
    "219": "For ladies' top tee - boxy fit in size medium, Chest: 43, Body Length: 20, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.",
    "220": "For ladies' top tee - boxy fit in size large, Chest: 45, Body Length: 20.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
    "221": "For ladies' top tee - boxy fit in size extra large, Chest: 47, Body Length: 21, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
    "222": "For ladies' v-neck - slim fit in size extra small, Chest: 31, Body Length: 22.5, Sleeve Length: 7.25, USA: 2, EU: 30, and UK: 6.",
    "223": "For ladies' v-neck - slim fit in size small, Chest: 33, Body Length: 23, Sleeve Length: 7.5, USA: 4, EU: 32, and UK: 8.",
    "224": "For ladies' v-neck - slim fit in size medium, Chest: 35, Body Length: 23.5, Sleeve Length: 7.75, USA: 6, EU: 36, and UK: 10.",
    "225": "For ladies' v-neck - slim fit in size large, Chest: 37, Body Length: 24, Sleeve Length: 8, USA: 8, EU: 40, and UK: 12.",
    "226": "For ladies' v-neck - slim fit in size extra large, Chest: 39, Body Length: 24.5, Sleeve Length: 8.25, USA: 10, EU: 42, and UK: 14.",
    "227": "For ladies' rib slim fit tee in size extra small, Chest: 28, Body Length: 22.5, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
    "228": "For ladies' rib slim fit tee in size small, Chest: 30, Body Length: 23, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
    "229": "For ladies' rib slim fit tee in size medium, Chest: 30, Body Length: 23, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
    "230": "For ladies' rib slim fit tee in size large, Chest: 34, Body Length: 24, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
    "231": "For ladies' rib slim fit tee in size extra large, Chest: 36, Body Length: 24.5, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
    "232": "For ladies' slim fit bundle tees in size extra small, Chest: 30, Body Length: 23, Sleeve Length: 4.75, USA: 2, EU: 30, and UK: 6.",
    "233": "For ladies' slim fit bundle tees in size small, Chest: 32, Body Length: 23.5, Sleeve Length: 5, USA: 4, EU: 32, and UK: 8.",
    "234": "For ladies' slim fit bundle tees in size medium, Chest: 34, Body Length: 24, Sleeve Length: 5.25, USA: 6, EU: 36, and UK: 10.",
    "235": "For ladies' slim fit bundle tees in size large, Chest: 36, Body Length: 24.5, Sleeve Length: 5.5, USA: 8, EU: 40, and UK: 12.",
    "236": "For ladies' slim fit bundle tees in size extra large, Chest: 38, Body Length: 25, Sleeve Length: 5.75, USA: 10, EU: 42, and UK: 14.",
    "237": "For ladies' top tee oversized fit in size extra small, Chest: 38, Body Length: 24, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
    "238": "For ladies' top tee oversized fit in size small, Chest: 40, Body Length: 24.5 , Sleeve Length: 8, USA: 4, EU: 32, and UK: 8",
    "239": "For ladies' top tee oversized fit in size medium, Chest: 42, Body Length: 25 , Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.",
    "240": "For ladies' top tee oversized fit in size large, Chest: 44, Body Length: 25.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
    "241": "For ladies' top tee oversized fit in size extra large, Chest: 46, Body Length: 26, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
    "242": "For ladies' top shirt - short sleeves in size extra small, Chest: 40, Body Length: 20.5, Sleeve Length: 7.5, USA: 2, EU: 30, and UK: 6.",
    "243": "For ladies' top shirt - short sleeves in size small, Chest: 42, Body Length: 21, Sleeve Length: 7.75, USA: 4, EU: 32, and UK: 8.",
    "244": "For ladies' top shirt - short sleeves in size medium, Chest: 44, Body Length: 21.5, Sleeve Length: 8, USA: 6, EU: 36, and UK: 10.",
    "245": "For ladies' top shirt - short sleeves in size large, Chest: 46, Body Length: 22, Sleeve Length: 8.25, USA: 8, EU: 40, and UK: 12.",
    "246": "For ladies' top shirt - short sleeves in size extra large, Chest: 48, Body Length: 22.5, Sleeve Length: 8.5, USA: 10, EU: 42, and UK: 14.",
    "247": "For ladies' top shirt - long sleeves in size extra small, Chest: 40, Body Length: 27.5, Sleeve Length: 20.5, USA: 2, EU: 30, and UK: 6.",
    "248": "For ladies' top shirt - long sleeves in size small, Chest: 42, Body Length: 28, Sleeve Length: 20.75, USA: 4, EU: 32, and UK: 8.",
    "249": "For ladies' top shirt - long sleeves in size medium, Chest: 44, Body Length: 28.5, Sleeve Length: 21, USA: 6, EU: 36, and UK: 10.",
    "250": "For ladies' top shirt - long sleeves in size large, Chest: 46, Body Length: 29, Sleeve Length: 21.25, USA: 8, EU: 40, and UK: 12.",
    "251": "For ladies' top shirt - long sleeves in size extra large, Chest: 48, Body Length: 29.5, Sleeve Length: 21.5, USA: 10, EU: 42, and UK: 14.",
    "252": "For ladies' top - sports bra in size extra small, Chest: 28, Body Length: 9, Sleeve Length: N/A, USA: 2, EU: 30, and UK: 6.",
    "253": "For ladies' top - sports bra in size small, Chest: 30, Body Length: 9.5, Sleeve Length: N/A, USA: 4, EU: 32, and UK: 8.",
    "254": "For ladies' top - sports bra in size medium, Chest: 32, Body Length: 10, Sleeve Length: N/A, USA: 6, EU: 36, and UK: 10.",
    "255": "For ladies' top - sports bra in size large, Chest: 34, Body Length: 10.5, Sleeve Length: N/A, USA: 8, EU: 40, and UK: 12.",
    "256": "For ladies' top - sports bra in size extra large, Chest: 36, Body Length: 11, Sleeve Length: N/A, USA: 10, EU: 42, and UK: 14.",
    "257": "For ladies' top - hoodie in size extra small, Chest: 44, Body Length: 24.5, Sleeve Length: 17.5, USA: 2, EU: 30, and UK: 6.",
    "258": "For ladies' top - hoodie in size small, Chest: 46, Body Length: 25, Sleeve Length: 17.75, USA: 4, EU: 32, and UK: 8.",
    "259": "For ladies' top - hoodie in size medium, Chest: 48, Body Length: 25.5, Sleeve Length: 18, USA: 6, EU: 36, and UK: 10.",
    "260": "For ladies' top - hoodie in size large, Chest: 50, Body Length: 26, Sleeve Length: 18.25, USA: 8, EU: 40, and UK: 12.",
    "261": "For ladies' top - hoodie in size extra large, Chest: 52, Body Length: 26.5, Sleeve Length: 18.5, USA: 10, EU: 42, and UK: 14.",
    "262": "For ladies' top - crop pullover in size extra small, Chest: 36, Body Length: 17.5, Sleeve Length: 18.75, USA: 2, EU: 30, and UK: 6.",
    "263": "For ladies' top - crop pullover in size small, Chest: 38, Body Length: 18, Sleeve Length: 19, USA: 4, EU: 32, and UK: 8.",
    "264": "For ladies' top - crop pullover in size medium, Chest: 40, Body Length: 18.5, Sleeve Length: 19.25, USA: 6, EU: 36, and UK: 10.",
    "265": "For ladies' top - crop pullover in size large, Chest: 42, Body Length: 19, Sleeve Length: 19.5, USA: 8, EU: 40, and UK: 12.",
    "266": "For ladies' top - crop pullover in size extra large, Chest: 44, Body Length: 19.5, Sleeve Length: 19.75, USA: 10, EU: 42, and UK: 14.",
    "267": "For ladies' top - regular pullover in size extra small, Chest: 38, Body Length: 25.5, Sleeve Length: 20.75, USA: 2, EU: 30, and UK: 6.",
    "268": "For ladies' top - regular pullover in size small, Chest: 40, Body Length: 26, Sleeve Length: 21, USA: 4, EU: 32, and UK: 8.",
    "269": "For ladies' top - regular pullover in size medium, Chest: 42, Body Length: 26.5, Sleeve Length: 21.25, USA: 6, EU: 36, and UK: 10.",
    "270": "For ladies' top - regular pullover in size large, Chest: 44, Body Length: 27, Sleeve Length: 21.5, USA: 8, EU: 40, and UK: 12.",
    "271": "For ladies' top - regular pullover in size extra large, Chest: 46, Body Length: 27.5, Sleeve Length: 21.75, USA: 10, EU: 42, and UK: 14.",
    "272": "For ladies' dress - regular fit in size extra small, Chest: 33, Body Length: 32.5, Sleeve Length: 7, USA: 2, EU: 30, and UK: 6.",
    "273": "For ladies' dress - regular fit in size small, Chest: 35, Body Length: 33, Sleeve Length: 7.25, USA: 4, EU: 32, and UK: 8.",
    "274": "For ladies' dress - regular fit in size medium, Chest: 37, Body Length: 33.5, Sleeve Length: 7.5, USA: 6, EU: 36, and UK: 10.",
    "275": "For ladies' dress - regular fit in size large, Chest: 39, Body Length: 34, Sleeve Length: 7.75, USA: 8, EU: 40, and UK: 12.",
    "276": "For ladies' dress - regular fit in size extra large, Chest: 41, Body Length: 34.5, Sleeve Length: 8, USA: 10, EU: 42, and UK: 14.",
    "277": "For ladies' dress - oversized fit in size extra small, Chest: 38, Body Length: 33, Sleeve Length: 8.5, USA: 2, EU: 30, and UK: 6.",
    "278": "For ladies' dress - oversized fit in size small, Chest: 40, Body Length: 33.5, Sleeve Length: 8.75, USA: 4, EU: 32, and UK: 8.",
    "279": "For ladies' dress - oversized fit in size medium, Chest: 42, Body Length: 34, Sleeve Length: 9, USA: 6, EU: 36, and UK: 10.",
    "280": "For ladies' dress - oversized fit in size large, Chest: 44, Body Length: 34.5, Sleeve Length: 9.25, USA: 8, EU: 40, and UK: 12.",
    "281": "For ladies' dress - oversized fit in size extra large, Chest: 46, Body Length: 35, Sleeve Length: 9.5, USA: 10, EU: 42, and UK: 14.",
    "282": "For ladies' dress - long sleeved regular fit in size extra small, Chest: 38, Body Length: 32, Sleeve Length: 20, USA: 2, EU: 30, and UK: 6.",
    "283": "For ladies' dress - long sleeved regular fit in size small, Chest: 40, Body Length: 32.5, Sleeve Length: 20.25, USA: 4, EU: 32, and UK: 8.",
    "284": "For ladies' dress - long sleeved regular fit in size medium, Chest: 42, Body Length: 33, Sleeve Length: 20.5, USA: 6, EU: 36, and UK: 10.",
    "285": "For ladies' dress - long sleeved regular fit in size large, Chest: 44, Body Length: 33.5, Sleeve Length: 20.75, USA: 8, EU: 40, and UK: 12.",
    "286": "For ladies' dress - long sleeved regular fit in size extra large, Chest: 46, Body Length: 34, Sleeve Length: 21, USA: 10, EU: 42, and UK: 14.",
    "287": "For ladies' dress - baby doll dress in size extra small, Chest: 33, Body Length: 31, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
    "288": "For ladies' dress - baby doll dress in size small, Chest: 35, Body Length: 31.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
    "289": "For ladies' dress - baby doll dress in size medium, Chest: 37, Body Length: 32, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.",
    "290": "For ladies' dress - baby doll dress in size large, Chest: 39, Body Length: 32.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
    "291": "For ladies' dress - baby doll dress in size extra large, Chest: 41, Body Length: 33, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
    "292": "For ladies' bottoms - skinny jeans (numerical sizing) in size 26, Waist: 26, Hip: 32,  Inseam: 26, USA: 4, EU: 32, and UK: 8.",
    "293": "For ladies' bottoms - skinny jeans (numerical sizing) in size 27, Waist: 27, Hip: 33,  Inseam: 26, USA: 6, EU: 36, and UK: 10.",
    "294": "For ladies' bottoms - skinny jeans (numerical sizing) in size 28, Waist: 28, Hip: 34,  Inseam: 26, USA: 8, EU: 38, and UK: 12.",
    "295": "For ladies' bottoms - skinny jeans (numerical sizing) in size 29, Waist: 29, Hip: 35,  Inseam: 26, USA: 10, EU: 40, and UK: 14.",
    "296": "For ladies' bottoms - skinny jeans (numerical sizing) in size 30, Waist: 30, Hip: 36,  Inseam: 26, USA: 10, EU: 42, and UK: 16.",
    "297": "For ladies' bottoms - skinny jeans (numerical sizing) in size 32, Waist: 32, Hip: 38,  Inseam: 26, USA: 12, EU: 40, and UK: 14.",
    "298": "For ladies' bottoms - skinny jeans (alpha sizing) in size extra small, Waist: 24, Hip: 36,  Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "299": "For ladies' bottoms - skinny jeans (alpha sizing) in size small, Waist: 26, Hip: 38, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "300": "For ladies' bottoms - skinny jeans (alpha sizing) in size medium, Waist: 28, Hip: 40, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
    "301": "For ladies' bottoms - skinny jeans (alpha sizing) in size large, Waist: 30, Hip: 42, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
    "302": "For ladies' bottoms - skinny jeans (alpha sizing) in size extra large, Waist: 32, Hip: 44, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "303": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "304": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 8.",
    "305": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
    "306": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 29, Waist: 39, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
    "307": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
    "308": "For ladies' bottoms - basic straight jeans (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "309": "For ladies' bottoms - wide leg jeans (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "310": "For ladies' bottoms - wide leg jeans (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "311": "For ladies' bottoms - wide leg jeans (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
    "312": "For ladies' bottoms - wide leg jeans (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
    "313": "For ladies' bottoms - wide leg jeans (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "314": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 26, Waist: 25, Hip: 36.5, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "315": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 27, Waist: 26, Hip: 37.5, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "316": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 28, Waist: 27, Hip: 38.5, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
    "317": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 29, Waist: 28, Hip: 39.5, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
    "318": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 30, Waist: 29, Hip: 40.5, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
    "319": "For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 32, Waist: 31, Hip: 42.5, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "320": "For ladies' bottoms - shorts (alpha sizing) in size extra small, Waist: 26, Hip: 36, Inseam: 2.5, USA: 4, EU: 32, and UK: 8.",
    "321": "For ladies' bottoms - shorts (alpha sizing) in size small, Waist: 28, Hip: 38, Inseam: 2.5, USA: 6, EU: 36, and UK: 10.",
    "322": "For ladies' bottoms - shorts (alpha sizing) in size medium, Waist: 30, Hip: 40, Inseam: 2.5, USA: 8, EU: 38, and UK: 12.",
    "323": "For ladies' bottoms - shorts (alpha sizing) in size large, Waist: 32, Hip: 42, Inseam: 2.5, USA: 10, EU: 40, and UK: 14.",
    "324": "For ladies' bottoms - shorts (alpha sizing) in size extra large, Waist: 34, Hip: 44, Inseam: 2.5, USA: 12, EU: 42, and UK: 16.",
    "325": "For ladies' bottoms - shorts (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 2.5, USA: 4, EU: 32, and UK: 8.",
    "326": "For ladies' bottoms - shorts (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 2.5, USA: 6, EU: 36, and UK: 10.",
    "327": "For ladies' bottoms - shorts (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 2.5, USA: 6, EU: 38, and UK: 10.",
    "328": "For ladies' bottoms - shorts (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 2.5, USA: 10, EU: 38, and UK: 12.",
    "329": "For ladies' bottoms - shorts (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 2.5, USA: 10, EU: 40, and UK: 12.",
    "330": "For ladies' bottoms - shorts (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 2.5, USA: 12, EU: 42, and UK: 16.",
    "331": "For ladies' bottoms - biker shorts (alpha sizing) in size extra small, Waist: 25, Hip: 28, Inseam: 10, USA: 4, EU: 32, and UK: 8.",
    "332": "For ladies' bottoms - biker shorts (alpha sizing) in size small, Waist: 27, Hip: 30, Inseam: 10, USA: 6, EU: 36, and UK: 10.",
    "333": "For ladies' bottoms - biker shorts (alpha sizing) in size medium, Waist: 29, Hip: 32, Inseam: 10, USA: 8, EU: 38, and UK: 12.",
    "334": "For ladies' bottoms - biker shorts (alpha sizing) in size large, Waist: 31, Hip: 34, Inseam: 10, USA: 10, EU: 40, and UK: 14.",
    "335": "For ladies' bottoms - biker shorts (alpha sizing) in size extra large, Waist: 33, Hip: 36, Inseam: 10, USA: 12, EU: 42, and UK: 16.",
    "336": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size extra small, Waist: 23, Hip: 25, Inseam: 26, USA: 4, EU: 32, and UK: 8.",
    "337": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size small, Waist: 25, Hip: 27, Inseam: 26, USA: 6, EU: 36, and UK: 10.",
    "338": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size medium, Waist: 27, Hip: 29, Inseam: 26, USA: 8, EU: 38, and UK: 12.",
    "339": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size large, Waist: 29, Hip: 31, Inseam: 26, USA: 10, EU: 40, and UK: 14.",
    "340": "For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size extra large, Waist: 31, Hip: 33, Inseam: 26, USA: 12, EU: 42, and UK: 16.",
    "341": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size extra small, Waist: 25, Hip: 28, Inseam: 26.5, USA: 4, EU: 32, and UK: 8.",
    "342": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size small, Waist: 25, Hip: 28, Inseam: 26.5, USA: 4, EU: 32, and UK: 8.",
    "343": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size medium, Waist: 29, Hip: 32, Inseam: 26.5, USA: 8, EU: 38, and UK: 12.",
    "344": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size large, Waist: 31, Hip: 34, Inseam: 26.5, USA: 10, EU: 40, and UK: 14.",
    "345": "For ladies' bottoms - sports leggings without pockets (alpha sizing) in size extra large, Waist: 33, Hip: 36, Inseam: 26.5, USA: 12, EU: 42, and UK: 16.",
    "346": "For ladies' bottoms - track pants (mid-waist) in size extra small, Waist: 23, Hip: 35, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "347": "For ladies' bottoms - track pants (mid-waist) in size small, Waist: 25, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "348": "For ladies' bottoms - track pants (mid-waist) in size medium, Waist: 27, Hip: 39, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
    "349": "For ladies' bottoms - track pants (mid-waist) in size large, Waist: 29, Hip: 41, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
    "350": "For ladies' bottoms - track pants (mid-waist) in size extra large, Waist: 31, Hip: 43, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
    "351": "For ladies' bottoms - track pants (high-waist) in size extra small, Waist: 21, Hip: 35, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "352": "For ladies' bottoms - track pants (high-waist) in size small, Waist: 23, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "353": "For ladies' bottoms - track pants (high-waist) in size medium, Waist: 25, Hip: 39, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
    "354": "For ladies' bottoms - track pants (high-waist) in size large, Waist: 27, Hip: 41, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
    "355": "For ladies' bottoms - track pants (high-waist) in size extra large, Waist: 29, Hip: 43, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "356": "For ladies' bottoms - track pants with cuff in size extra small, Waist: 23, Hip: 35, Inseam: 26, USA: 4, EU: 32, and UK: 8.",
    "357": "For ladies' bottoms - track pants with cuff in size small, Waist: 25, Hip: 37, Inseam: 26, USA: 6, EU: 36, and UK: 10.",
    "358": "For ladies' bottoms - track pants with cuff in size medium, Waist: 27, Hip: 39, Inseam: 26, USA: 6, EU: 38, and UK: 10.",
    "359": "For ladies' bottoms - track pants with cuff in size large, Waist: 29, Hip: 41, Inseam: 26, USA: 10, EU: 38, and UK: 12.",
    "360": "For ladies' bottoms - track pants with cuff in size extra large, Waist: 31, Hip: 43, Inseam: 26, USA: 10, EU: 40, and UK: 12.",
    "361": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "362": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "363": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
    "364": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
    "365": "For ladies' bottoms - urban tapered trousers (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "366": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "367": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "368": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
    "369": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
    "370": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
    "371": "For ladies' bottoms - urban tapered trousers (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "372": "For ladies' bottoms - urban wide trousers (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "373": "For ladies' bottoms - urban wide trousers (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "374": "For ladies' bottoms - urban wide trousers (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.",
    "375": "For ladies' bottoms - urban wide trousers (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.",
    "376": "For ladies' bottoms - urban wide trousers (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "377": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.",
    "378": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.",
    "379": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.",
    "380": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.",
    "381": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.",
    "382": "For ladies' bottoms - urban wide trousers (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.",
    "383": "For men's loungewear robe in size small, Chest: 44, Body Length: 40, Sleeve Length: 19.5, USA: 12-16, EU: 46-50, and UK: 36-40.",
    "384": "For men's loungewear robe in size medium, Chest: 48, Body Length: 42, Sleeve Length: 19.5, USA: 18-20, EU: 52-54, and UK: 42-44.",
    "385": "For ladies' lounge set - top (oversized tee) in size extra small, Chest: 39, Body Length: 19, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.",
    "386": "For ladies' lounge set - top (oversized tee) in size small, Chest: 41, Body Length: 19.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.",
    "387": "For ladies' lounge set - top (oversized tee) in size medium, Chest: 43, Body Length: 20, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.",
    "388": "For ladies' lounge set - top (oversized tee) in size large, Chest: 45, Body Length: 20.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.",
    "389": "For ladies' lounge set - top (oversized tee) in size extra large, Chest: 47, Body Length: 21, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.",
    "390": "For ladies' lounge set - bottom (shorts) in size extra small, Chest: 26, Body Length: 18, Sleeve Length: 2, USA: 4, EU: 32, and UK: 8.",
    "391": "For ladies' lounge set - bottom (shorts) in size small, Chest: 28, Body Length: 19, Sleeve Length: 2, USA: 6, EU: 36, and UK: 10.",
    "392": "For ladies' lounge set - bottom (shorts) in size medium, Chest: 30, Body Length: 20, Sleeve Length: 2, USA: 8, EU: 38, and UK: 12.",
    "393": "For ladies' lounge set - bottom (shorts) in size large, Chest: 32, Body Length: 21, Sleeve Length: 2, USA: 10, EU: 40, and UK: 14.",
    "394": "For ladies' lounge set - bottom (shorts) in size extra large, Chest: 34, Body Length: 22, Sleeve Length: 2, USA: 12, EU: 42, and UK: 16.",
    "395": "For ladies' lounge set - top (spaghetti) in size extra small, Chest: 31, Body Length: 16, Sleeve Length: N/A, USA: 2, EU: 30, and UK: 6.",
    "396": "For ladies' lounge set - top (spaghetti) in size small, Chest: 33, Body Length: 16.5, Sleeve Length: N/A, USA: 4, EU: 32, and UK: 8.",
    "397": "For ladies' lounge set - top (spaghetti) in size medium, Chest: 35, Body Length: 17, Sleeve Length: N/A, USA: 6, EU: 36, and UK: 10.",
    "398": "For ladies' lounge set - top (spaghetti) in size large, Chest: 37, Body Length: 17.5, Sleeve Length: N/A, USA: 8, EU: 40, and UK: 12.",
    "399": "For ladies' lounge set - top (spaghetti) in size extra large, Chest: 39, Body Length: 18, Sleeve Length: N/A, USA: 10, EU: 42, and UK: 14.",
    "400": "For ladies' lounge set - bottom (shorts) in size extra small, Chest: 26, Body Length: 36, Sleeve Length: 2, USA: 4, EU: 32, and UK: 8.",
    "401": "For ladies' lounge set - bottom (shorts) in size small, Chest: 28, Body Length: 38, Sleeve Length: 2, USA: 6, EU: 36, and UK: 10.",
    "402": "For ladies' lounge set - bottom (shorts) in size medium, Chest: 30, Body Length: 40, Sleeve Length: 2, USA: 8, EU: 38, and UK: 12.",
    "403": "For ladies' lounge set - bottom (shorts) in size large, Chest: 32, Body Length: 42, Sleeve Length: 2, USA: 10, EU: 40, and UK: 14.",
    "404": "For ladies' lounge set - bottom (shorts) in size extra large, Chest: 34, Body Length: 44, Sleeve Length: 2, USA: 12, EU: 42, and UK: 16.",
    "405": "For ladies' footwear single band sliders, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24.2), (US: 7, EU: 37, CM: 24.9), (US: 8, EU: 38, CM: 25.6), (US: 9, EU: 39, CM: 26.3).",
    "406": "For ladies' footwear velcro sliders, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24.2), (US: 7, EU: 37, CM: 24.9), (US: 8, EU: 38, CM: 25.6), (US: 9, EU: 39, CM: 26.3).",
    "407": "For ladies' regular flipflops, (US: 5, EU: 35, CM: 23.2), (US: 6, EU: 36, CM: 23.9), (US: 7, EU: 37, CM: 24.6), (US: 8, EU: 38, CM: 25.3), (US: 9, EU: 39, CM: 26), then the slippers thicknes is 15mm.",
    "408": "For ladies' lace-up sneakers, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24), (US: 7, EU: 37, CM: 24.5), (US: 8, EU: 38, CM: 25), (US: 9, EU: 39, CM: 25.5).",
    "409": "For ladies' runner shoes, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24), (US: 7, EU: 37, CM: 24.5), (US: 8, EU: 38, CM: 25), (US: 9, EU: 39, CM: 25.5).",
    "410": "For men's coed tee in size double extra small, Chest: 36, Body Length: 27.5, Sleeve Length: 7, USA: 10, EU: 44, and UK: 34.",
    "411": "For men's coed tee in size extra small, Chest: 38, Body Length: 27.5, Sleeve Length: 7.25, USA: 12, EU: 46, and UK: 36.",
    "412": "For men's coed tee in size small, Chest: 40, Body Length: 28, Sleeve Length: 7.5, USA: 14, EU: 48, and UK: 38.",
    "413": "For men's coed tee in size medium, Chest: 42, Body Length: 28.5, Sleeve Length: 7.75, USA: 16, EU: 50, and UK: 40.",
    "414": "For men's coed tee in size large, Chest: 44, Body Length: 29, Sleeve Length: 8, USA: 18, EU: 52, and UK: 42.",
    "415": "For men's coed tee in size extra large, Chest: 46, Body Length: 30, Sleeve Length: 8.25, USA: 20, EU: 54, and UK: 44.",
    "416": "For men's coed tee in size double extra large, Chest: 48, Body Length: 30.5, Sleeve Length: 8.5, USA: 22, EU: 56, and UK: 46.",
    "417": "For men's coed tee in size triple extra large, Chest: 50, Body Length: 31, Sleeve Length: 8.75, USA: 24, EU: 58, and UK: 48.",
    "418": "For ladies' coed tee in size double extra small, Chest: 36, Body Length: 27.5, Sleeve Length: 7, USA: 10, EU: 44, and UK: 34.",
    "419": "For ladies' coed tee in size medium, Chest: 38, Body Length: 27.5, Sleeve Length: 7.25, USA: 12, EU: 46, and UK: 36.",
    "420": "For ladies' coed tee in size large, Chest: 40, Body Length: 28, Sleeve Length: 7.5, USA: 14, EU: 48, and UK: 38.",
    "421": "For ladies' coed tee in size extra large, Chest: 42, Body Length: 28.5, Sleeve Length: 7.75, USA: 16, EU: 50, and UK: 40.",
    "422": "For men's coed pullover in size double extra small, Chest: 38, Body Length: 24, Sleeve Length: 21.25, USA: 10, EU: 44, and UK: 34.",
    "423": "For men's coed pullover in size extra small, Chest: 40, Body Length: 25, Sleeve Length: 21.5, USA: 12, EU: 46, and UK: 36.",
    "424": "For men's coed pullover in size small, Chest: 42, Body Length: 26, Sleeve Length: 21.75, USA: 14, EU: 48, and UK: 38.",
    "425": "For men's coed pullover in size medium, Chest: 44, Body Length: 27, Sleeve Length: 22, USA: 16, EU: 50, and UK: 42.",
    "426": "For men's coed pullover in size large, Chest: 46, Body Length: 28, Sleeve Length: 22.25, USA: 18, EU: 52, and UK: 42.",
    "427": "For men's coed pullover in size extra large, Chest: 48, Body Length: 29, Sleeve Length: 22.5, USA: 20, EU: 54, and UK: 44.",
    "428": "For men's coed pullover in size double extra large, Chest: 50, Body Length: 30, Sleeve Length: 22.75, USA: 22, EU: 56, and UK: 46.",
    "429": "For men's coed pullover in size triple extra large, Chest: 52, Body Length: 31, Sleeve Length: 23, USA: 24, EU: 58, and UK: 48.",
    "430": "For ladies' coed pullover in size small, Chest: 38, Body Length: 24, Sleeve Length: 21.25, USA: 10, EU: 44, and UK: 34.",
    "431": "For ladies' coed pullover in size medium, Chest: 40, Body Length: 25, Sleeve Length: 21.5, USA: 12, EU: 46, and UK: 36.",
    "432": "For ladies' coed pullover in size large, Chest: 42, Body Length: 26, Sleeve Length: 21.75, USA: 14, EU: 48, and UK: 38.",
    "433": "For ladies' coed pullover in size extra large, Chest: 44, Body Length: 27, Sleeve Length: 22, USA: 16, EU: 50, and UK: 40.",
    "434": "For men's coed woven trousers (urban straight trouser) in size double extra small, Waist: 22, Hip: 34, Inseam: 23, USA: 34-36, EU: 35-36, and UK: 27-28.",
    "435": "For men's coed woven trousers (urban straight trouser) in size extra small, Waist: 24, Hip: 36, Inseam: 23, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "436": "For men's coed woven trousers (urban straight trouser) in size small, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "437": "For men's coed woven trousers (urban straight trouser) in size medium, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "438": "For men's coed woven trousers (urban straight trouser) in size large, Waist: 30, Hip: 42, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.",
    "439": "For men's coed woven trousers (urban straight trouser) in size extra large, Waist: 32, Hip: 44, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.",
    "440": "For ladies' coed woven trousers (urban straight trouser) in size small, Waist: 22, Hip: 34, Inseam: 23, USA: 34-36, EU: 35-36, and UK: 27-28.",
    "441": "For ladies' coed woven trousers (urban straight trouser) in size medium, Waist: 24, Hip: 36, Inseam: 23, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "442": "For ladies' coed woven trousers (urban straight trouser) in size large, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "443": "For ladies' coed woven trousers (urban straight trouser) in size extra large, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "444": "For men's coed shorts (woven) in size double extra small, Waist: 27, Hip: 37, Inseam: 5, USA: 34-36, EU: 35-36, and UK: 27-28.",
    "445": "For men's coed shorts (woven) in size extra small, Waist: 29, Hip: 39, Inseam: 5, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "446": "For men's coed shorts (woven) in size small, Waist: 31, Hip: 41, Inseam: 5, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "447": "For men's coed shorts (woven) in size medium, Waist: 33, Hip: 43, Inseam: 5, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "448": "For men's coed shorts (woven) in size large, Waist: 35, Hip: 45, Inseam: 5, USA: 43-44, EU: 43-44, and UK: 33-34.",
    "449": "For men's coed shorts (woven) in size extra large, Waist: 37, Hip: 47, Inseam: 5, USA: 45-46, EU: 45-46, and UK: 35-36.",
    "450": "For ladies' coed shorts (woven) in size small, Waist: 27, Hip: 37, Inseam: 5, USA: 34-36, EU: 35-36, and UK: 27-28.",
    "451": "For ladies' coed shorts (woven) in size medium, Waist: 29, Hip: 39, Inseam: 5, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "452": "For ladies' coed shorts (woven) in size large, Waist: 31, Hip: 41, Inseam: 5, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "453": "For ladies' coed shorts (woven) in size extra large, Waist: 33, Hip: 43, Inseam: 5, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "454": "For men's coed woven trousers (tapered) in size double extra small, Waist: 22, Hip: 34, Inseam: 24, USA: 34-36, EU: 35-36, and UK: 27-28.",
    "455": "For men's coed woven trousers (tapered) in size extra small, Waist: 24, Hip: 36, Inseam: 24, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "456": "For men's coed woven trousers (tapered) in size small, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "457": "For men's coed woven trousers (tapered) in size medium, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "458": "For men's coed woven trousers (tapered) in size large, Waist: 30, Hip: 42, Inseam: 24, USA: 43-44, EU: 43-44, and UK: 33-34.",
    "459": "For men's coed woven trousers (tapered) in size extra large, Waist: 32, Hip: 44, Inseam: 24, USA: 45-46, EU: 45-46, and UK: 35-36.",
    "460": "For ladies' coed woven trousers (tapered) in size small, Waist: 22, Hip: 34, Inseam: 24, USA: 34-36, EU: 35-36, and UK: 27-28.",
    "461": "For ladies' coed woven trousers (tapered) in size medium, Waist: 24, Hip: 36, Inseam: 24, USA: 36-38, EU: 36-38, and UK: 29-30.",
    "462": "For ladies' coed woven trousers (tapered) in size large, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.",
    "463": "For ladies' coed woven trousers (tapered) in size extra large, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.",
    "464": "For accessories - regular backpack, Length: 17 inches, Width: 11 inches, Depth: 5 inches, Strap Length: 16.5 inches, Strap Width: 2.75 inches.",
    "465": "For accessories - large backpack, Length: 18 inches, Width: 12 inches, Depth: 6 inches, Strap Length: 16.5 inches, Strap Width: 2.75 inches.",
    "466": "For accessories - drawstring backpack, Length: 17 inches and Width: 13 inches.",
    "467": "For accessories - regular bum bag, Length: 4.5 inches, Width: 13.5 inches, Depth: 3 inches, Strap Length: 43 inches, Strap Width: 1.5 inches.",
    "468": "For accessories - multi pockets bum bag, Length: 7.5 inches, Width: 13 inches, Depth: 3.5 inches.",
    "469": "For accessories - oversized bum bag, Length: 9 inches, Width: 18 inches, Depth: 4 inches.",
    "470": "For accessories - duffle bag, Length: 9.75 inches, Width: 18 inches, Depth: 9.75 inches.",
    "471": "For accessories - mini sling bag, Length: 7 inches, Width: 5 inches, Depth: 1.5 inches, Strap Length: 53 inches, Strap Width: 1 inches.",
    "472": "For accessories - regular sling bag, Length: 9 inches, Width: 7 inches, Depth: 2 inches, Strap Length: 53 inches, Strap Width: 1 inches.",
    "473": "For accessories - mid sized messenger bag, Length: 10 inches, Width: 8 inches, Depth: 2.5 inches, Strap Length: 5.3 inches, Strap Width: 1.5 inches.",
    "474": "For accessories - type 1 tote bag, Length: 16 inches, Width: 10 inches, Depth: 4 inches, Strap Length: 26 inches, Strap Width: 1.5 inches.",
    "475": "For accessories - type 2 tote bag, Length: 16 inches, Width: 13.5 inches, Depth: N/A, Strap Length: 26 inches, Strap Width: 1 inches.",
    "476": "For accessories - beanie, Circumference: 57 cm and Height: 8.89 cm.",
    "477": "For accessories - bucket hat, Circumference: 59 cm, Height: 8.89 cm, Visor Length: 6.35 cm.",
    "478": "For accessories - curved cap, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.99 cm.",
    "479": "For accessories - dad hat, Circumference: 57 cm, Height: 17.78 cm, Visor Length: 6.99 cm.",
    "480": "For accessories - snap back, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.35 cm.", 
    "481": "For accessories - trucker cap, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.99 cm.",
    "482": "For accessories - type 1 coin purse, Length: 4 inches, Width: 3 inches, Depth: 0.75 inch, Zipper Length: 3 3/4 inches.",
    "483": "For accessories - type 2 coin purse, Length: 3 inches, Width: 4 inches, Depth: 0.75 inch, Zipper Length: 5 1/2 inches.",
    "484": "For accessories - bifold wallet, Folded: (Length: 3.5 inches, Width: 3 1/2 inches, Depth: 0.5 inch), Bill Compartment: (Length: 4.5 inches and Height: 3 1/4 inches).",
    "485": "For accessories - midsized bifold wallet, Folded: (Length: 3 1/4 inches, Width: 4 1/2 inches), Bill Compartment: (Length: 6 1/2 inches and Height: 3 3/4 inches).",
    "486": "For accessories - trifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).",
    "487": "For accessories - zip around bifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches, Depth: 0.5 inch), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).",
    "488": "Hello Good Day Ma'am/Sir! how may I help you?",
    "489": "Yes, we offer the option to book private events or fashion shows in our store. Contact our events team for details and availability.",
    "490": "Yes, we offer international shipping. You can place your order online, and we'll ship it to your desired destination.",
    "491": "Sale items are eligible for returns and exchanges within 14 days of purchase. Please ensure the item is in its original condition.",
    "492": "You can stay updated on our upcoming sales and events by subscribing to our newsletter or following our social media channels.",
    "493": "Yes, we offer personal shopping and styling services. Our team can help you find the perfect outfit tailored to your style.",
    "494": "You can apply for our store credit card, which offers benefits like exclusive discounts, rewards, and special financing options.",
    "495": "We welcome your feedback! You can provide feedback through our website, in-store, or by contacting our customer support.",
    "496": "Yes, we can place a special order for items that are out of stock in your size. Please speak with a store associate for assistance.",
    "497": "We offer gift wrapping services for a small fee. Your items will be beautifully packaged for a special touch.",
    "498": "We release new collections regularly, typically with the changing seasons. Stay tuned for our latest arrivals.",
    "499": "Yes, we offer the option to book private events or fashion shows in our store. Contact our events team for details and availability.",
    "500":  "Yes, we offer express and same-day delivery options for online orders in select locations.",
    "501": "Absolutely! We have a loyalty program where you can earn rewards and exclusive discounts.",
    "502": "We offer price adjustments for items purchased within seven days before a sale. Bring your receipt for verification.",
    "503": "Yes, you can request clothing alterations at the time of purchase. Our team can assist you with adjustments.",
    "504": "Yes, we offer special discounts for students and senior citizens. Please bring a valid ID for verification.",
    "505": "We encourage clothing donations and recycling. You can inquire in-store about our donation programs.",
    "506": "Certainly! We can assist with bulk orders for special events. Please contact our sales team for more details.",
    "507": "Yes, we have a mobile app that makes online shopping convenient and offers exclusive deals.",
    "508": "You can track your online order and check its status by logging into your account on our website or app.",
    "509": "Yes, we offer gift-wrapping services for online orders, making it a perfect gift option.",
    "510": "We offer store credit for returns within 30 days, or refunds in the original payment method.",
    "511": "Yes, email subscribers receive exclusive discounts and early access to promotions.",
    "512": "You can check the availability of items in a specific store location by contacting the store directly.",
    "513": "We have robust security measures to protect customer data and adhere to strict privacy policies.",
    "514": "Yes, we offer a gift registry service for weddings and special occasions. Visit our website for details.",
    "515": "Yes, we have a referral program where you and your friends can enjoy discounts.",
    "516": "You can apply for job openings by visiting our careers page on our website and submitting your application.",
    "517": "Yes, we offer corporate gifting solutions, including branded and customized gift options.",
    "518": "Yes, we can arrange to transfer items from other store locations for your pickup.",
    "519": "You can schedule a fashion consultation or appointment by contacting our store and booking a time slot.",
    "520": "Yes, we accept mobile payment options like Apple Pay and Google Pay for your convenience.",
    "521": "Yes, you can exchange a gift for a different size or style within our return policy guidelines.",
    "522": "Yes, we have a store newsletter that provides updates, promotions, and exclusive offers to subscribers.",
    "523": "We have implemented measures to ensure a safe and comfortable shopping experience, including enhanced cleaning and safety protocols.",
    "524": "For custom-made clothing, we require measurements and specific details to tailor the item to your preferences.",
    "525": "Yes, you can use multiple payment methods for a single purchase. Our staff can assist with split payments.",
    "526": "We will gladly replace or refund defective or damaged items. Please bring them to our attention within our return policy timeframe.",
    "527": "We value your feedback! You can provide feedback in our stores or through our website.",
    "528": "Yes, you can place orders for gifts and have them shipped to multiple addresses during the checkout process.",
    "529": "You can sign up for a store credit card in-store or online, and eligibility requirements will be reviewed during the application process.",
    "530": "Good morning! How can I assist you today?",
    "531": "Hello! How can I help you?",
    "532": "Good afternoon! What can I do for you?",
    "533": "Good day! How may I assist you?",
    "534": "Good evening! How can I be of service?",
    "535": "Good night! How can I assist you at this hour?",
    "536": "Good dawn! What brings you here so early?",
    "537": "Greetings! How can I help you as the day winds down?",
    "538": "Good morning and sunrise to you! How can I assist you?",
    "539": "Good evening and sunset to you! What can I do for you?",
    "540": "Hello! How may I assist you during this beautiful twilight?",
    "541": "A tie bar, when worn properly, holds the tie in place and adds a stylish and functional element to a business outfit.",
    "542": "When selecting dress shoes, consider classic styles like oxfords or derbies, and ensure they are well-polished and match the color of your belt.",
    "543": "To incorporate seasonal trends, add accessories or select clothing items in seasonal colors while maintaining a professional core look.",
    "544": "For a black-tie event, men should wear a black tuxedo, a black bow tie, and polished black dress shoes.",
    "545": "Women have various options for footwear, including pumps, loafers, ankle boots, and closed-toe heels to match their business outfits.",
    "546": "Statement jewelry can be worn in business attire but should be used sparingly to avoid appearing too flashy or distracting.",
    "547": "A tailored dress shirt ensures a perfect fit and a sharp, professional appearance, as it's designed to complement your body shape.",
    "548": "Transition from day to night by adding a blazer, statement accessories, and a change of shoes to elevate your look for evening events.",
    "549": "Turtlenecks can be appropriate in business attire, especially in colder months, when worn with a well-fitted blazer or suit.",
    "550": "Incorporate colors like navy or deep blue to convey confidence, or choose power colors like red or burgundy for a bolder statement.",
    "551": "Good midnight! How can I assist you at this late hour?",
    "552": "Good noon! How can I help you at this point in the day?",
    "553": "Good morning! I'm here to help. What can I do for you?",
    "554": "Hello! I'm here to assist you with anything you need.",
    "555": "Good afternoon! Feel free to ask any questions or seek assistance.",
    "556": "Hello! I'm here to assist you with anything you need. How can I help?",
    "557": "Good evening! I'm here to help. How can I be of service?",
    "558": "Good night! I'm here to assist you at this hour. What can I do for you?",
    "559": "Good dawn! I'm here to assist you. How can I help at this early hour?",
    "560": "Greetings! I'm here to help as the day winds down. What can I do for you?",
    "561": "Good morning and sunrise to you! I'm here to help. How can I assist you today?",
    "562": "Good evening and sunset to you! I'm here to assist you. What can I do for you?",
    "563": "Hello! I'm here to help during this lovely twilight. How may I assist you?",
    "564": "Good midnight! I'm here to assist you at this late hour. What can I do for you?",
    "565": "Good noon! I'm here to assist you. What can I do for you at this time?",
    "566": "Business casual attire typically includes slacks, a button-up shirt, and closed-toe shoes for men, and a blouse, slacks or skirt, and closed-toe shoes for women.",
    "567": "Business formal attire is more formal than business casual and often includes a suit and tie for men, and a tailored suit or dress for women.",
    "568": "Essential accessories may include a classic watch, a leather belt, and tasteful jewelry.",
    "569": "To prolong the life of your suits, dry clean them only when necessary, hang them properly, and store them in a garment bag.",
    "570": "Neutral colors like black, gray, navy, and white are often considered appropriate for business attire.",
    "571": "A power tie, typically a bold and confident color, is worn to convey authority and confidence in business settings.",
    "572": "Dress codes in the workplace help maintain a professional image and set a standard for appropriate attire.",
    "573": "Choose lightweight, breathable fabrics like linen or cotton and opt for short-sleeved shirts or dresses.",
    "574": "In many business settings, open-toed shoes are not considered appropriate; closed-toe shoes are a safer choice.",
    "575": "Dress in professional attire that's slightly more formal than the company's dress code to make a positive impression.",
    "576": "Denim is generally not considered business casual, but some workplaces allow it on specific days or with certain conditions.",
    "577": "Wear solid, non-distracting colors, and make sure your top half looks professional as it's what's visible on camera.",
    "578": "A blazer is a versatile piece that can elevate a business casual look and add a touch of formality.",
    "579": "Sneakers are typically not considered appropriate in business settings; opt for more formal footwear.",
    "580": "Personal grooming is crucial for making a positive impression and projecting professionalism.",
    "581": "Look for sales, outlet stores, and second-hand shops to find high-quality attire at affordable prices.",
    "582": "A well-fitted suit can enhance your confidence and convey attention to detail, which is highly regarded in the business world.",
    "583": "Subtle patterns like stripes or checks are acceptable in business attire, but avoid loud or distracting prints.",
    "584": "Women can accessorize with a stylish handbag, tasteful jewelry, and closed-toe heels or flats.",
    "585": "Dress formally, often in a tailored suit for men and an elegant dress for women, to show respect for the occasion.",
    "586": "Turtlenecks can be appropriate in a business casual setting, especially in colder months, as long as they are clean and well-maintained.",
    "587": "A classic Windsor knot or a four-in-hand knot are suitable for a professional look in the business world.",
    "588": "Incorporate your company's branding colors subtly in your attire, such as with a tie or scarf, to show alignment with the brand.",
    "589": "Women can wear pantsuits in the business world, as they offer a professional and stylish alternative to dresses or skirts.",
    "590": "Cufflinks add a touch of elegance and personal style to dress shirts with French cuffs, making them a symbol of sophistication.",
    "591": "For a business-casual event, consider wearing chinos, a button-down shirt, and loafers for a polished yet relaxed appearance.",
    "592": "All-black attire can be appropriate in a business setting, but it's important to add some variety with accessories or subtle patterns.",
    "593": "Dress in professional attire that's slightly more formal than the company's dress code to make a positive impression.",
    "594": "Look for sales, outlet stores, and second-hand shops to find high-quality attire at affordable prices.",
    "595": "Business casual attire typically includes slacks, a button-up shirt, and closed-toe shoes for men, and a blouse, slacks or skirt, and closed-toe shoes for women.",
    "596": "For a business dress, opt for fabrics like wool, wool blends, or high-quality synthetic materials for a professional look.",
    "597": "A business skirt should typically fall just above or below the knee, ensuring a professional and modest appearance.",
    "598": "Vests or waistcoats can add a touch of sophistication to business attire and are still considered stylish.",
    "599": "A pocket square adds a touch of sophistication and personal style to a suit jacket, enhancing its overall appearance.",
    "600": "Different collar styles suit different face shapes; for example, spread collars can complement round faces, while point collars work well with oval faces.",
    "601": "When selecting a clothing store, consider factors such as location, product range, price range, and customer reviews to find the best fit for your shopping preferences.",
    "602": "To find the best deals, sign up for newsletters, follow the store's social media accounts, and keep an eye on sales and clearance sections.",
    "603": "Popular clothing store chains known for quality and selection include OXGN, Zara, H&M, Nordstrom, and Macy's.",
    "604": "A well-organized store layout makes shopping easier, helping customers find what they need quickly and enhancing their overall shopping experience.",
    "605": "Use the store's sizing charts, and don't hesitate to ask for assistance from store staff to ensure you find the right size.",
    "606": "To make the changing room experience smoother, limit the number of items you bring in, and ensure you check the store's return policy.",
    "607": "Stay updated on fashion trends by following fashion magazines, blogs, and influencers, and then shop for trendy items at your favorite clothing stores.",
    "608": "Yes, there are eco-friendly clothing stores that offer sustainable and environmentally-conscious fashion choices, promoting ethical and responsible shopping.",
    "609": "A personal shopper can assist customers in finding clothing that suits their style and needs, providing a tailored shopping experience.",
    "610": "To make the most of sales and promotions, plan your shopping trips during sale events, and prioritize items you need.",
    "611": "Shopping at thrift stores can be budget-friendly, so look for well-maintained and timeless pieces to score high-quality clothing at affordable prices.",
    "612": "Yes, many clothing stores provide personalized tailoring services to ensure your clothing fits perfectly and is tailored to your preferences.",
    "613": "Customer reviews and ratings provide valuable insights into the store's quality, service, and customer satisfaction, helping you make informed decisions.",
    "614": "To maintain the quality of your clothing, follow care instructions, store garments properly, and handle stains promptly.",
    "615": "Yes, some clothing stores specialize in specific fashion niches or styles, catering to unique tastes, such as vintage, bohemian, or streetwear.",
    "616": "Loyalty programs can offer rewards, discounts, and exclusive deals for repeat customers, enhancing the shopping experience and building brand loyalty.",
    "617": "To identify counterfeit items, research authentic designer features and logos, and be cautious when the price seems too good to be true.",
    "618": "A fashion stylist can provide style advice, help you put together outfits, and suggest clothing items that complement your personal style.",
    "619": "To make the most of the fitting room experience, take your time, try on different sizes, and assess the comfort and fit of each item.",
    "620": "Several clothing stores, such as Patagonia, EILEEN FISHER, and Reformation, are known for their commitment to sustainable and ethical fashion practices.",
    "621": "Budget your spending by setting a limit, making a shopping list, and considering cost per wear when evaluating potential purchases.",
    "622": "Yes, many clothing stores collaborate with designers or brands to offer exclusive, limited-edition releases that attract fashion enthusiasts.",
    "623": "When shopping online, consider checking size charts, reading product descriptions, and reviewing return policies to make informed decisions.",
    "624": "Make sustainable choices by opting for eco-friendly materials, supporting ethical brands, and recycling or upcycling clothing items.",
    "625": "When searching for a special occasion dress, consider the dress code, venue, and your personal style to find the perfect outfit.",
    "626": "Avoid impulse buying by creating a shopping list, setting a budget, and taking your time to carefully consider each purchase.",
    "627": "Many clothing stores now offer inclusive sizing options including us in OXGN store, catering to a wide range of body types and promoting body positivity.",
    "628": "Seasonal sales and clearance events provide an opportunity to purchase clothing items at reduced prices, making it easier to update your wardrobe.",
    "629": "To find clothing stores catering to specific age groups or demographics, read customer reviews, consult fashion magazines, and explore social media for recommendations.",
    "630": "Clothing stores like Aerie and Target are known for their commitment to diversity and inclusivity, both in marketing and product offerings.",
    "631": "Trying on clothing items allows you to assess the fit, comfort, and overall look of the garment, helping you make confident purchasing decisions.",
    "632": "After a shopping spree, organize and declutter your closet by categorizing items, donating or selling clothing you no longer need, and maximizing storage space.",
    "633": "When searching for the perfect pair of jeans, consider your body shape, preferred fit, and wash options, and consult store staff for assistance.",
    "634": "Discover local boutiques by exploring your neighborhood, seeking recommendations from friends or online platforms, and attending local fashion events.",
    "635": "Online customer reviews offer insights into a store's product quality, service, and customer experiences, aiding in your decision-making process.",
    "636": "Brands and stores like Everlane, People Tree, and Outerknown prioritize fair labor practices and ethical sourcing in their fashion production.",
    "637": "Look for clothing stores that offer customization or made-to-measure services to ensure your clothing fits perfectly and suits your unique style.",
    "638": "Clothing store loyalty programs reward frequent shoppers with exclusive discounts, early access to sales, and other perks, enhancing their shopping experience.",
    "639": "To find vintage or retro clothing stores, explore local thrift shops, consignment stores, or online marketplaces specializing in vintage fashion.",
    "640": "Make environmentally conscious choices by opting for eco-friendly materials, supporting sustainable brands, and choosing clothing made with ethical practices.",
    "641": "Inspect clothing items for sturdy stitching, quality fabrics, no defects, and attention to detail, which indicate good quality and durability.",
    "642": "Department stores like Bloomingdale's and specialty stores like Men's Wearhouse offer a wide range of formal wear options for special events.",
    "643": "To align your clothing purchases with your style and needs, plan your wardrobe, create a color palette, and focus on versatile pieces that can be mixed and matched.",
    "644": "Online tools and apps like fashion advice websites, virtual fitting rooms, and wardrobe management apps can enhance your clothing shopping experience.",
    "645": "Brands like Aerie, Dove, and Fenty Beauty are recognized for promoting body positivity and diverse representation in their advertising campaigns.",
    "646": "A fashion consultant can offer personalized style advice, recommend clothing items, and help customers discover fashion choices that suit their preferences.",
    "647": "Look for timeless and versatile pieces like classic blazers, white shirts, and well-fitted jeans that can be worn in various outfits and styles.",
    "648": "Calculate the cost per wear by dividing the item's price by the number of times you anticipate wearing it, helping you make cost-effective decisions.",
    "649": "A clothing store's return and exchange policy ensures flexibility in case a purchase doesn't meet your expectations or fit as intended.",
    "650": "Look for clothing stores that actively support charitable causes or donate a portion of their proceeds to social initiatives, making a positive impact through fashion.",
    "651": "Customers often engage with clothing brands through social media, customer reviews, and feedback forms to share their opinions and preferences.",
    "652": "Customer feedback is used to identify areas for improvement, refine product designs, and create offerings that align better with customer preferences.",
    "653": "Clothing stores employ strategies like consistent pricing, inventory availability across channels, and responsive customer service to create a seamless shopping experience.",
    "654": "Businesses protect customer data by implementing robust cybersecurity measures, encryption, and complying with data protection regulations to ensure security and privacy.",
    "655": "Social media platforms provide a channel for clothing brands to engage with customers, gather feedback, and address concerns in real-time.",
    "656": "Clothing retailers can encourage feedback through surveys, incentives, and user-friendly feedback platforms, making it convenient for customers to share their insights.",
    "657": "Building trust involves delivering on promises, transparency, and addressing issues promptly, ultimately fostering strong and credible customer relationships.",
    "658": "Personalization in marketing campaigns tailors content to individual customer preferences, increasing engagement and conversion rates for clothing brands.",
    "659": "Effective complaint management involves prompt acknowledgment, thorough investigation, resolution, and feedback, ensuring customer satisfaction.",
    "660": "Strategies include loyalty programs, personalized discounts, and exceptional customer service to retain loyal customers and reduce churn.",
    "661": "Data analytics help clothing retailers analyze customer behavior and preferences, enabling data-driven decision-making and personalized experiences.",
    "662": "Businesses benefit from CRM solutions with improved customer insights, streamlined processes, and the ability to nurture leads and retain customers effectively.",
    "663": "Personalization in email marketing campaigns increases engagement and conversion rates by delivering relevant content tailored to individual preferences.",
    "664": "Clothing brands should regularly update CRM strategies by monitoring customer trends, preferences, and feedback to remain responsive to changing needs.",
    "665": "Omni-channel customer service ensures consistency and convenience across all customer touchpoints, whether in-store, online, or via mobile.",
    "666": "A successful CRM strategy includes customer data management, personalization, analytics, and a customer-centric culture within the organization.",
    "667": "Businesses safeguard data security and privacy through robust cybersecurity measures, data encryption, and compliance with privacy regulations.",
    "668": "CRM allows clothing retailers to segment their customer base based on demographics, purchase history, and behavior, making it easier to tailor marketing efforts.",
    "669": "Employee training and engagement are critical for providing excellent customer service, as motivated and well-informed staff are more likely to satisfy customer needs.",
    "670": "Encouraging feedback involves creating user-friendly feedback platforms, conducting surveys, and offering incentives to make it easy for customers to share their insights.",
    "671": "Omni-channel customer service ensures that customers experience consistent and convenient interactions across all channels, fostering strong and loyal relationships.",
    "672": "Best practices include prompt acknowledgment, thorough investigation, resolution, and seeking customer feedback to ensure complaints are resolved effectively.",
    "673": "CRM systems enable businesses to identify and nurture leads by tracking interactions and automating lead scoring to prioritize potential customers.",
    "674": "Strategies for retaining loyal customers include offering rewards, personalized discounts, and exceptional customer service to reduce churn.",
    "675": "Data analytics in CRM helps businesses analyze customer behavior patterns, enabling them to make data-driven decisions and personalize interactions.",
    "676": "Clothing retailers measure effectiveness through metrics such as customer satisfaction, retention rates, lead conversion, and return on investment (ROI).",
    "677": "Automation simplifies routine tasks, enhances efficiency, and ensures timely follow-ups, improving the overall customer experience.",
    "678": "Personalization involves segmenting customers, tailoring content, and delivering personalized offers, promotions, and recommendations.",
    "679": "Customer feedback provides valuable insights into customer preferences and areas for improvement, helping shape CRM strategies.",
    "680": "Strategies include offering loyalty programs, conducting surveys, and tracking online behavior to collect and update customer data.",
    "681": "Clothing brands protect customer data through data encryption, strict access controls, and compliance with privacy regulations to ensure responsible usage.",
    "682": "Key components include customer data management, personalization, analytics, and a customer-centric culture within the organization.",
    "683": "Social media platforms allow clothing brands to engage with customers, gather feedback, and address concerns in real-time, fostering strong customer relationships.",
    "684": "Loyalty programs and incentives reward customers for repeat purchases, encourage brand loyalty, and boost customer retention.",
    "685": "CRM systems help businesses track sales leads throughout the sales funnel, monitor progress, and measure lead conversion effectively.",
    "686": "Clothing retailers adapt by monitoring trends, offering new styles, and communicating with customers to understand their preferences.",
    "687": "Customer service representatives play a key role in resolving issues, addressing inquiries, and ensuring customers have a positive experience with the brand.",
    "688": "Strategies include follow-up emails, post-purchase surveys, and personalized recommendations to maintain engagement after a sale.",
    "689": "Ensuring consistency involves synchronizing inventory, pricing, and providing uniform customer service across all channels.",
    "690": "Factors include product quality, personalized experiences, rewards programs, and excellent customer service that inspire loyalty and advocacy.",
    "691": "CRM systems track and analyze customer interactions, helping brands identify and address emerging concerns promptly.",
    "692": "Community engagement events provide opportunities for customers to connect with the brand, fostering a sense of belonging and stronger relationships.",
    "693": "Effective returns and exchanges involve clear policies, hassle-free processes, and a focus on customer satisfaction to maintain positive relationships.",
    "694": "Personalized loyalty programs offer tailored rewards, discounts, and incentives that enhance customer retention and engagement.",
    "695": "Clothing brands invest in employee training programs to ensure staff can provide exceptional service and build strong customer relationships.",
    "696": "CRM systems help identify high-value customers, allowing brands to offer exclusive benefits and rewards to enhance customer loyalty.",
    "697": "Data analytics help brands analyze sales trends, customer preferences, and fashion shifts, allowing them to adjust inventory accordingly.",
    "698": "Strategies include transparent communication, eco-friendly product lines, and participation in social and environmental initiatives to engage customers in sustainability.",
    "699": "Brands can implement regular data cleansing processes and encourage customers to update their information to maintain accurate CRM data.",
    "700": "A feedback and improvement loop allows brands to continually gather insights, make necessary improvements, and demonstrate responsiveness to customer needs, strengthening relationships.",
}

# Open the file for writing
with open("output.txt", "w") as f:
    # Loop through the questions and answers
    for example in to_predict:
        context = example["context"]
        question = example["qas"][0]["question"]
        example_id = example["qas"][0]["id"]

        # Make predictions for the current question
        answers, _ = model.predict([example])
        example_id = example["qas"][0]["id"]

        # Extract the true answer based on the example_id
        true_answer = true_answers.get(example_id, "No")  # Default to "No" if ID is not found

        # Extract the predicted answer strings from the list (if there are multiple)
        predicted_answers = [answer for answer in answers[0]["answer"] if answer.strip()]

        # Initialize variables to keep track of EM and F1 for this example
        em = 0  # Initialize EM to 0
        f1 = 0.0  # Initialize F1 to 0
        precision = 0.0  # Initialize precision to 0

        # Iterate through predicted answers (if multiple)
        for predicted_answer in predicted_answers:
            # Calculate EM for this prediction
            em = max(em, int(true_answer == predicted_answer))  # Case-insensitive comparison for EM

            # Calculate precision for this prediction
            common_tokens = set(true_answer.lower().split()) & set(predicted_answer.lower().split())
            precision = len(common_tokens) / len(predicted_answer.split()) if len(predicted_answer.split()) > 0 else 0

            # Calculate recall for this prediction
            recall = len(common_tokens) / len(true_answer.split()) if len(true_answer.split()) > 0 else 0

            # Calculate F1 score for this prediction
            f1 = max(f1, 2 * (precision * recall) / (precision + recall)) if (precision + recall) > 0 else 0

        # Calculate EM based on whether the true answer is in predicted answers
        em = 1 if true_answer in predicted_answers else 0

        # Accumulate the scores for overall evaluation
        total_f1_score += f1
        total_em += em
        total_precision += precision  # Accumulate precision score

        # Append the individual F1 score to the list
        f1_scores.append(f1)

        # Write the context, question, true answer, predicted answers, precision, F1, and EM to the file
        f.write(f"Item: {example_id}:\n")
        f.write(f"Context: {context}\n")
        f.write(f"Question: {question}\n")
        f.write(f"True answer: {true_answer}\n")
        f.write(f"Predicted answers: {predicted_answers}\n")
        f.write(f"Precision score: {precision}\n")
        f.write(f"F1 score: {f1}\n")
        f.write(f"EM: {em}\n")
        f.write("\n")  # Add a separator between examples

    # Calculate the overall F1 score and precision for all examples
    overall_avg_f1 = total_f1_score / len(to_predict)
    overall_avg_em = total_em / len(to_predict)
    overall_avg_precision = total_precision / len(to_predict)

    # Write the overall F1 score, precision, and EM to the file
    f.write(f"\nOverall Average F1 score: {overall_avg_f1}\n")
    f.write(f"Overall Average Precision score: {overall_avg_precision}\n")
    f.write(f"Overall Average EM: {overall_avg_em}\n")
    f.write(f"\nProposed Training and Prediction Finish!\n")

In [ ]:
# Define the compute_f1_em function
def compute_f1_em(true_answer, predicted_answer):
    common_tokens = set(true_answer.lower().split()) & set(predicted_answer.lower().split())
    precision = len(common_tokens) / len(predicted_answer.split()) if len(predicted_answer.split()) > 0 else 0
    recall = len(common_tokens) / len(true_answer.split()) if len(true_answer.split()) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    em = 1 if true_answer.lower() == predicted_answer.lower() else 0
    return f1, em

while True:
    user_input = input("User: ")

    if user_input.lower() == "exit":
        print("Goodbye!")
        break

    context1 = ("OXGN is a fashion brand designed for all. From cool styles, affordable essentials and one of a kind collaboration pieces, our customers are sure to find that signature OXGN brand of style. With timeless classics from our GENERATIONS collection, the leveled-up sports inspired pieces from PREMIUM THREADS and our inclusive co-gender line COED, our brand serves to make looking cool as easy as breathing.")
    context2 = ("GOLDEN ABC, Inc. (GABC) is a multi-awarded international fashion enterprise that is home to top proprietary brands shaping the retail industry today. Produced, marketed, and retailed under a fast-growing, dynamic family of well-differentiated, proprietary brands: PENSHOPPE, OXYGEN, FORME, MEMO, REGATTA, and BOCU. Originating in the Philippines in 1986, GABC now has more than 1000 strategically located sites and presence in different countries around Asia.")
    context3 = ("The OXGN VIP Loyalty Card is not applicable online and it has been already been discontinued in our physical stores last December 31, 2021.")
    context4 = ("A confirmation e-mail/ SMS will be sent for all successful orders made. This contains your order number, the order amount, item(s) ordered, your billing and delivery address, including the payment method used and how you will be receiving order updates.")
    context5 = ("Yes, you have the option cancel your order within two (2) hours from the time it was created. You can see this on your order confirmation email or on your online store order history. Once the time frame has passed, the option to cancel will no longer appear on your end. If in case you still need to cancel your order but you no longer have the option to do so, you can send an email to shop@oxgnfashion.com and our Customer Care Team will get in touch with you.")
    context6 = ("We cannot include paper bags for Online Orders shipped to their chosen delivery address but we can definitely provide for those Online orders that were checked-out on a Store Pick-up option.")
    context7 = ("After you tap on REVIEW MY CART, you will see the option to send your order as a Gift. Sending it as a gift means that a gift receipt will be used instead of a regular sales invoice. A gift receipt shows proof of purchase but leaves out the amount spent.")
    context8 = ("Once the check out process has been completed, changes to the order in terms of item/size/color/quantity can no longer be made. What you can do is cancel your order completely within two (2) hours from the time you placed it so you can place a new one with the changes you want.")
    context9 = ("The first step is to click 'Add to Cart' once you've found something you want to save the items in your order basket. Second, click the 'Review My Cart' button to be redirected to the cart page, where you can make any final changes to your order, such as quantity, size, or style. Third, address Finder will appear on your screen. Please enter your delivery address by using the dropdown menus. If you accidentally exit out before finishing your details, click on 'Address Finder' to bring it back up. This is to prevent your orders from being held due to incorrect or incomplete shipping information.")
    context10 = ("Yes, you have the option cancel your order within two (2) hours from the time it was created either it is shipped to your prefered address or pick-up in your chosen You can view this on your order confirmation email or on your online store order history. Once the time frame has passed, the option to cancel will no longer appear on your end.")
    context11 = ("All unclaimed packages beyond the scheduled pick-up date are considered canceled and will be put back to our stocks.")
    context12 = ("You can now order online and pick up the items in your chosen OXGN branches: SM CITY NORTH EDSA, SM CITY SAN LAZARO, AYALA TRINOMA, ROBINSONS GALLERIA, SM MEGAMALL, SM CITY CEBU ,AYALA CENTER CEBU, MARKET MARKET, SM CITY FAIRVIEW, SM SEASIDE CEBU, SM LANANG, GMALL TAGUM, SM CITY DAVAO, GAISANO MALL OF DAVAO, SM CITY EAST ORTIGAS, SM MARIKINA, AYALA GLORIETTA, SM CITY MANILA, SM CITY GRAND CENTRAL, MALL OF ASIA, SM CITY SUCAT, ROBINSONS LAOAG, SM CITY BAGUIO, SM CITY PAMPANGA, SM CITY CLARK, SM CITY DASMARIÑAS, SM CALAMBA, SM CITY LUCENA, SM CITY LIPA, SM CITY ILO-ILO, SM CITY BACOLOD, SM Roxas, AYALA CENTRAL BLOC, Robinsons Tacloban, MINDPRO, ROBINSONS ILIGAN, Gaisano Mall Digos, SM CITY GENERAL SANTOS, KCC MALL MARBEL, SM CITY CAUAYAN, ISABELA, SM CITY MARILAO, TUTUBAN CENTER, Val Alabang, AYALA MALLS MANILA BAY, SM CITY LEGAZPI, ALTURAS MALL TAGBILARAN, SM PREMIER, SM BUTUAN, Mall of Alnor, SM City Tuguegarao, Ever Commonwelath, ROBINSONS BACOLOD, Fishermall.")
    context13 = ("As much as we'd like to, this option is not yet allowed at the moment.")
    context14 = ("This feature gives you the option to buy the items online and pick it up in our stores.")
    context15 = ("Oxgnfashion.com accepts the following forms of payment: GCash, Credit/Debit Card and COD (Cash-on-Delivery). During the order checkout process, you will be asked to select the payment method of your choice. If you select Credit/Debit Card, you will be redirected to our payment portal where you will be asked to enter your card details.")
    context16 = ("No, there is no option to save your credit or debit card information on your online store profile. Even if you have already used the same card, we do not store your card details to protect the security of your data.")
    context17 = ("Customers will receive an e-mail notification when their order is read for pick up. It is stated in the notification that they can pick up the order within 7 days from the receipt of e-mail.")
    context18 = ("No worries! You may assign a representative that may pick-up your order and have them present the following: Representative's Valid ID, Authorization letter with your Full Name and Signature, Photocopy of your Valid ID, and Screenshot of Order No. from email.")
    context19 = ("If you have placed order via our online store and chose to pick it up in one of branches, please prepare the following to be checked by our store partners: Valid ID, Screenshot of Order No. from email.")
    context20 = ("Customers with prepaid orders are allowed to request for up to 7 days extension from the first pick-up schedule.")
    context21 = ("If your order gets returned to our warehouse, we're sorry to let you know that it is already considered as canceled from our end. This is only done after two (2) unsuccessful delivery attempts.")
    context22 = ("Our delivery partners will make two (2) attempts in getting your package delivered. If they missed you on the first instance, the courier will either send you an SMS or give you a call to let you know that he came.")
    context23 = ("As we believe in making the latest fashion styles accessible to everyone, we can assure you that we have wide reach across the Philippines. However, due to geographical and logistical restrictions, there might be some isolated areas we cannot serve but rest assured we'll do our best to get to your location.")
    context24 = ("Customers who need to contact our shipping partners can do so using the following methods: For LBC, customers can send an email to customercare@lbcexpress.com or call (02) 8 858 59 99. For Go21, customers can visit the Inquiry Page at https://booking.go21.com.ph/ or call +63 2 832 99 27. For J&T, customers can visit https://www.jtexpress.sg/contact-us or call (02) 911 1888. For WSI, customers can send an email to communicationsupport@wsi.ph or call (02) 8533 3888. For feedback on our shipping partners or delivery concerns, customers can send an email to shop@oxgnfashion.com or fill out the Website Contact Us Form.")
    context25 = ("Once your order has been handed over to our shipping partners, you will receive an email notification along with your order tracking number.")
    context26 = ("Customers who wish to return items purchased from our online store can do so by packing the item along with the sales invoice and bringing it to any Penshoppe store nationwide. The return must be made within 30 days from the date of purchase, with all tags attached and the item unused and unwashed.")
    context27 = ("If you receive a gift item from our online store and want to have it exchanged for a different size/color/style, you have the option to do so following our Returns Policy.")
    context28 = ("If you need to process a return or exchange transaction and have lost your online store receipt, please reach out to us with the following information: your order number, the item you will be returning, the reason for the return, the boutique you will be going to, and the date.")
    context29 = ("No, our warehouse does not accept returns from customers but our physical stores do.")
    context30 = ("In the event that you received a damaged or incorrect item, it is important to reach out to us immediately so that we can investigate the situation.")
    context31 = ("Refund requests are always subject to validation. It is important to note that a change in size, style, or color is not eligible for a refund.")
    context32 = ("Creating an account on the OXGN Online Store is easy and allows you to keep track of your orders through the Order History feature.")
    context33 = ("Yes, you can still shop without making an online store account! Just place the items you want in your cart and proceed to check out.")
    context34 = ("If you have forgotten your password for your online store account, don't worry as it is an easy process to reset it.")
    context35 = ("First time to sign in? Tap on the account icon so you can start adding your account details.")
    context36 = ("Reach out to +639399101047 or complete the form below and we’ll gladly assist you.")
    context37 = ("These Terms and Conditions ('Terms', 'Terms and Conditions') govern your relationship with OXGN Website (the 'Service') operated by Golden ABC, Incorporated ('us', 'we', or 'our').")
    context38 = ("There are ten available variations of tops for men: Regular Fit Shirt, Resort Fit Shirt, Long Sleeve, Hoodie, Relax set-in Pullover, Easy Fit, Jersy, Slim Fit, Oversized, and Tank Top.")
    context39 = ("There are five available variations of bottoms for men: Skinny, Tapered (Slim Fit, Woven, Knit), and Jogger Pants.")
    context40 = ("There are two available variations of loungewear for men: Robe and Lounge Set.")
    context41 = ("There are six available variations of footwear for men: Sliders, Single Band, Velcro Sliders, Regular Fliptops, Lace-up, and Lace-up Runner.")
    context42 = ("There are seven available variations of tops for ladies: Regular, Boxy, Slim fit Tee, Shirt (Short Sleeve), Sports Bra, Hoodie, and Regular Pullover.")
    context43 = ("There are two available variations of dress for ladies: Oversized Dress, Long Sleeve Regular Fit.")
    context44 = ("There are ten available variations of bottoms for ladies: Skinny Jeans (Numerical,  Alpha), Wide Leg Jeans, Shorts (Numerical, Alpha), Biker Shorts (Alpha), Leggings (With Pockets, Without Pockets), Track Pants, and Urban Tapered Trousers.")
    context45 = ("There are four available variations of footwear for ladies: Sliders, Single Band Sliders, Velcro Sliders, and Lace-up Sneakers.")
    context46 = ("There are three available categories of accessories in total: Bags, Caps and Hats, and Wallet and Purses.")
    context47 = ("There are four available variations of Bags in accessories: Drawstring Backpack, Bum Bag, Sling Bag, and Tote Bag.")
    context48 = ("There are two available variations of Caps/Hat in accessories: Trucker Hat and Bucket Hat.")
    context49 = ("There are three available variations of walltes/purses in accessories: Coin Purse (Horizontal & Vertical) and Midsized Bifold Wallet.")
    context50 = ("For men's top regular fit shirt in double extra small, Chest: 37, Body Length: 26, Sleeve Length: 7.75, USA: 10, EU: 44, and UK: 34.")
    context51 = ("For men's top regular fit shirt in extra small, Chest: 39, Body Length: 27, Sleeve Length: 8, USA: 12, EU: 46, and UK: 36.")
    context52 = ("For men's top regular fit shirt in small, Chest: 41, Body Length: 28, Sleeve Length: 8.25, USA: 14, EU: 48, and UK: 38.")
    context53 = ("For men's top regular fit shirt in medium, Chest: 43, Body Length: 29, Sleeve Length: 8.5, USA: 16, EU: 50, and UK: 40.")
    context54 = ("For men's top regular fit shirt in large, Chest: 45, Body Length: 30, Sleeve Length: 8.75, USA: 18, EU: 52, and UK: 42.")
    context55 = ("For men's top regular fit shirt in extra large, Chest: 47, Body Length: 31, Sleeve Length: 9, USA: 20, EU: 54, and UK: 44.")
    context56 = ("For men's top regular fit shirt in double extra large, Chest: 49, Body Length: 32, Sleeve Length: 9.25, USA: 22, EU: 56, and UK: 46.")
    context57 = ("For men's top resort fit shirt in double extra small, Chest: 39, Body Length: 27.5, Sleeve Length: 9.25, USA: 10, EU: 44, and UK: 34.")
    context58 = ("For men's top resort fit shirt in extra small, Chest: 41, Body Length: 28, Sleeve Length: 9.5, USA: 12, EU: 46, and UK: 36.")
    context59 = ("For men's top resort fit shirt in small, Chest: 43, Body Length: 28.5, Sleeve Length: 9.75, USA: 14, EU: 48, and UK: 38.")
    context60 = ("For men's top resort fit shirt in medium, Chest: 45, Body Length: 29, Sleeve Length: 10, USA: 16, EU: 50, and UK: 40.")
    context61 = ("For men's top resort fit shirt in large, Chest: 47, Body Length: 29.5, Sleeve Length: 10.25, USA: 18, EU: 52, and UK: 42.")
    context62 = ("For men's top resort fit shirt in extra large, Chest: 49, Body Length: 30, Sleeve Length: 10.5, USA: 20, EU: 54, and UK: 44.")
    context63 = ("For men's top resort fit shirt in double extra large, Chest: 51, Body Length: 30.5, Sleeve Length: 10.75, USA: 22, EU: 56, and UK: 46.")
    context64 = ("For men's top long sleeve shirt in double extra small, Chest: 36, Body Length: 26, Sleeve Length: 24.75, USA: 10, EU: 44, and UK: 34.")
    context65 = ("For men's top long sleeve shirt in extra small, Chest: 38, Body Length: 27, Sleeve Length: 25, USA: 12, EU: 46, and UK: 36.")
    context66 = ("For men's top long sleeve shirt in small, Chest: 40, Body Length: 28, Sleeve Length: 25.25, USA: 14, EU: 48, and UK: 38.")
    context67 = ("For men's top long sleeve shirt in medium, Chest: 42, Body Length: 29, Sleeve Length: 25.5, USA: 16, EU: 50, and UK: 40.")
    context68 = ("For men's top long sleeve shirt in large, Chest: 44, Body Length: 30, Sleeve Length: 25.75, USA: 18, EU: 52, and UK: 42.")
    context69 = ("For men's top long sleeve shirt in extra large, Chest: 46, Body Length: 31, Sleeve Length: 26, USA: 20, EU: 54, and UK: 44.")
    context70 = ("For men's top long sleeve shirt in double extra large, Chest: 48, Body Length: 32, Sleeve Length: 26.25, USA: 22, EU: 56, and UK: 46.")
    context71 = ("For men's top long sleeve shirt in triple extra large, Chest: 50, Body Length: 33, Sleeve Length: 26.5, USA: 24, EU: 58, and UK: 48.")
    context72 = ("For men's top hoodie dropped shoulder shirt in double extra small, Chest: 44, Body Length: 25.5, Sleeve Length: 17.25, USA: 10, EU: 44, and UK: 34.")
    context73 = ("For men's top hoodie dropped shoulder shirt in extra small, Chest: 46, Body Length: 26, Sleeve Length: 17.5, USA: 12, EU: 46, and UK: 36.")
    context74 = ("For men's top hoodie dropped shoulder shirt in small, Chest: 48, Body Length: 26.5, Sleeve Length: 17.75, USA: 14, EU: 48, and UK: 38.")
    context75 = ("For men's top hoodie dropped shoulder shirt in medium, Chest: 50, Body Length: 27, Sleeve Length: 18, USA: 16, EU: 50, and UK: 40.")
    context76 = ("For men's top hoodie dropped shoulder shirt in large, Chest: 52, Body Length: 27.5, Sleeve Length: 18.25, USA: 18, EU: 52, and UK: 42.")
    context77 = ("For men's top hoodie dropped shoulder shirt in extra large, Chest: 54, Body Length: 28, Sleeve Length: 18.5, USA: 20, EU: 54, and UK: 44.")
    context78 = ("For men's top hoodie dropped shoulder shirt in double extra large, Chest: 56, Body Length: 28.5, Sleeve Length: 18.75, USA: 22, EU: 56, and UK: 46.")
    context79 = ("For men's top hoodie dropped shoulder shirt in triple extra large, Chest: 58, Body Length: 29, Sleeve Length: 19, USA: 24, EU: 58, and UK: 48.")
    context80 = ("For men's top hoodie set-in shoulder shirt in double extra small, Chest: 38, Body Length: 25, Sleeve Length: 24.25, USA: 10, EU: 44, and UK: 34.")
    context81 = ("For men's top hoodie set-in shoulder shirt in extra small, Chest: 40, Body Length: 26, Sleeve Length: 24.5, USA: 12, EU: 46, and UK: 36.")
    context82 = ("For men's top hoodie set-in shoulder shirt in small, Chest: 42, Body Length: 27, Sleeve Length: 24.75, USA: 14, EU: 48, and UK: 38.")
    context83 = ("For men's top hoodie set-in shoulder shirt in medium, Chest: 44, Body Length: 28, Sleeve Length: 25, USA: 16, EU: 50, and UK: 40.")
    context84 = ("For men's top hoodie set-in shoulder shirt in large, Chest: 46, Body Length: 29, Sleeve Length: 25.25, USA: 18, EU: 52, and UK: 42.")
    context85 = ("For men's top hoodie set-in shoulder shirt in extra large, Chest: 48, Body Length: 30, Sleeve Length: 25.5, USA: 20, EU: 54, and UK: 44.")
    context86 = ("TFor men's top hoodie set-in shoulder shirt in double extra large, Chest: 50, Body Length: 31, Sleeve Length: 25.75, USA: 22, EU: 56, and UK: 46.")
    context87 = ("For men's top hoodie set-in shoulder shirt in triple extra large, Chest: 52, Body Length: 32, Sleeve Length: 26, USA: 24, EU: 58, and UK: 48.")
    context88 = ("For men's top relax set-in pullover shirt in double extra small, Chest: 19, Body Length: 26, Sleeve Length: 24.25, USA: 10, EU: 44, and UK: 34.")
    context89 = ("For men's top relax set-in pullover shirt in extra small, Chest: 20, Body Length: 27, Sleeve Length: 24.5, USA: 12, EU: 46, and UK: 36.")
    context90 = ("For men's top relax set-in pullover shirt in small, Chest: 21, Body Length: 27, Sleeve Length: 24.75, USA: 14, EU: 48, and UK: 38.")
    context91 = ("For men's top relax set-in pullover shirt in medium, Chest: 22, Body Length: 28, Sleeve Length: 25, USA: 16, EU: 50, and UK: 40.")
    context92 = ("For men's top relax set-in pullover shirt in large, Chest: 23, Body Length: 29, Sleeve Length: 25.25, USA: 18, EU: 52, and UK: 42.")
    context93 = ("For men's top relax set-in pullover shirt in extra large, Chest: 24, Body Length: 30, Sleeve Length: 25.5, USA: 20, EU: 54, and UK: 44.")
    context94 = ("For men's top relax set-in pullover shirt in double extra large, Chest: 25, Body Length: 31, Sleeve Length: 25.75, USA: 22, EU: 56, and UK: 46.")
    context95 = ("For men's tee easy fit shirt in double extra small, Chest: 18, Body Length: 26, Sleeve Length: 7.25, USA: 10, EU: 44, and UK: 34.")
    context96 = ("For men's tee easy fit shirt in extra small, Chest: 19, Body Length: 26, Sleeve Length: 7.5, USA: 12, EU: 46, and UK: 36.")
    context97 = ("For men's tee easy fit shirt in small, Chest: 20, Body Length: 27, Sleeve Length: 7.75, USA: 14, EU: 48, and UK: 38.")
    context98 = ("For men's tee easy fit shirt in medium, Chest: 21, Body Length: 28, Sleeve Length: 8, USA: 16, EU: 50, and UK: 40.")
    context99 = ("For men's tee easy fit shirt in large, Chest: 22, Body Length: 29, Sleeve Length: 8.25, USA: 18, EU: 52, and UK: 42.")
    context100 = ("For men's tee easy fit shirt in double extra large, Chest: 24, Body Length: 31, Sleeve Length: 8.75, USA: 22, EU: 56, and UK: 46.")
    context101 = ("For men's top jersey with raglan sleeves shirt in double extra small, Chest: 36, Body Length: 25, Sleeve Length: 13.5, USA: 10, EU: 44, and UK: 34.")
    context102 = ("For men's top jersey with raglan sleeves shirt in extra small, Chest: 38, Body Length: 26, Sleeve Length: 14, USA: 12, EU: 46, and UK: 36.")
    context103 = ("For men's top jersey with raglan sleeves shirt in small, Chest: 40, Body Length: 27, Sleeve Length: 14.5, USA: 14, EU: 48, and UK: 38.")
    context104 = ("For men's top jersey with raglan sleeves shirt in medium, Chest: 42, Body Length: 28, Sleeve Length: 15, USA: 16, EU: 50, and UK: 40.")
    context105 = ("For men's top jersey with raglan sleeves shirt in large, Chest: 44, Body Length: 29, Sleeve Length: 15.5, USA: 18, EU: 52, and UK: 42.")
    context106 = ("For men's top jersey with raglan sleeves shirt in extra large, Chest: 46, Body Length: 30, Sleeve Length: 16, USA: 20, EU: 54, and UK: 44.")
    context107 = ("For men's top jersey with raglan sleeves shirt in double extra large, Chest: 48, Body Length: 31, Sleeve Length: 16.5, USA: 22, EU: 56, and UK: 46.")
    context108 = ("For men's top jersey with raglan sleeves shirt in triple extra large, Chest: 50, Body Length: 32, Sleeve Length: 17, USA: 24, EU: 58, and UK: 48.")
    context109 = ("For men's tee slim fit shirt in double extra small, Chest: 17, Body Length: 25.5, Sleeve Length: 6.75, USA: 10, EU: 44, and UK: 34.")
    context110 = ("For men's tee slim fit shirt in extra small, Chest: 18, Body Length: 26, Sleeve Length: 7, USA: 12, EU: 46, and UK: 36.")
    context111 = ("For men's tee slim fit shirt in small, Chest: 19, Body Length: 26.5, Sleeve Length: 7.25, USA: 14, EU: 48, and UK: 38.")
    context112 = ("For men's tee slim fit shirt in medium, Chest: 20, Body Length: 27, Sleeve Length: 7.5, USA: 16, EU: 50, and UK: 40.")
    context113 = ("For men's tee slim fit shirt in large, Chest: 21, Body Length: 27.5, Sleeve Length: 7.75, USA: 18, EU: 52, and UK: 42.")
    context114 = ("For men's tee slim fit shirt in extra large, Chest: 22, Body Length: 28, Sleeve Length: 8, USA: 20, EU: 54, and UK: 44.")
    context115 = ("For men's tee slim fit shirt in double extra large, Chest: 23, Body Length: 28.5, Sleeve Length: 8.25, USA: 22, EU: 56, and UK: 46.")
    context116 = ("For men's tee oversized fit shirt in double extra small, Chest: 19, Body Length: 27, Sleeve Length: 8.5, USA: 10, EU: 44, and UK: 34.")
    context117 = ("For men's tee oversized fit shirt in extra small, Chest: 20, Body Length: 28, Sleeve Length: 8.75, USA: 12, EU: 46, and UK: 36.")
    context118 = ("For men's tee oversized fit shirt in small, Chest: 21, Body Length: 29, Sleeve Length: 9, USA: 14, EU: 48, and UK: 38.")
    context119 = ("For men's tee oversized fit shirt in medium, Chest: 22, Body Length: 30, Sleeve Length: 9.25, USA: 16, EU: 50, and UK: 40.")
    context120 = ("For men's tee oversized fit shirt in large, Chest: 23, Body Length: 31, Sleeve Length: 9.5, USA: 18, EU: 52, and UK: 42.")
    context121 = ("For men's tee oversized fit shirt in extra large, Chest: 24, Body Length: 32, Sleeve Length: 9.75, USA: 20, EU: 54, and UK: 44.")
    context122 = ("For men's tee oversized fit shirt in double extra large, Chest: 25, Body Length: 33, Sleeve Length: 10, USA: 22, EU: 56, and UK: 46.")
    context123 = ("For men's top tank top shirt in double extra small, Chest: 36, Body Length: 26, Sleeve Length: N/A, USA: 10, EU: 44, and UK: 34.")
    context124 = ("For men's top tank top shirt in extra small, Chest: 38, Body Length: 27, Sleeve Length: N/A, USA: 12, EU: 46, and UK: 36.")
    context125 = ("For men's top tank top shirt in small, Chest: 40, Body Length: 28, Sleeve Length: N/A, USA: 14, EU: 48, and UK: 38.")
    context126 = ("For men's top tank top shirt in medium, Chest: 42, Body Length: 29, Sleeve Length: N/A, USA: 16, EU: 50, and UK: 40.")
    context127 = ("For men's top tank top shirt in large, Chest: 44, Body Length: 30, Sleeve Length: N/A, USA: 18, EU: 52, and UK: 42.")
    context128 = ("For men's top tank top shirt in double extra large, Chest: 48, Body Length: 32, Sleeve Length: N/A, USA: 22, EU: 56, and UK: 46.")
    context129 = ("For men's jeans skinny fit (regular waist band) in size 29, Waist: 28, Hips: 34, Inseam: 28, USA: 38, EU: 38, and UK: 30.")
    context130 = ("For men's jeans skinny fit (regular waist band) in size 30, Waist: 29, Hips: 35, Inseam: 28, USA: 39, EU: 39, and UK: 30-31.")
    context131 = ("For men's jeans skinny fit (regular waist band) in size 31, Waist: 30, Hips: 36, Inseam: 28, USA: 40, EU: 40, and UK: 31.")
    context132 = ("For men's jeans skinny fit (regular waist band) in size 32, Waist: 31, Hips: 37, Inseam: 29, USA: 42, EU: 42, and UK: 32.")
    context133 = ("For men's jeans skinny fit (regular waist band) in size 34, Waist: 31, Hips: 37, Inseam: 29, USA: 42, EU: 42, and UK: 32.")
    context134 = ("For men's jeans slim fit (regular waist band) in size 29, Waist: 27, Hips: 36, Inseam: 28, USA: 38, EU: 38, and UK: 30.")
    context135 = ("For men's jeans slim fit (regular waist band) in size 30, Waist: 28, Hips: 37, Inseam: 28, USA: 39, EU: 39, and UK: 30-31.")
    context136 = ("For men's jeans slim fit (regular waist band) in size 31, Waist: 29, Hips: 38, Inseam: 28, USA: 40, EU: 40, and UK: 31.")
    context137 = ("For men's jeans slim fit (regular waist band) in size 32, Waist: 30, Hips: 39, Inseam: 29, USA: 42, EU: 42, and UK: 32.")
    context138 = ("For men's jeans slim fit (regular waist band) in size 34, Waist: 32, Hips: 41, Inseam: 29, USA: 44, EU: 44, and UK: 34.")
    context139 = ("For men's jeans jogger pants (garterized waist band) in size extra small, Waist: 28, Hips: 38, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context140 = ("For men's jeans jogger pants (garterized waist band) in size small, Waist: 30, Hips: 40, Inseam: 25, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context141 = ("For men's jeans jogger pants (garterized waist band) in size medium, Waist: 32, Hips: 42, Inseam: 25, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context142 = ("For men's jeans jogger pants (garterized waist band) in size large, Waist: 34, Hips: 44, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.")
    context143 = ("For men's jeans jogger pants (garterized waist band) in size extra large, Waist: 36, Hips: 46, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.")
    context144 = ("For men's tapered pants woven (regular waist band) in size extra small, Waist: 24, Hips: 37, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context145 = ("For men's tapered pants woven (regular waist band) in size small, Waist: 26, Hips: 39, Inseam: 25, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context146 = ("For men's tapered pants woven (regular waist band) in size medium, Waist: 28, Hips: 41, Inseam: 25, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context147 = ("For men's tapered pants woven (regular waist band) in size large, Waist: 30, Hips: 43, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.")
    context148 = ("For men's tapered pants woven (regular waist band) in size extra large, Waist: 32, Hips: 45, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.")
    context149 = ("For men's tapered pants knit (garterized waist band) in size extra small, Waist: 26, Hips: 38, Inseam: 26, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context150 = ("For men's tapered pants knit (garterized waist band) in size small, Waist: 28, Hips: 40, Inseam: 26, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context151 = ("For men's tapered pants knit (garterized waist band) in size medium, Waist: 30, Hips: 42, Inseam: 26, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context152 = ("For men's tapered pants knit (garterized waist band) in size large, Waist: 32, Hips: 44, Inseam: 26, USA: 43-44, EU: 43-44, and UK: 33-34.")
    context153 = ("For men's tapered pants knit (garterized waist band) in size extra large, Waist: 34, Hips: 46, Inseam: 26, USA: 45-46, EU: 45-46, and UK: 35-36.")
    context154 = ("For men's tapered pants with cuff-knit (garterized waist band) in size double extra small, Waist: 23, Hips: 36, Inseam: 25, USA: 32-34, EU: 32-34, and UK: 27-28.")
    context155 = ("For men's tapered pants with cuff-knit (garterized waist band) in size extra small, Waist: 25, Hips: 38, Inseam: 25, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context156 = ("For men's tapered pants with cuff-knit (garterized waist band) in size small, Waist: 27, Hips: 40, Inseam: 25, USA: 39-40, EU: 39-40 and UK: 30-31.")
    context157 = ("For men's tapered pants with cuff-knit (garterized waist band) in size medium, Waist: 29, Hips: 42, Inseam: 25, USA: 43-44, EU: 43-44 and UK: 31-32.")
    context158 = ("For men's tapered pants with cuff-knit (garterized waist band) in size large, Waist: 31, Hips: 44, Inseam: 25, USA: 43-44, EU: 43-44 and UK: 33-34.")
    context159 = ("For men's tapered pants with cuff-knit (garterized waist band) in size extra large, Waist: 33, Hips: 46, Inseam: 25, USA: 45-46, EU: 45-46 and UK: 35-36.")
    context160 = ("For men's tapered pants with cuff-knit (garterized waist band) in size double extra large, Waist: 35, Hips: 48, Inseam: 25, USA: 48-50, EU: 48-50 and UK: 37-38.")
    context161 = ("For men's urban shorts - knit & woven (garterized waist band) in size extra small, Waist: 28, Hips: 40, Inseam: 5, USA: 36-38, EU: 36-38 and UK: 29-30.")
    context162 = ("For men's urban shorts - knit & woven (garterized waist band) in size small, Waist: 30, Hips: 42, Inseam: 5, USA: 39-40, EU: 39-40 and UK: 30-31.")
    context163 = ("For men's urban shorts - knit & woven (garterized waist band) in size medium, Waist: 32, Hips: 44, Inseam: 5, USA: 40-42, EU: 40-42 and UK: 31-32.")
    context164 = ("For men's urban shorts - knit & woven (garterized waist band) in size large, Waist: 34, Hips: 46, Inseam: 5, USA: 43-44, EU: 43-44 and UK: 33-34.")
    context165 = ("For men's urban shorts - knit & woven (garterized waist band) in size extra large, Waist: 36, Hips: 48, Inseam: 5, USA: 45-46, EU: 45-46 and UK: 35-36.")
    context166 = ("For men's chino shorts - woven (regular waist band) in size 28, Waist: 28, Hips: 38, Inseam: 5, USA: 36, EU: 36, and UK: 29.")
    context167 = ("For men's chino shorts - woven (regular waist band) in size 29, Waist: 29, Hips: 39, Inseam: 5, USA: 38, EU: 38, and UK: 30.")
    context168 = ("For men's chino shorts - woven (regular waist band) in size 30, Waist: 30, Hips: 40, Inseam: 5, USA: 39, EU: 39, and UK: 30-31.")
    context169 = ("For men's chino shorts - woven (regular waist band) in size 31, Waist: 31, Hips: 41, Inseam: 5, USA: 40, EU: 40, and UK: 31.")
    context170 = ("For men's chino shorts - woven (regular waist band) in size 32, Waist: 32, Hips: 42, Inseam: 5, USA: 42, EU: 42, and UK: 32.")
    context171 = ("For men's chino shorts - woven (regular waist band) in size 34, Waist: 34, Hips: 44, Inseam: 5, USA: 44, EU: 44, and UK: 34.")
    context172 = ("For men's walk/board shorts - woven (garterized waist band) in size extra small, Waist: 28, Hips: 41, Inseam: 4.5, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context173 = ("For men's walk/board shorts - woven (garterized waist band) in size small, Waist: 30, Hips: 43, Inseam: 4.5, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context174 = ("For men's walk/board shorts - woven (garterized waist band) in size medium, Waist: 32, Hips: 45, Inseam: 4.5, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context175 = ("For men's walk/board shorts - woven (garterized waist band) in size large, Waist: 34, Hips: 47, Inseam: 4.5, USA: 43-44, EU: 43-44, and UK: 33-34.")
    context176 = ("For men's walk/board shorts - woven (garterized waist band) in size extra large, Waist: 36, Hips: 49, Inseam: 4.5, USA: 45-46, EU: 45-46, and UK: 35-36.")
    context177 = ("For men's boxer shorts (garterized waist band) in size double extra small, Waist: 20, Hips: 35, Inseam: 3.25, USA: 32, EU: 42, and UK: 8.")
    context178 = ("For men's boxer shorts (garterized waist band) in size extra small, Waist: 22, Hips: 36, Inseam: 3.25, USA: 34, EU: 44, and UK: 10.")
    context179 = ("For men's boxer shorts (garterized waist band) in size small, Waist: 24, Hips: 38, Inseam: 3.5, USA: 36, EU: 46, and UK: 12.")
    context180 = ("For men's boxer shorts (garterized waist band) in size medium, Waist: 26, Hips: 40, Inseam: 3.5, USA: 38, EU: 48, and UK: 14.")
    context181 = ("For men's boxer shorts (garterized waist band) in size large, Waist: 28, Hips: 42, Inseam: 3.75, USA: 40, EU: 50, and UK: 16.")
    context182 = ("For men's boxer shorts (garterized waist band) in size extra large, Waist: 30, Hips: 44, Inseam: 3.75, USA: 42, EU: 52, and UK: 18.")
    context183 = ("For men's boxer shorts (garterized waist band) in size double extra large, Waist: 32, Hips: 46, Inseam: 3.75, USA: 44, EU: 54, and UK: 20.")
    context184 = ("For men's lounge - robe in size small, Waist: 44, Hips: 40, Inseam: 19.5, USA: 12-16, EU: 46-50, and UK: 36-40.")
    context185 = ("For men's lounge - robe in size medium, Waist: 48, Hips: 42, Inseam: 19.5, USA: 18-20, EU: 52-54, and UK: 42-44.")
    context186 = ("For men's lounge set - top slim fit in size double extra small, Waist: 34, Hips: 25.5, Inseam: 6.75, USA: 10, EU: 44, and UK: 34.")
    context187 = ("For men's lounge set - top slim fit in size extra small, Waist: 36, Hips: 26, Inseam: 7, USA: 12, EU: 46, and UK: 36.")
    context188 = ("For men's lounge set - top slim fit in size small, Waist: 38, Hips: 26.5, Inseam: 7.25, USA: 14, EU: 48, and UK: 38.")
    context189 = ("For men's lounge set - top slim fit in size medium, Waist: 40, Hips: 27, Inseam: 7.5, USA: 16, EU: 50, and UK: 40.")
    context190 = ("For men's lounge set - top slim fit in size large, Waist: 42, Hips: 27.5, Inseam: 7.75, USA: 18, EU: 52, and UK: 42.")
    context191 = ("For men's lounge set - top slim fit in size extra large, Waist: 44, Hips: 28, Inseam: 8, USA: 20, EU: 54, and UK: 44.")
    context192 = ("For men's lounge set - top slim fit in size double extra large, Waist: 46, Hips: 28.5, Inseam: 8.25, USA: 22, EU: 56, and UK: 46.")
    context193 = ("For men's lounge set - bottom boxer shorts in size double extra small, Waist: 20, Hips: 36, Inseam: 3.25, USA: 33-35, EU: 33-35, and UK: 27-28.")
    context194 = ("For men's lounge set - bottom boxer shorts in size extra small, Waist: 22, Hips: 38, Inseam: 3.25, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context195 = ("For men's lounge set - bottom boxer shorts in size small, Waist: 24, Hips: 40, Inseam: 3.5, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context196 = ("For men's lounge set - bottom boxer shorts in size medium, Waist: 26, Hips: 42, Inseam: 3.5, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context197 = ("For men's lounge set - bottom boxer shorts in size large, Waist: 28, Hips: 44, Inseam: 3.75, USA: 43-44, EU: 43-44, and UK: 33-34.")
    context198 = ("For men's lounge set - bottom boxer shorts in size extra large, Waist: 30, Hips: 46, Inseam: 3.75, USA: 45-46, EU: 45-46, and UK: 35-36.")
    context199 = ("For men's lounge set - bottom boxer shorts in size double extra large, Waist: 32, Hips: 48, Inseam: 3.75, USA: 47-48, EU: 47-48, and UK: 37-38.")
    context200 = ("For men's footwear sliders unisex in size small, Ladies' US Size: 5, Men's US Size: N/A, Footbed Length: 23.5cm.")
    context201 = ("For men's footwear sliders unisex in size medium, Ladies' US Size: 6-7, Men's US Size: N/A, Footbed Length: 24.9cm.")
    context202 = ("For men's footwear sliders unisex in size large, Ladies' US Size: 8-9, Men's US Size: 6, Footbed Length: 26.3cm.")
    context203 = ("For men's footwear sliders unisex in size extra large, Ladies' US Size: N/A, Men's US Size: 7-8, Footbed Length: 27.5cm.")
    context204 = ("For men's footwear sliders unisex in size double extra large, Ladies' US Size: N/A, Men's US Size: 9-10, Footbed Length: 28.9cm.")
    context205 = ("For men's footwear single band sliders, (US: 6, EU: 39, CM: 26.1), (US: 7, EU: 40, CM: 26.8), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28.2), (US: 10, EU: 43, CM: 28.9).")
    context206 = ("For men's footwear velcro sliders, (US: 6, EU: 39, CM: 26.1), (US: 7, EU: 40, CM: 26.8), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28.2), (US: 10, EU: 43, CM: 28.5).")
    context207 = ("For men's regular flipflops, (US: 6, EU: 39, CM: 26.6), (US: 7, EU: 40, CM: 27.3), (US: 8, EU: 41, CM: 28), (US: 9, EU: 42, CM: 28.7), (US: 10, EU: 43, CM: 29.4), then the slippers thicknes is 15mm.")
    context208 = ("For men's lace-up sneakers, (US: 6, EU: 39, CM: 26.5), (US: 7, EU: 40, CM: 27), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28), (US: 10, EU: 43, CM: 28.5).")
    context209 = ("For men's lace-up runner shoes, (US: 6, EU: 39, CM: 26.5), (US: 7, EU: 40, CM: 27), (US: 8, EU: 41, CM: 27.5), (US: 9, EU: 42, CM: 28), (US: 10, EU: 43, CM: 28.5).")
    context210 = ("For ladies' top tee - regular fit in size extra small, Chest: 34, Body Length: 22.5, Sleeve Length: 7, USA: 2, EU: 30, and UK: 6.")
    context211 = ("For ladies' top tee - regular fit in size small, Chest: 36, Body Length: 23, Sleeve Length: 7.25, USA: 4, EU: 32, and UK: 8.")
    context212 = ("For ladies' top tee - regular fit in size medium, Chest: 38, Body Length: 23.5, Sleeve Length: 7.5, USA: 6, EU: 36, and UK: 10.")
    context213 = ("For ladies' top tee - regular fit in size large, Chest: 40, Body Length: 24, Sleeve Length: 7.75, USA: 8, EU: 40, and UK: 12.")
    context214 = ("For ladies' top tee - regular fit in size extra large, Chest: 42, Body Length: 24.5, Sleeve Length: 8, USA: 10, EU: 42, and UK: 42.")
    context215 = ("For ladies' top tee - boxy fit in size extra small, Chest: 39, Body Length: 19, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.")
    context216 = ("For ladies' top tee - boxy fit in size small, Chest: 41, Body Length: 19.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.")
    context217 = ("For ladies' top tee - boxy fit in size medium, Chest: 43, Body Length: 20, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.")
    context218 = ("For ladies' top tee - boxy fit in size large, Chest: 45, Body Length: 20.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.")
    context219 = ("For ladies' top tee - boxy fit in size extra large, Chest: 47, Body Length: 21, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.")
    context220 = ("For ladies' v-neck - slim fit in size extra small, Chest: 31, Body Length: 22.5, Sleeve Length: 7.25, USA: 2, EU: 30, and UK: 6.")
    context221 = ("For ladies' v-neck - slim fit in size small, Chest: 33, Body Length: 23, Sleeve Length: 7.5, USA: 4, EU: 32, and UK: 8.")
    context222 = ("For ladies' v-neck - slim fit in size medium, Chest: 35, Body Length: 23.5, Sleeve Length: 7.75, USA: 6, EU: 36, and UK: 10.")
    context223 = ("For ladies' v-neck - slim fit in size large, Chest: 37, Body Length: 24, Sleeve Length: 8, USA: 8, EU: 40, and UK: 12.")
    context224 = ("For ladies' v-neck - slim fit in size extra large, Chest: 39, Body Length: 24.5, Sleeve Length: 8.25, USA: 10, EU: 42, and UK: 14.")
    context225 = ("For ladies' rib slim fit tee in size extra small, Chest: 28, Body Length: 22.5, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.")
    context226 = ("For ladies' rib slim fit tee in size small, Chest: 30, Body Length: 23, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.")
    context227 = ("For ladies' rib slim fit tee in size medium, Chest: 30, Body Length: 23, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.")
    context228 = ("For ladies' rib slim fit tee in size large, Chest: 34, Body Length: 24, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.")
    context229 = ("For ladies' rib slim fit tee in size extra large, Chest: 36, Body Length: 24.5, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.")
    context230 = ("For ladies' slim fit bundle tees in size extra small, Chest: 30, Body Length: 23, Sleeve Length: 4.75, USA: 2, EU: 30, and UK: 6.")
    context231 = ("For ladies' slim fit bundle tees in size small, Chest: 32, Body Length: 23.5, Sleeve Length: 5, USA: 4, EU: 32, and UK: 8.")
    context232 = ("For ladies' slim fit bundle tees in size medium, Chest: 34, Body Length: 24, Sleeve Length: 5.25, USA: 6, EU: 36, and UK: 10.")
    context233 = ("For ladies' slim fit bundle tees in size large, Chest: 36, Body Length: 24.5, Sleeve Length: 5.5, USA: 8, EU: 40, and UK: 12.")
    context234 = ("For ladies' slim fit bundle tees in size extra large, Chest: 38, Body Length: 25, Sleeve Length: 5.75, USA: 10, EU: 42, and UK: 14.")
    context235 = ("For ladies' top tee oversized fit in size extra small, Chest: 38, Body Length: 24, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.")
    context236 = ("For ladies' top tee oversized fit in size small, Chest: 40, Body Length: 24.5 , Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.")
    context237 = ("For ladies' top tee oversized fit in size medium, Chest: 42, Body Length: 25 , Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.")
    context238 = ("For ladies' top tee oversized fit in size large, Chest: 44, Body Length: 25.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.")
    context239 = ("For ladies' top tee oversized fit in size extra large, Chest: 46, Body Length: 26, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.")
    context240 = ("For ladies' top shirt - short sleeves in size extra small, Chest: 40, Body Length: 20.5, Sleeve Length: 7.5, USA: 2, EU: 30, and UK: 6.")
    context241 = ("For ladies' top shirt - short sleeves in size small, Chest: 42, Body Length: 21, Sleeve Length: 7.75, USA: 4, EU: 32, and UK: 8.")
    context242 = ("For ladies' top shirt - short sleeves in size medium, Chest: 44, Body Length: 21.5, Sleeve Length: 8, USA: 6, EU: 36, and UK: 10.")
    context243 = ("For ladies' top shirt - short sleeves in size large, Chest: 46, Body Length: 22, Sleeve Length: 8.25, USA: 8, EU: 40, and UK: 12.")
    context244 = ("For ladies' top shirt - short sleeves in size extra large, Chest: 48, Body Length: 22.5, Sleeve Length: 8.5, USA: 10, EU: 42, and UK: 14.")
    context245 = ("For ladies' top shirt - long sleeves in size extra small, Chest: 40, Body Length: 27.5, Sleeve Length: 20.5, USA: 2, EU: 30, and UK: 6.")
    context246 = ("For ladies' top shirt - long sleeves in size small, Chest: 42, Body Length: 28, Sleeve Length: 20.75, USA: 4, EU: 32, and UK: 8.")
    context247 = ("For ladies' top shirt - long sleeves in size medium, Chest: 44, Body Length: 28.5, Sleeve Length: 21, USA: 6, EU: 36, and UK: 10.")
    context248 = ("For ladies' top shirt - long sleeves in size large, Chest: 46, Body Length: 29, Sleeve Length: 21.25, USA: 8, EU: 40, and UK: 12.")
    context249 = ("For ladies' top shirt - long sleeves in size extra large, Chest: 48, Body Length: 29.5, Sleeve Length: 21.5, USA: 10, EU: 42, and UK: 14.")
    context250 = ("For ladies' top - sports bra in size extra small, Chest: 28, Body Length: 9, Sleeve Length: N/A, USA: 2, EU: 30, and UK: 6.")
    context251 = ("For ladies' top - sports bra in size small, Chest: 30, Body Length: 9.5, Sleeve Length: N/A, USA: 4, EU: 32, and UK: 8.")
    context252 = ("For ladies' top - sports bra in size medium, Chest: 32, Body Length: 10, Sleeve Length: N/A, USA: 6, EU: 36, and UK: 10.")
    context253 = ("For ladies' top - sports bra in size large, Chest: 34, Body Length: 10.5, Sleeve Length: N/A, USA: 8, EU: 40, and UK: 12.")
    context254 = ("For ladies' top - sports bra in size extra large, Chest: 36, Body Length: 11, Sleeve Length: N/A, USA: 10, EU: 42, and UK: 14.")
    context255 = ("For ladies' top - hoodie in size extra small, Chest: 44, Body Length: 24.5, Sleeve Length: 17.5, USA: 2, EU: 30, and UK: 6.")
    context256 = ("For ladies' top - hoodie in size small, Chest: 46, Body Length: 25, Sleeve Length: 17.75, USA: 4, EU: 32, and UK: 8.")
    context257 = ("For ladies' top - hoodie in size medium, Chest: 48, Body Length: 25.5, Sleeve Length: 18, USA: 6, EU: 36, and UK: 10.")
    context258 = ("For ladies' top - hoodie in size large, Chest: 50, Body Length: 26, Sleeve Length: 18.25, USA: 8, EU: 40, and UK: 12.")
    context259 = ("For ladies' top - hoodie in size extra large, Chest: 52, Body Length: 26.5, Sleeve Length: 18.5, USA: 10, EU: 42, and UK: 14.")
    context260 = ("For ladies' top - crop pullover in size extra small, Chest: 36, Body Length: 17.5, Sleeve Length: 18.75, USA: 2, EU: 30, and UK: 6.")
    context261 = ("For ladies' top - crop pullover in size small, Chest: 38, Body Length: 18, Sleeve Length: 19, USA: 4, EU: 32, and UK: 8.")
    context262 = ("For ladies' top - crop pullover in size medium, Chest: 40, Body Length: 18.5, Sleeve Length: 19.25, USA: 6, EU: 36, and UK: 10.")
    context263 = ("For ladies' top - crop pullover in size large, Chest: 42, Body Length: 19, Sleeve Length: 19.5, USA: 8, EU: 40, and UK: 12.")
    context264 = ("For ladies' top - crop pullover in size extra large, Chest: 44, Body Length: 19.5, Sleeve Length: 19.75, USA: 10, EU: 42, and UK: 14.")
    context265 = ("For ladies' top - regular pullover in size extra small, Chest: 38, Body Length: 25.5, Sleeve Length: 20.75, USA: 2, EU: 30, and UK: 6.")
    context266 = ("For ladies' top - regular pullover in size small, Chest: 40, Body Length: 26, Sleeve Length: 21, USA: 4, EU: 32, and UK: 8.")
    context267 = ("For ladies' top - regular pullover in size medium, Chest: 42, Body Length: 26.5, Sleeve Length: 21.25, USA: 6, EU: 36, and UK: 10.")
    context268 = ("For ladies' top - regular pullover in size large, Chest: 44, Body Length: 27, Sleeve Length: 21.5, USA: 8, EU: 40, and UK: 12.")
    context269 = ("For ladies' top - regular pullover in size extra large, Chest: 46, Body Length: 27.5, Sleeve Length: 21.75, USA: 10, EU: 42, and UK: 14.")
    context270 = ("For ladies' dress - regular fit in size extra small, Chest: 33, Body Length: 32.5, Sleeve Length: 7, USA: 2, EU: 30, and UK: 6.")
    context271 = ("For ladies' dress - regular fit in size small, Chest: 35, Body Length: 33, Sleeve Length: 7.25, USA: 4, EU: 32, and UK: 8.")
    context272 = ("For ladies' dress - regular fit in size medium, Chest: 37, Body Length: 33.5, Sleeve Length: 7.5, USA: 6, EU: 36, and UK: 10.")
    context273 = ("For ladies' dress - regular fit in size large, Chest: 39, Body Length: 34, Sleeve Length: 7.75, USA: 8, EU: 40, and UK: 12.")
    context274 = ("For ladies' dress - regular fit in size extra large, Chest: 41, Body Length: 34.5, Sleeve Length: 8, USA: 10, EU: 42, and UK: 14.")
    context275 = ("For ladies' dress - oversized fit in size extra small, Chest: 38, Body Length: 33, Sleeve Length: 8.5, USA: 2, EU: 30, and UK: 6.")
    context276 = ("For ladies' dress - oversized fit in size small, Chest: 40, Body Length: 33.5, Sleeve Length: 8.75, USA: 4, EU: 32, and UK: 8.")
    context277 = ("For ladies' dress - oversized fit in size medium, Chest: 42, Body Length: 34, Sleeve Length: 9, USA: 6, EU: 36, and UK: 10.")
    context278 = ("For ladies' dress - oversized fit in size large, Chest: 44, Body Length: 34.5, Sleeve Length: 9.25, USA: 8, EU: 40, and UK: 12.")
    context279 = ("For ladies' dress - oversized fit in size extra large, Chest: 46, Body Length: 35, Sleeve Length: 9.5, USA: 10, EU: 42, and UK: 14.")
    context280 = ("For ladies' dress - long sleeved regular fit in size extra small, Chest: 38, Body Length: 32, Sleeve Length: 20, USA: 2, EU: 30, and UK: 6.")
    context281 = ("For ladies' dress - long sleeved regular fit in size small, Chest: 40, Body Length: 32.5, Sleeve Length: 20.25, USA: 4, EU: 32, and UK: 8.")
    context282 = ("For ladies' dress - long sleeved regular fit in size medium, Chest: 42, Body Length: 33, Sleeve Length: 20.5, USA: 6, EU: 36, and UK: 10.")
    context283 = ("For ladies' dress - long sleeved regular fit in size large, Chest: 44, Body Length: 33.5, Sleeve Length: 20.75, USA: 8, EU: 40, and UK: 12.")
    context284 = ("For ladies' dress - long sleeved regular fit in size extra large, Chest: 46, Body Length: 34, Sleeve Length: 21, USA: 10, EU: 42, and UK: 14.")
    context285 = ("For ladies' dress - baby doll dress in size extra small, Chest: 33, Body Length: 31, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.")
    context286 = ("For ladies' dress - baby doll dress in size small, Chest: 35, Body Length: 31.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.")
    context287 = ("For ladies' dress - baby doll dress in size medium, Chest: 37, Body Length: 32, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.")
    context288 = ("For ladies' dress - baby doll dress in size large, Chest: 39, Body Length: 32.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.")
    context289 = ("For ladies' dress - baby doll dress in size extra large, Chest: 41, Body Length: 33, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.")
    context290 = ("For ladies' bottoms - skinny jeans (numerical sizing) in size 26, Waist: 26, Hip: 32,  Inseam: 26, USA: 4, EU: 32, and UK: 8.")
    context291 = ("For ladies' bottoms - skinny jeans (numerical sizing) in size 27, Waist: 27, Hip: 33,  Inseam: 26, USA: 6, EU: 36, and UK: 10.")
    context292 = ("For ladies' bottoms - skinny jeans (numerical sizing) in size 28, Waist: 28, Hip: 34,  Inseam: 26, USA: 8, EU: 38, and UK: 12.")
    context293 = ("For ladies' bottoms - skinny jeans (numerical sizing) in size 29, Waist: 29, Hip: 35,  Inseam: 26, USA: 10, EU: 40, and UK: 14.")
    context294 = ("For ladies' bottoms - skinny jeans (numerical sizing) in size 30, Waist: 30, Hip: 36,  Inseam: 26, USA: 10, EU: 42, and UK: 16.")
    context295 = ("For ladies' bottoms - skinny jeans (numerical sizing) in size 32, Waist: 32, Hip: 38,  Inseam: 26, USA: 12, EU: 40, and UK: 14.")
    context296 = ("For ladies' bottoms - skinny jeans (alpha sizing) in size extra small, Waist: 24, Hip: 36,  Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context297 = ("For ladies' bottoms - skinny jeans (alpha sizing) in size small, Waist: 26, Hip: 38, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context298 = ("For ladies' bottoms - skinny jeans (alpha sizing) in size medium, Waist: 28, Hip: 40, Inseam: 27, USA: 8, EU: 38, and UK: 12.")
    context299 = ("For ladies' bottoms - skinny jeans (alpha sizing) in size large, Waist: 30, Hip: 42, Inseam: 27, USA: 10, EU: 40, and UK: 14.")
    context300 = ("For ladies' bottoms - skinny jeans (alpha sizing) in size extra large, Waist: 32, Hip: 44, Inseam: 27, USA: 12, EU: 42, and UK: 16.")
    context301 = ("For ladies' bottoms - basic straight jeans (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context302 = ("For ladies' bottoms - basic straight jeans (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 8.")
    context303 = ("For ladies' bottoms - basic straight jeans (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.")
    context304 = ("For ladies' bottoms - basic straight jeans (numerical sizing) in size 29, Waist: 39, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.")
    context305 = ("For ladies' bottoms - basic straight jeans (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.")
    context306 = ("For ladies' bottoms - basic straight jeans (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.")
    context307 = ("For ladies' bottoms - wide leg jeans (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context308 = ("For ladies' bottoms - wide leg jeans (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context309 = ("For ladies' bottoms - wide leg jeans (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.")
    context310 = ("For ladies' bottoms - wide leg jeans (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.")
    context311 = ("For ladies' bottoms - wide leg jeans (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.")
    context312 = ("For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 26, Waist: 25, Hip: 36.5, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context313 = ("For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 27, Waist: 26, Hip: 37.5, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context314 = ("For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 28, Waist: 27, Hip: 38.5, Inseam: 27, USA: 6, EU: 38, and UK: 10.")
    context315 = ("For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 29, Waist: 28, Hip: 39.5, Inseam: 27, USA: 10, EU: 38, and UK: 12.")
    context316 = ("For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 30, Waist: 29, Hip: 40.5, Inseam: 27, USA: 10, EU: 40, and UK: 12.")
    context317 = ("For ladies' bottoms - wide leg jeans (numeiracal sizing) in size 32, Waist: 31, Hip: 42.5, Inseam: 27, USA: 12, EU: 42, and UK: 16.")
    context318 = ("For ladies' bottoms - shorts (alpha sizing) in size extra small, Waist: 26, Hip: 36, Inseam: 2.5, USA: 4, EU: 32, and UK: 8.")
    context319 = ("For ladies' bottoms - shorts (alpha sizing) in size small, Waist: 28, Hip: 38, Inseam: 2.5, USA: 6, EU: 36, and UK: 10.")
    context320 = ("For ladies' bottoms - shorts (alpha sizing) in size medium, Waist: 30, Hip: 40, Inseam: 2.5, USA: 8, EU: 38, and UK: 12.")
    context321 = ("For ladies' bottoms - shorts (alpha sizing) in size large, Waist: 32, Hip: 42, Inseam: 2.5, USA: 10, EU: 40, and UK: 14.")
    context322 = ("For ladies' bottoms - shorts (alpha sizing) in size extra large, Waist: 34, Hip: 44, Inseam: 2.5, USA: 12, EU: 42, and UK: 16.")
    context323 = ("For ladies' bottoms - shorts (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 2.5, USA: 4, EU: 32, and UK: 8.")
    context324 = ("For ladies' bottoms - shorts (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 2.5, USA: 6, EU: 36, and UK: 10.")
    context325 = ("For ladies' bottoms - shorts (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 2.5, USA: 6, EU: 38, and UK: 10.")
    context326 = ("For ladies' bottoms - shorts (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 2.5, USA: 10, EU: 38, and UK: 12.")
    context327 = ("For ladies' bottoms - shorts (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 2.5, USA: 10, EU: 40, and UK: 12.")
    context328 = ("For ladies' bottoms - shorts (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 2.5, USA: 12, EU: 42, and UK: 16.")
    context329 = ("For ladies' bottoms - biker shorts (alpha sizing) in size extra small, Waist: 25, Hip: 28, Inseam: 10, USA: 4, EU: 32, and UK: 8.")
    context330 = ("For ladies' bottoms - biker shorts (alpha sizing) in size small, Waist: 27, Hip: 30, Inseam: 10, USA: 6, EU: 36, and UK: 10.")
    context331 = ("For ladies' bottoms - biker shorts (alpha sizing) in size medium, Waist: 29, Hip: 32, Inseam: 10, USA: 8, EU: 38, and UK: 12.")
    context332 = ("For ladies' bottoms - biker shorts (alpha sizing) in size large, Waist: 31, Hip: 34, Inseam: 10, USA: 10, EU: 40, and UK: 14.")
    context333 = ("For ladies' bottoms - biker shorts (alpha sizing) in size extra large, Waist: 33, Hip: 36, Inseam: 10, USA: 12, EU: 42, and UK: 16.")
    context334 = ("For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size extra small, Waist: 23, Hip: 25, Inseam: 26, USA: 4, EU: 32, and UK: 8.")
    context335 = ("For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size small, Waist: 25, Hip: 27, Inseam: 26, USA: 6, EU: 36, and UK: 10.")
    context336 = ("For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size medium, Waist: 27, Hip: 29, Inseam: 26, USA: 8, EU: 38, and UK: 12.")
    context337 = ("For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size large, Waist: 29, Hip: 31, Inseam: 26, USA: 10, EU: 40, and UK: 14.")
    context338 = ("For ladies' bottoms - sports leggings with side pockets (alpha sizing) in size extra large, Waist: 31, Hip: 33, Inseam: 26, USA: 12, EU: 42, and UK: 16.")
    context339 = ("For ladies' bottoms - sports leggings without pockets (alpha sizing) in size extra small, Waist: 25, Hip: 28, Inseam: 26.5, USA: 4, EU: 32, and UK: 8.")
    context340 = ("For ladies' bottoms - sports leggings without pockets (alpha sizing) in size small, Waist: 25, Hip: 28, Inseam: 26.5, USA: 4, EU: 32, and UK: 8.")
    context341 = ("For ladies' bottoms - sports leggings without pockets (alpha sizing) in size medium, Waist: 29, Hip: 32, Inseam: 26.5, USA: 8, EU: 38, and UK: 12.")
    context342 = ("For ladies' bottoms - sports leggings without pockets (alpha sizing) in size large, Waist: 31, Hip: 34, Inseam: 26.5, USA: 10, EU: 40, and UK: 14.")
    context343 = ("For ladies' bottoms - sports leggings without pockets (alpha sizing) in size extra large, Waist: 33, Hip: 36, Inseam: 26.5, USA: 12, EU: 42, and UK: 16.")
    context344 = ("For ladies' bottoms - track pants (mid-waist) in size extra small, Waist: 23, Hip: 35, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context345 = ("For ladies' bottoms - track pants (mid-waist) in size small, Waist: 25, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context346 = ("For ladies' bottoms - track pants (mid-waist) in size medium, Waist: 27, Hip: 39, Inseam: 27, USA: 6, EU: 38, and UK: 10.")
    context347 = ("For ladies' bottoms - track pants (mid-waist) in size large, Waist: 29, Hip: 41, Inseam: 27, USA: 10, EU: 38, and UK: 12.")
    context348 = ("For ladies' bottoms - track pants (mid-waist) in size extra large, Waist: 31, Hip: 43, Inseam: 27, USA: 10, EU: 40, and UK: 12.")
    context349 = ("For ladies' bottoms - track pants (high-waist) in size extra small, Waist: 21, Hip: 35, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context350 = ("For ladies' bottoms - track pants (high-waist) in size small, Waist: 23, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context351 = ("For ladies' bottoms - track pants (high-waist) in size medium, Waist: 25, Hip: 39, Inseam: 27, USA: 8, EU: 38, and UK: 12.")
    context352 = ("For ladies' bottoms - track pants (high-waist) in size large, Waist: 27, Hip: 41, Inseam: 27, USA: 10, EU: 40, and UK: 14.")
    context353 = ("For ladies' bottoms - track pants (high-waist) in size extra large, Waist: 29, Hip: 43, Inseam: 27, USA: 12, EU: 42, and UK: 16.")
    context354 = ("For ladies' bottoms - track pants with cuff in size extra small, Waist: 23, Hip: 35, Inseam: 26, USA: 4, EU: 32, and UK: 8.")
    context355 = ("For ladies' bottoms - track pants with cuff in size small, Waist: 25, Hip: 37, Inseam: 26, USA: 6, EU: 36, and UK: 10.")
    context356 = ("For ladies' bottoms - track pants with cuff in size medium, Waist: 27, Hip: 39, Inseam: 26, USA: 6, EU: 38, and UK: 10.")
    context357 = ("For ladies' bottoms - track pants with cuff in size large, Waist: 29, Hip: 41, Inseam: 26, USA: 10, EU: 38, and UK: 12.")
    context358 = ("For ladies' bottoms - track pants with cuff in size extra large, Waist: 31, Hip: 43, Inseam: 26, USA: 10, EU: 40, and UK: 12.")
    context359 = ("For ladies' bottoms - urban tapered trousers (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context360 = ("For ladies' bottoms - urban tapered trousers (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context361 = ("For ladies' bottoms - urban tapered trousers (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.")
    context362 = ("For ladies' bottoms - urban tapered trousers (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.")
    context363 = ("For ladies' bottoms - urban tapered trousers (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.")
    context364 = ("For ladies' bottoms - urban tapered trousers (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context365 = ("For ladies' bottoms - urban tapered trousers (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context366 = ("For ladies' bottoms - urban tapered trousers (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.")
    context367 = ("For ladies' bottoms - urban tapered trousers (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.")
    context368 = ("For ladies' bottoms - urban tapered trousers (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.")
    context369 = ("For ladies' bottoms - urban tapered trousers (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16..")
    context370 = ("For ladies' bottoms - urban wide trousers (alpha sizing) in size extra small, Waist: 24, Hip: 34, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context371 = ("For ladies' bottoms - urban wide trousers (alpha sizing) in size small, Waist: 26, Hip: 36, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context372 = ("For ladies' bottoms - urban wide trousers (alpha sizing) in size medium, Waist: 28, Hip: 38, Inseam: 27, USA: 8, EU: 38, and UK: 12.")
    context373 = ("For ladies' bottoms - urban wide trousers (alpha sizing) in size large, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 14.")
    context374 = ("For ladies' bottoms - urban wide trousers (alpha sizing) in size extra large, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.")
    context375 = ("For ladies' bottoms - urban wide trousers (numerical sizing) in size 26, Waist: 26, Hip: 36, Inseam: 27, USA: 4, EU: 32, and UK: 8.")
    context376 = ("For ladies' bottoms - urban wide trousers (numerical sizing) in size 27, Waist: 27, Hip: 37, Inseam: 27, USA: 6, EU: 36, and UK: 10.")
    context377 = ("For ladies' bottoms - urban wide trousers (numerical sizing) in size 28, Waist: 28, Hip: 38, Inseam: 27, USA: 6, EU: 38, and UK: 10.")
    context378 = ("For ladies' bottoms - urban wide trousers (numerical sizing) in size 29, Waist: 29, Hip: 39, Inseam: 27, USA: 10, EU: 38, and UK: 12.")
    context379 = ("For ladies' bottoms - urban wide trousers (numerical sizing) in size 30, Waist: 30, Hip: 40, Inseam: 27, USA: 10, EU: 40, and UK: 12.")
    context380 = ("For ladies' bottoms - urban wide trousers (numerical sizing) in size 32, Waist: 32, Hip: 42, Inseam: 27, USA: 12, EU: 42, and UK: 16.")
    context381 = ("For men's loungewear robe in size small, Chest: 44, Body Length: 40, Sleeve Length: 19.5, USA: 12-16, EU: 46-50, and UK: 36-40.")
    context382 = ("For men's loungewear robe in size medium, Chest: 48, Body Length: 42, Sleeve Length: 19.5, USA: 18-20, EU: 52-54, and UK: 42-44.")
    context383 = ("For ladies' lounge set - top (oversized tee) in size extra small, Chest: 39, Body Length: 19, Sleeve Length: 7.75, USA: 2, EU: 30, and UK: 6.")
    context384 = ("For ladies' lounge set - top (oversized tee) in size small, Chest: 41, Body Length: 19.5, Sleeve Length: 8, USA: 4, EU: 32, and UK: 8.")
    context385 = ("For ladies' lounge set - top (oversized tee) in size medium, Chest: 43, Body Length: 20, Sleeve Length: 8.25, USA: 6, EU: 36, and UK: 10.")
    context386 = ("For ladies' lounge set - top (oversized tee) in size large, Chest: 45, Body Length: 20.5, Sleeve Length: 8.5, USA: 8, EU: 40, and UK: 12.")
    context387 = ("For ladies' lounge set - top (oversized tee) in size extra large, Chest: 47, Body Length: 21, Sleeve Length: 8.75, USA: 10, EU: 42, and UK: 14.")
    context388 = ("For ladies' lounge set - bottom (shorts) in size extra small, Chest: 26, Body Length: 18, Sleeve Length: 2, USA: 4, EU: 32, and UK: 8.")
    context389 = ("For ladies' lounge set - bottom (shorts) in size small, Chest: 28, Body Length: 19, Sleeve Length: 2, USA: 6, EU: 36, and UK: 10.")
    context390 = ("For ladies' lounge set - bottom (shorts) in size medium, Chest: 30, Body Length: 20, Sleeve Length: 2, USA: 8, EU: 38, and UK: 12.")
    context391 = ("For ladies' lounge set - bottom (shorts) in size large, Chest: 32, Body Length: 21, Sleeve Length: 2, USA: 10, EU: 40, and UK: 14.")
    context392 = ("For ladies' lounge set - bottom (shorts) in size extra large, Chest: 34, Body Length: 22, Sleeve Length: 2, USA: 12, EU: 42, and UK: 16.")
    context393 = ("For ladies' lounge set - top (spaghetti) in size extra small, Chest: 31, Body Length: 16, Sleeve Length: N/A, USA: 2, EU: 30, and UK: 6.")
    context394 = ("For ladies' lounge set - top (spaghetti) in size small, Chest: 33, Body Length: 16.5, Sleeve Length: N/A, USA: 4, EU: 32, and UK: 8.")
    context395 = ("For ladies' lounge set - top (spaghetti) in size medium, Chest: 35, Body Length: 17, Sleeve Length: N/A, USA: 6, EU: 36, and UK: 10.")
    context396 = ("For ladies' lounge set - top (spaghetti) in size large, Chest: 37, Body Length: 17.5, Sleeve Length: N/A, USA: 8, EU: 40, and UK: 12.")
    context397 = ("For ladies' lounge set - top (spaghetti) in size extra large, Chest: 39, Body Length: 18, Sleeve Length: N/A, USA: 10, EU: 42, and UK: 14.")
    context398 = ("For ladies' lounge set - bottom (shorts) in size extra small, Chest: 26, Body Length: 36, Sleeve Length: 2, USA: 4, EU: 32, and UK: 8.")
    context399 = ("For ladies' lounge set - bottom (shorts) in size small, Chest: 28, Body Length: 38, Sleeve Length: 2, USA: 6, EU: 36, and UK: 10.")
    context400 = ("For ladies' lounge set - bottom (shorts) in size medium, Chest: 30, Body Length: 40, Sleeve Length: 2, USA: 8, EU: 38, and UK: 12.")
    context401 = ("For ladies' lounge set - bottom (shorts) in size large, Chest: 32, Body Length: 42, Sleeve Length: 2, USA: 10, EU: 40, and UK: 14.")
    context402 = ("For ladies' lounge set - bottom (shorts) in size extra large, Chest: 34, Body Length: 44, Sleeve Length: 2, USA: 12, EU: 42, and UK: 16.")
    context403 = ("For ladies' footwear single band sliders, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24.2), (US: 7, EU: 37, CM: 24.9), (US: 8, EU: 38, CM: 25.6), (US: 9, EU: 39, CM: 26.3).")
    context404 = ("For ladies' footwear velcro sliders, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24.2), (US: 7, EU: 37, CM: 24.9), (US: 8, EU: 38, CM: 25.6), (US: 9, EU: 39, CM: 26.3).")
    context405 = ("For ladies' regular flipflops, (US: 5, EU: 35, CM: 23.2), (US: 6, EU: 36, CM: 23.9), (US: 7, EU: 37, CM: 24.6), (US: 8, EU: 38, CM: 25.3), (US: 9, EU: 39, CM: 26), then the slippers thicknes is 15mm.")
    context406 = ("For ladies' lace-up sneakers, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24), (US: 7, EU: 37, CM: 24.5), (US: 8, EU: 38, CM: 25), (US: 9, EU: 39, CM: 25.5).")
    context407 = ("For ladies' runner shoes, (US: 5, EU: 35, CM: 23.5), (US: 6, EU: 36, CM: 24), (US: 7, EU: 37, CM: 24.5), (US: 8, EU: 38, CM: 25), (US: 9, EU: 39, CM: 25.5).")
    context408 = ("For men's coed tee in size double extra small, Chest: 36, Body Length: 27.5, Sleeve Length: 7, USA: 10, EU: 44, and UK: 34.")
    context409 = ("For men's coed tee in size extra small, Chest: 38, Body Length: 27.5, Sleeve Length: 7.25, USA: 12, EU: 46, and UK: 36.")
    context410 = ("For men's coed tee in size small, Chest: 40, Body Length: 28, Sleeve Length: 7.5, USA: 14, EU: 48, and UK: 38.")
    context411 = ("For men's coed tee in size medium, Chest: 42, Body Length: 28.5, Sleeve Length: 7.75, USA: 16, EU: 50, and UK: 40.")
    context412 = ("For men's coed tee in size large, Chest: 44, Body Length: 29, Sleeve Length: 8, USA: 18, EU: 52, and UK: 42.")
    context413 = ("For men's coed tee in size extra large, Chest: 46, Body Length: 30, Sleeve Length: 8.25, USA: 20, EU: 54, and UK: 44.")
    context414 = ("For men's coed tee in size double extra large, Chest: 48, Body Length: 30.5, Sleeve Length: 8.5, USA: 22, EU: 56, and UK: 46.")
    context415 = ("For men's coed tee in size triple extra large, Chest: 50, Body Length: 31, Sleeve Length: 8.75, USA: 24, EU: 58, and UK: 48.")
    context416 = ("For ladies' coed tee in size double extra small, Chest: 36, Body Length: 27.5, Sleeve Length: 7, USA: 10, EU: 44, and UK: 34.")
    context417 = ("For ladies' coed tee in size medium, Chest: 38, Body Length: 27.5, Sleeve Length: 7.25, USA: 12, EU: 46, and UK: 36.")
    context418 = ("For ladies' coed tee in size large, Chest: 40, Body Length: 28, Sleeve Length: 7.5, USA: 14, EU: 48, and UK: 38.")
    context419 = ("For ladies' coed tee in size extra large, Chest: 42, Body Length: 28.5, Sleeve Length: 7.75, USA: 16, EU: 50, and UK: 40.")
    context420 = ("For men's coed pullover in size double extra small, Chest: 38, Body Length: 24, Sleeve Length: 21.25, USA: 10, EU: 44, and UK: 34.")
    context421 = ("For men's coed pullover in size extra small, Chest: 40, Body Length: 25, Sleeve Length: 21.5, USA: 12, EU: 46, and UK: 36.")
    context422 = ("For men's coed pullover in size small, Chest: 42, Body Length: 26, Sleeve Length: 21.75, USA: 14, EU: 48, and UK: 38.")
    context423 = ("For men's coed pullover in size medium, Chest: 44, Body Length: 27, Sleeve Length: 22, USA: 16, EU: 50, and UK: 42.")
    context424 = ("For men's coed pullover in size large, Chest: 46, Body Length: 28, Sleeve Length: 22.25, USA: 18, EU: 52, and UK: 42.")
    context425 = ("For men's coed pullover in size extra large, Chest: 48, Body Length: 29, Sleeve Length: 22.5, USA: 20, EU: 54, and UK: 44.")
    context426 = ("For men's coed pullover in size double extra large, Chest: 50, Body Length: 30, Sleeve Length: 22.75, USA: 22, EU: 56, and UK: 46.")
    context427 = ("For men's coed pullover in size triple extra large, Chest: 52, Body Length: 31, Sleeve Length: 23, USA: 24, EU: 58, and UK: 48.")
    context428 = ("For ladies' coed pullover in size small, Chest: 38, Body Length: 24, Sleeve Length: 21.25, USA: 10, EU: 44, and UK: 34.")
    context429 = ("For ladies' coed pullover in size medium, Chest: 40, Body Length: 25, Sleeve Length: 21.5, USA: 12, EU: 46, and UK: 36.")
    context430 = ("For ladies' coed pullover in size large, Chest: 42, Body Length: 26, Sleeve Length: 21.75, USA: 14, EU: 48, and UK: 38.")
    context431 = ("or ladies' coed pullover in size extra large, Chest: 44, Body Length: 27, Sleeve Length: 22, USA: 16, EU: 50, and UK: 40.")
    context432 = ("For men's coed woven trousers (urban straight trouser) in size double extra small, Waist: 22, Hip: 34, Inseam: 23, USA: 34-36, EU: 35-36, and UK: 27-28.")
    context433 = ("For men's coed woven trousers (urban straight trouser) in size extra small, Waist: 24, Hip: 36, Inseam: 23, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context434 = ("For men's coed woven trousers (urban straight trouser) in size small, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context435 = ("For men's coed woven trousers (urban straight trouser) in size medium, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context436 = ("For men's coed woven trousers (urban straight trouser) in size large, Waist: 30, Hip: 42, Inseam: 25, USA: 43-44, EU: 43-44, and UK: 33-34.")
    context437 = ("For men's coed woven trousers (urban straight trouser) in size extra large, Waist: 32, Hip: 44, Inseam: 25, USA: 45-46, EU: 45-46, and UK: 35-36.")
    context438 = ("For ladies' coed woven trousers (urban straight trouser) in size small, Waist: 22, Hip: 34, Inseam: 23, USA: 34-36, EU: 35-36, and UK: 27-28.")
    context439 = ("For ladies' coed woven trousers (urban straight trouser) in size medium, Waist: 24, Hip: 36, Inseam: 23, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context440 = ("For ladies' coed woven trousers (urban straight trouser) in size large, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context441 = ("For ladies' coed woven trousers (urban straight trouser) in size extra large, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context442 = ("For men's coed shorts (woven) in size double extra small, Waist: 27, Hip: 37, Inseam: 5, USA: 34-36, EU: 35-36, and UK: 27-28.")
    context443 = ("For men's coed shorts (woven) in size extra small, Waist: 29, Hip: 39, Inseam: 5, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context444 = ("For men's coed shorts (woven) in size small, Waist: 31, Hip: 41, Inseam: 5, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context445 = ("For men's coed shorts (woven) in size medium, Waist: 33, Hip: 43, Inseam: 5, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context446 = ("For men's coed shorts (woven) in size large, Waist: 35, Hip: 45, Inseam: 5, USA: 43-44, EU: 43-44, and UK: 33-34.")
    context447 = ("For men's coed shorts (woven) in size extra large, Waist: 37, Hip: 47, Inseam: 5, USA: 45-46, EU: 45-46, and UK: 35-36.")
    context448 = ("For ladies' coed shorts (woven) in size small, Waist: 27, Hip: 37, Inseam: 5, USA: 34-36, EU: 35-36, and UK: 27-28.")
    context449 = ("For ladies' coed shorts (woven) in size medium, Waist: 29, Hip: 39, Inseam: 5, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context450 = ("For ladies' coed shorts (woven) in size large, Waist: 31, Hip: 41, Inseam: 5, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context451 = ("For ladies' coed shorts (woven) in size extra large, Waist: 33, Hip: 43, Inseam: 5, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context452 = ("For men's coed woven trousers (tapered) in size double extra small, Waist: 22, Hip: 34, Inseam: 24, USA: 34-36, EU: 35-36, and UK: 27-28.")
    context453 = ("For men's coed woven trousers (tapered) in size extra small, Waist: 24, Hip: 36, Inseam: 24, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context454 = ("For men's coed woven trousers (tapered) in size small, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context455 = ("For men's coed woven trousers (tapered) in size medium, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context456 = ("For men's coed woven trousers (tapered) in size large, Waist: 30, Hip: 42, Inseam: 24, USA: 43-44, EU: 43-44, and UK: 33-34.")
    context457 = ("For men's coed woven trousers (tapered) in size extra large, Waist: 32, Hip: 44, Inseam: 24, USA: 45-46, EU: 45-46, and UK: 35-36.")
    context458 = ("For ladies' coed woven trousers (tapered) in size small, Waist: 22, Hip: 34, Inseam: 24, USA: 34-36, EU: 35-36, and UK: 27-28.")
    context459 = ("For ladies' coed woven trousers (tapered) in size medium, Waist: 24, Hip: 36, Inseam: 24, USA: 36-38, EU: 36-38, and UK: 29-30.")
    context460 = ("For ladies' coed woven trousers (tapered) in size large, Waist: 26, Hip: 38, Inseam: 24, USA: 39-40, EU: 39-40, and UK: 30-31.")
    context461 = ("For ladies' coed woven trousers (tapered) in size extra large, Waist: 28, Hip: 40, Inseam: 24, USA: 40-42, EU: 40-42, and UK: 31-32.")
    context462 = ("For accessories - regular backpack, Length: 17 inches, Width: 11 inches, Depth: 5 inches, Strap Length: 16.5 inches, Strap Width: 2.75 inches.")
    context463 = ("For accessories - large backpack, Length: 18 inches, Width: 12 inches, Depth: 6 inches, Strap Length: 16.5 inches, Strap Width: 2.75 inches.")
    context464 = ("For accessories - drawstring backpack, Length: 17 inches and Width: 13 inches.")
    context465 = ("For accessories - regular bum bag, Length: 4.5 inches, Width: 13.5 inches, Depth: 3 inches, Strap Length: 43 inches, Strap Width: 1.5 inches.")
    context466 = ("For accessories - multi pockets bum bag, Length: 7.5 inches, Width: 13 inches, Depth: 3.5 inches.")
    context467 = ("For accessories - oversized bum bag, Length: 9 inches, Width: 18 inches, Depth: 4 inches.")
    context468 = ("For accessories - duffle bag, Length: 9.75 inches, Width: 18 inches, Depth: 9.75 inches.")
    context469 = ("For accessories - mini sling bag, Length: 7 inches, Width: 5 inches, Depth: 1.5 inches, Strap Length: 53 inches, Strap Width: 1 inches.")
    context470 = ("For accessories - regular sling bag, Length: 9 inches, Width: 7 inches, Depth: 2 inches, Strap Length: 53 inches, Strap Width: 1 inches.")
    context471 = ("For accessories - mid sized messenger bag, Length: 10 inches, Width: 8 inches, Depth: 2.5 inches, Strap Length: 5.3 inches, Strap Width: 1.5 inches.")
    context472 = ("For accessories - type 1 tote bag, Length: 16 inches, Width: 10 inches, Depth: 4 inches, Strap Length: 26 inches, Strap Width: 1.5 inches.")
    context473 = ("For accessories - type 2 tote bag, Length: 16 inches, Width: 13.5 inches, Depth: N/A, Strap Length: 26 inches, Strap Width: 1 inches.")
    context474 = ("For accessories - beanie, Circumference: 57 cm and Height: 8.89 cm.")
    context475 = ("For accessories - bucket hat, Circumference: 59 cm, Height: 8.89 cm, Visor Length: 6.35 cm.")
    context476 = ("For accessories - curved cap, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.99 cm.")
    context477 = ("For accessories - dad hat, Circumference: 57 cm, Height: 17.78 cm, Visor Length: 6.99 cm.")
    context478 = ("For accessories - snap back, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.35 cm.")
    context479 = ("For accessories - trucker cap, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.99 cm.")
    context480 = ("For accessories - type 1 coin purse, Length: 4 inches, Width: 3 inches, Depth: 0.75 inch, Zipper Length: 3 3/4 inches.")
    context481 = ("For accessories - type 2 coin purse, Length: 3 inches, Width: 4 inches, Depth: 0.75 inch, Zipper Length: 5 1/2 inches.")
    context482 = ("For accessories - bifold wallet, Folded: (Length: 3.5 inches, Width: 3 1/2 inches, Depth: 0.5 inch), Bill Compartment: (Length: 4.5 inches and Height: 3 1/4 inches).")
    context483 = ("For accessories - midsized bifold wallet, Folded: (Length: 3 1/4 inches, Width: 4 1/2 inches), Bill Compartment: (Length: 6 1/2 inches and Height: 3 3/4 inches).")
    context484 = ("For accessories - trifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).")
    context485 = ("For accessories - zip around bifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches, Depth: 0.5 inch), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).")
    context486 = ("For accessories - curved cap, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.99 cm.")
    context487 = ("For accessories - type 2 tote bag, Length: 16 inches, Width: 13.5 inches, Depth: N/A, Strap Length: 26 inches, Strap Width: 1 inches.")
    context488 = ("For accessories - curved cap, Circumference: 57 cm, Height: 16.51 cm, Visor Length: 6.99 cm.")
    context489 = ("For accessories - zip around bifold wallet, Folded: (Length: 4 1/4 inches, Width: 3 1/2 inches, Depth: 0.5 inch), Bill Compartment: (Length: 8 1/2 inches and Height: 3 1/4 inches).")
    context490 = ("Yes, we offer international shipping. You can place your order online, and we'll ship it to your desired destination.")
    context491 = ("Sale items are eligible for returns and exchanges within 14 days of purchase. Please ensure the item is in its original condition.")
    context492 = ("You can stay updated on our upcoming sales and events by subscribing to our newsletter or following our social media channels.")
    context493 = ("Yes, we offer personal shopping and styling services. Our team can help you find the perfect outfit tailored to your style.")
    context494 = ("You can apply for our store credit card, which offers benefits like exclusive discounts, rewards, and special financing options.")
    context495 = ("We welcome your feedback! You can provide feedback through our website, in-store, or by contacting our customer support.")
    context496 = ("Yes, we can place a special order for items that are out of stock in your size. Please speak with a store associate for assistance.")
    context497 = ("We offer gift wrapping services for a small fee. Your items will be beautifully packaged for a special touch.")
    context498 = ("We release new collections regularly, typically with the changing seasons. Stay tuned for our latest arrivals.")
    context499 = ("Yes, we offer the option to book private events or fashion shows in our store. Contact our events team for details and availability.")
    context500 = ("Yes, we offer express and same-day delivery options for online orders in select locations.")
    context501 = ("Absolutely! We have a loyalty program where you can earn rewards and exclusive discounts.")
    context502 = ("We offer price adjustments for items purchased within seven days before a sale. Bring your receipt for verification.")
    context503 = ("Yes, you can request clothing alterations at the time of purchase. Our team can assist you with adjustments.")
    context504 = ("Yes, we offer special discounts for students and senior citizens. Please bring a valid ID for verification.")
    context505 = ("We encourage clothing donations and recycling. You can inquire in-store about our donation programs.")
    context506 = ("Certainly! We can assist with bulk orders for special events. Please contact our sales team for more details.")
    context507 = ("Yes, we have a mobile app that makes online shopping convenient and offers exclusive deals.")
    context508 = ("You can track your online order and check its status by logging into your account on our website or app.")
    context509 = ("Yes, we offer gift-wrapping services for online orders, making it a perfect gift option.")
    context510 = ("We offer store credit for returns within 30 days, or refunds in the original payment method.")
    context511 = ("Yes, email subscribers receive exclusive discounts and early access to promotions.")
    context512 = ("You can check the availability of items in a specific store location by contacting the store directly.")
    context513 = ("We have robust security measures to protect customer data and adhere to strict privacy policies.")
    context514 = ("Yes, we offer a gift registry service for weddings and special occasions. Visit our website for details.")
    context515 = ("Yes, we have a referral program where you and your friends can enjoy discounts.")
    context516 = ("You can apply for job openings by visiting our careers page on our website and submitting your application.")
    context517 = ("Yes, we offer corporate gifting solutions, including branded and customized gift options.")
    context518 = ("Yes, we can arrange to transfer items from other store locations for your pickup.")
    context519 = ("You can schedule a fashion consultation or appointment by contacting our store and booking a time slot.")
    context520 = ("Yes, we accept mobile payment options like Apple Pay and Google Pay for your convenience.")
    context521 = ("Yes, you can exchange a gift for a different size or style within our return policy guidelines.")
    context522 = ("Yes, we have a store newsletter that provides updates, promotions, and exclusive offers to subscribers.")
    context523 = ("We have implemented measures to ensure a safe and comfortable shopping experience, including enhanced cleaning and safety protocols.")
    context524 = ("For custom-made clothing, we require measurements and specific details to tailor the item to your preferences.")
    context525 = ("Yes, you can use multiple payment methods for a single purchase. Our staff can assist with split payments.")
    context526 = ("We will gladly replace or refund defective or damaged items. Please bring them to our attention within our return policy timeframe.")
    context527 = ("We value your feedback! You can provide feedback in our stores or through our website.")
    context528 = ("Yes, you can place orders for gifts and have them shipped to multiple addresses during the checkout process.")
    context529 = ("You can sign up for a store credit card in-store or online, and eligibility requirements will be reviewed during the application process.")
    context530 = ("Good morning! How can I assist you today?")
    context531 = ("Hello! How can I help you?")
    context532 = ("Good afternoon! What can I do for you?")
    context533 = ("Good day! How may I assist you?")
    context534 = ("Good evening! How can I be of service?")
    context535 = ("Good night! How can I assist you at this hour?")
    context536 = ("Good dawn! What brings you here so early?")
    context537 = ("Greetings! How can I help you as the day winds down?")
    context538 = ("Good morning and sunrise to you! How can I assist you?")
    context539 = ("Good evening and sunset to you! What can I do for you?")
    context540 = ("Hello! How may I assist you during this beautiful twilight?")
    context541 = ("A tie bar, when worn properly, holds the tie in place and adds a stylish and functional element to a business outfit.")
    context542 = ("When selecting dress shoes, consider classic styles like oxfords or derbies, and ensure they are well-polished and match the color of your belt.")
    context543 = ("To incorporate seasonal trends, add accessories or select clothing items in seasonal colors while maintaining a professional core look.")
    context544 = ("For a black-tie event, men should wear a black tuxedo, a black bow tie, and polished black dress shoes.")
    context545 = ("Women have various options for footwear, including pumps, loafers, ankle boots, and closed-toe heels to match their business outfits.")
    context546 = ("Statement jewelry can be worn in business attire but should be used sparingly to avoid appearing too flashy or distracting.")
    context547 = ("A tailored dress shirt ensures a perfect fit and a sharp, professional appearance, as it's designed to complement your body shape.")
    context548 = ("Transition from day to night by adding a blazer, statement accessories, and a change of shoes to elevate your look for evening events.")
    context549 = ("Turtlenecks can be appropriate in business attire, especially in colder months, when worn with a well-fitted blazer or suit.")
    context550 = ("Incorporate colors like navy or deep blue to convey confidence, or choose power colors like red or burgundy for a bolder statement.")
    context551 = ("Good midnight! How can I assist you at this late hour?")
    context552 = ("Good noon! How can I help you at this point in the day?")
    context553 = ("Good morning! I'm here to help. What can I do for you?")
    context554 = ("Hello! I'm here to assist you with anything you need.")
    context555 = ("Good afternoon! Feel free to ask any questions or seek assistance.")
    context556 = ("Hello! I'm here to assist you with anything you need. How can I help?")
    context557 = ("Good evening! I'm here to help. How can I be of service?")
    context558 = ("Good night! I'm here to assist you at this hour. What can I do for you?")
    context559 = ("Good dawn! I'm here to assist you. How can I help at this early hour?")
    context560 = ("Greetings! I'm here to help as the day winds down. What can I do for you?")
    context561 = ("Good morning and sunrise to you! I'm here to help. How can I assist you today?")
    context562 = ("Good evening and sunset to you! I'm here to assist you. What can I do for you?")
    context563 = ("Hello! I'm here to help during this lovely twilight. How may I assist you?")
    context564 = ("Good midnight! I'm here to assist you at this late hour. What can I do for you?")
    context565 = ("Good noon! I'm here to assist you. What can I do for you at this time?")
    context566 = ("Business casual attire typically includes slacks, a button-up shirt, and closed-toe shoes for men, and a blouse, slacks or skirt, and closed-toe shoes for women.")
    context567 = ("Business formal attire is more formal than business casual and often includes a suit and tie for men, and a tailored suit or dress for women.")
    context568 = ("Essential accessories may include a classic watch, a leather belt, and tasteful jewelry.")
    context569 = ("To prolong the life of your suits, dry clean them only when necessary, hang them properly, and store them in a garment bag.")
    context570 = ("Neutral colors like black, gray, navy, and white are often considered appropriate for business attire.")
    context571 = ("A power tie, typically a bold and confident color, is worn to convey authority and confidence in business settings.")
    context572 = ("Dress codes in the workplace help maintain a professional image and set a standard for appropriate attire.")
    context573 = ("Choose lightweight, breathable fabrics like linen or cotton and opt for short-sleeved shirts or dresses.")
    context574 = ("In many business settings, open-toed shoes are not considered appropriate; closed-toe shoes are a safer choice.")
    context575 = ("Dress in professional attire that's slightly more formal than the company's dress code to make a positive impression.")
    context576 = ("Denim is generally not considered business casual, but some workplaces allow it on specific days or with certain conditions.")
    context577 = ("Wear solid, non-distracting colors, and make sure your top half looks professional as it's what's visible on camera.")
    context578 = ("A blazer is a versatile piece that can elevate a business casual look and add a touch of formality.")
    context579 = ("Sneakers are typically not considered appropriate in business settings; opt for more formal footwear.")
    context580 = ("Personal grooming is crucial for making a positive impression and projecting professionalism.")
    context581 = ("Look for sales, outlet stores, and second-hand shops to find high-quality attire at affordable prices.")
    context582 = ("A well-fitted suit can enhance your confidence and convey attention to detail, which is highly regarded in the business world.")
    context583 = ("Subtle patterns like stripes or checks are acceptable in business attire, but avoid loud or distracting prints.")
    context584 = ("Women can accessorize with a stylish handbag, tasteful jewelry, and closed-toe heels or flats.")
    context585 = ("Dress formally, often in a tailored suit for men and an elegant dress for women, to show respect for the occasion.")
    context586 = ("Turtlenecks can be appropriate in a business casual setting, especially in colder months, as long as they are clean and well-maintained.")
    context587 = ("A classic Windsor knot or a four-in-hand knot are suitable for a professional look in the business world.")
    context588 = ("Incorporate your company's branding colors subtly in your attire, such as with a tie or scarf, to show alignment with the brand.")
    context589 = ("Women can wear pantsuits in the business world, as they offer a professional and stylish alternative to dresses or skirts.")
    context590 = ("Cufflinks add a touch of elegance and personal style to dress shirts with French cuffs, making them a symbol of sophistication.")
    context591 = ("For a business-casual event, consider wearing chinos, a button-down shirt, and loafers for a polished yet relaxed appearance.")
    context592 = ("All-black attire can be appropriate in a business setting, but it's important to add some variety with accessories or subtle patterns.")
    context593 = ("Dress in professional attire that's slightly more formal than the company's dress code to make a positive impression.")
    context594 = ("Look for sales, outlet stores, and second-hand shops to find high-quality attire at affordable prices.")
    context595 = ("Business casual attire typically includes slacks, a button-up shirt, and closed-toe shoes for men, and a blouse, slacks or skirt, and closed-toe shoes for women.")
    context596 = ("For a business dress, opt for fabrics like wool, wool blends, or high-quality synthetic materials for a professional look.")
    context597 = ("A business skirt should typically fall just above or below the knee, ensuring a professional and modest appearance.")
    context598 = ("Vests or waistcoats can add a touch of sophistication to business attire and are still considered stylish.")
    context599 = ("A pocket square adds a touch of sophistication and personal style to a suit jacket, enhancing its overall appearance.")
    context600 = ("Different collar styles suit different face shapes; for example, spread collars can complement round faces, while point collars work well with oval faces.")
    context601 = ("When selecting a clothing store, consider factors such as location, product range, price range, and customer reviews to find the best fit for your shopping preferences.")
    context602 = ("To find the best deals, sign up for newsletters, follow the store's social media accounts, and keep an eye on sales and clearance sections.")
    context603 = ("Popular clothing store chains known for quality and selection include OXGN, Zara, H&M, Nordstrom, and Macy's.")
    context604 = ("A well-organized store layout makes shopping easier, helping customers find what they need quickly and enhancing their overall shopping experience.")
    context605 = ("Use the store's sizing charts, and don't hesitate to ask for assistance from store staff to ensure you find the right size.")
    context606 = ("To make the changing room experience smoother, limit the number of items you bring in, and ensure you check the store's return policy.")
    context607 = ("Stay updated on fashion trends by following fashion magazines, blogs, and influencers, and then shop for trendy items at your favorite clothing stores.")
    context608 = ("Yes, there are eco-friendly clothing stores that offer sustainable and environmentally-conscious fashion choices, promoting ethical and responsible shopping.")
    context609 = ("A personal shopper can assist customers in finding clothing that suits their style and needs, providing a tailored shopping experience.")
    context610 = ("To make the most of sales and promotions, plan your shopping trips during sale events, and prioritize items you need.")
    context611 = ("Shopping at thrift stores can be budget-friendly, so look for well-maintained and timeless pieces to score high-quality clothing at affordable prices.")
    context612 = ("Yes, many clothing stores provide personalized tailoring services to ensure your clothing fits perfectly and is tailored to your preferences.")
    context613 = ("Customer reviews and ratings provide valuable insights into the store's quality, service, and customer satisfaction, helping you make informed decisions.")
    context614 = ("To maintain the quality of your clothing, follow care instructions, store garments properly, and handle stains promptly.")
    context615 = ("Yes, some clothing stores specialize in specific fashion niches or styles, catering to unique tastes, such as vintage, bohemian, or streetwear.")
    context616 = ("Loyalty programs can offer rewards, discounts, and exclusive deals for repeat customers, enhancing the shopping experience and building brand loyalty.")
    context617 = ("To identify counterfeit items, research authentic designer features and logos, and be cautious when the price seems too good to be true.")
    context618 = ("A fashion stylist can provide style advice, help you put together outfits, and suggest clothing items that complement your personal style.")
    context619 = ("To make the most of the fitting room experience, take your time, try on different sizes, and assess the comfort and fit of each item.")
    context620 = ("Several clothing stores, such as Patagonia, EILEEN FISHER, and Reformation, are known for their commitment to sustainable and ethical fashion practices.")
    context621 = ("Budget your spending by setting a limit, making a shopping list, and considering cost per wear when evaluating potential purchases.")
    context622 = ("Yes, many clothing stores collaborate with designers or brands to offer exclusive, limited-edition releases that attract fashion enthusiasts.")
    context623 = ("When shopping online, consider checking size charts, reading product descriptions, and reviewing return policies to make informed decisions.")
    context624 = ("Make sustainable choices by opting for eco-friendly materials, supporting ethical brands, and recycling or upcycling clothing items.")
    context625 = ("When searching for a special occasion dress, consider the dress code, venue, and your personal style to find the perfect outfit.")
    context626 = ("Avoid impulse buying by creating a shopping list, setting a budget, and taking your time to carefully consider each purchase.")
    context627 = ("Many clothing stores now offer inclusive sizing options including us in OXGN store, catering to a wide range of body types and promoting body positivity.")
    context628 = ("Seasonal sales and clearance events provide an opportunity to purchase clothing items at reduced prices, making it easier to update your wardrobe.")
    context629 = ("To find clothing stores catering to specific age groups or demographics, read customer reviews, consult fashion magazines, and explore social media for recommendations.")
    context630 = ("Clothing stores like Aerie and Target are known for their commitment to diversity and inclusivity, both in marketing and product offerings.")
    context631 = ("Trying on clothing items allows you to assess the fit, comfort, and overall look of the garment, helping you make confident purchasing decisions.")
    context632 = ("After a shopping spree, organize and declutter your closet by categorizing items, donating or selling clothing you no longer need, and maximizing storage space.")
    context633 = ("When searching for the perfect pair of jeans, consider your body shape, preferred fit, and wash options, and consult store staff for assistance.")
    context634 = ("Discover local boutiques by exploring your neighborhood, seeking recommendations from friends or online platforms, and attending local fashion events.")
    context635 = ("Online customer reviews offer insights into a store's product quality, service, and customer experiences, aiding in your decision-making process.")
    context636 = ("Brands and stores like Everlane, People Tree, and Outerknown prioritize fair labor practices and ethical sourcing in their fashion production.")
    context637 = ("Look for clothing stores that offer customization or made-to-measure services to ensure your clothing fits perfectly and suits your unique style.")
    context638 = ("Clothing store loyalty programs reward frequent shoppers with exclusive discounts, early access to sales, and other perks, enhancing their shopping experience.")
    context639 = ("To find vintage or retro clothing stores, explore local thrift shops, consignment stores, or online marketplaces specializing in vintage fashion.")
    context640 = ("Make environmentally conscious choices by opting for eco-friendly materials, supporting sustainable brands, and choosing clothing made with ethical practices.")
    context641 = ("Inspect clothing items for sturdy stitching, quality fabrics, no defects, and attention to detail, which indicate good quality and durability.")
    context642 = ("Department stores like Bloomingdale's and specialty stores like Men's Wearhouse offer a wide range of formal wear options for special events.")
    context643 = ("To align your clothing purchases with your style and needs, plan your wardrobe, create a color palette, and focus on versatile pieces that can be mixed and matched.")
    context644 = ("Brands like Aerie, Dove, and Fenty Beauty are recognized for promoting body positivity and diverse representation in their advertising campaigns.")
    context645 = ("A fashion consultant can offer personalized style advice, recommend clothing items, and help customers discover fashion choices that suit their preferences.")
    context646 = ("Look for timeless and versatile pieces like classic blazers, white shirts, and well-fitted jeans that can be worn in various outfits and styles.")
    context647 = ("A fashion consultant can offer personalized style advice, recommend clothing items, and help customers discover fashion choices that suit their preferences.")
    context648 = ("Calculate the cost per wear by dividing the item's price by the number of times you anticipate wearing it, helping you make cost-effective decisions.")
    context649 = ("A clothing store's return and exchange policy ensures flexibility in case a purchase doesn't meet your expectations or fit as intended.")
    context650 = ("Look for clothing stores that actively support charitable causes or donate a portion of their proceeds to social initiatives, making a positive impact through fashion.")
    context651 = ("Customers often engage with clothing brands through social media, customer reviews, and feedback forms to share their opinions and preferences.")
    context652 = ("Customer feedback is used to identify areas for improvement, refine product designs, and create offerings that align better with customer preferences.")
    context653 = ("Clothing stores employ strategies like consistent pricing, inventory availability across channels, and responsive customer service to create a seamless shopping experience.")
    context654 = ("Businesses protect customer data by implementing robust cybersecurity measures, encryption, and complying with data protection regulations to ensure security and privacy.")
    context655 = ("Social media platforms provide a channel for clothing brands to engage with customers, gather feedback, and address concerns in real-time.")
    context656 = ("Clothing retailers can encourage feedback through surveys, incentives, and user-friendly feedback platforms, making it convenient for customers to share their insights.")
    context657 = ("Building trust involves delivering on promises, transparency, and addressing issues promptly, ultimately fostering strong and credible customer relationships.")
    context658 = ("Personalization in marketing campaigns tailors content to individual customer preferences, increasing engagement and conversion rates for clothing brands.")
    context659 = ("Effective complaint management involves prompt acknowledgment, thorough investigation, resolution, and feedback, ensuring customer satisfaction.")
    context660 = ("Strategies include loyalty programs, personalized discounts, and exceptional customer service to retain loyal customers and reduce churn.")
    context661 = ("Data analytics help clothing retailers analyze customer behavior and preferences, enabling data-driven decision-making and personalized experiences.")
    context662 = ("Businesses benefit from CRM solutions with improved customer insights, streamlined processes, and the ability to nurture leads and retain customers effectively.")
    context663 = ("Personalization in email marketing campaigns increases engagement and conversion rates by delivering relevant content tailored to individual preferences.")
    context664 = ("Clothing brands should regularly update CRM strategies by monitoring customer trends, preferences, and feedback to remain responsive to changing needs.")
    context665 = ("Omni-channel customer service ensures consistency and convenience across all customer touchpoints, whether in-store, online, or via mobile.")
    context666 = ("A successful CRM strategy includes customer data management, personalization, analytics, and a customer-centric culture within the organization.")
    context667 = ("Businesses safeguard data security and privacy through robust cybersecurity measures, data encryption, and compliance with privacy regulations.")
    context668 = ("CRM allows clothing retailers to segment their customer base based on demographics, purchase history, and behavior, making it easier to tailor marketing efforts.")
    context669 = ("Employee training and engagement are critical for providing excellent customer service, as motivated and well-informed staff are more likely to satisfy customer needs.")
    context670 = ("Encouraging feedback involves creating user-friendly feedback platforms, conducting surveys, and offering incentives to make it easy for customers to share their insights.")
    context671 = ("Omni-channel customer service ensures that customers experience consistent and convenient interactions across all channels, fostering strong and loyal relationships.")
    context672 = ("Best practices include prompt acknowledgment, thorough investigation, resolution, and seeking customer feedback to ensure complaints are resolved effectively.")
    context673 = ("CRM systems enable businesses to identify and nurture leads by tracking interactions and automating lead scoring to prioritize potential customers.")
    context674 = ("Strategies for retaining loyal customers include offering rewards, personalized discounts, and exceptional customer service to reduce churn.")
    context675 = ("Data analytics in CRM helps businesses analyze customer behavior patterns, enabling them to make data-driven decisions and personalize interactions.")
    context676 = ("Clothing retailers measure effectiveness through metrics such as customer satisfaction, retention rates, lead conversion, and return on investment (ROI).")
    context677 = ("Automation simplifies routine tasks, enhances efficiency, and ensures timely follow-ups, improving the overall customer experience.")
    context678 = ("Personalization involves segmenting customers, tailoring content, and delivering personalized offers, promotions, and recommendations.")
    context679 = ("Customer feedback provides valuable insights into customer preferences and areas for improvement, helping shape CRM strategies.")
    context680 = ("Strategies include offering loyalty programs, conducting surveys, and tracking online behavior to collect and update customer data.")
    context681 = ("Clothing brands protect customer data through data encryption, strict access controls, and compliance with privacy regulations to ensure responsible usage.")
    context682 = ("Key components include customer data management, personalization, analytics, and a customer-centric culture within the organization.")
    context683 = ("Social media platforms allow clothing brands to engage with customers, gather feedback, and address concerns in real-time, fostering strong customer relationships.")
    context684 = ("Loyalty programs and incentives reward customers for repeat purchases, encourage brand loyalty, and boost customer retention.")
    context685 = ("CRM systems help businesses track sales leads throughout the sales funnel, monitor progress, and measure lead conversion effectively.")
    context686 = ("Clothing retailers adapt by monitoring trends, offering new styles, and communicating with customers to understand their preferences.")
    context687 = ("Customer service representatives play a key role in resolving issues, addressing inquiries, and ensuring customers have a positive experience with the brand.")
    context688 = ("Strategies include follow-up emails, post-purchase surveys, and personalized recommendations to maintain engagement after a sale.")
    context689 = ("Ensuring consistency involves synchronizing inventory, pricing, and providing uniform customer service across all channels.")
    context690 = ("Factors include product quality, personalized experiences, rewards programs, and excellent customer service that inspire loyalty and advocacy.")
    context691 = ("CRM systems track and analyze customer interactions, helping brands identify and address emerging concerns promptly.")
    context692 = ("Community engagement events provide opportunities for customers to connect with the brand, fostering a sense of belonging and stronger relationships.")
    context693 = ("Effective returns and exchanges involve clear policies, hassle-free processes, and a focus on customer satisfaction to maintain positive relationships.")
    context694 = ("Personalized loyalty programs offer tailored rewards, discounts, and incentives that enhance customer retention and engagement.")
    context695 = ("Clothing brands invest in employee training programs to ensure staff can provide exceptional service and build strong customer relationships.")
    context696 = ("CRM systems help identify high-value customers, allowing brands to offer exclusive benefits and rewards to enhance customer loyalty.")
    context697 = ("Data analytics help brands analyze sales trends, customer preferences, and fashion shifts, allowing them to adjust inventory accordingly.")
    context698 = ("Strategies include transparent communication, eco-friendly product lines, and participation in social and environmental initiatives to engage customers in sustainability.")
    context699 = ("Brands can implement regular data cleansing processes and encourage customers to update their information to maintain accurate CRM data.")
    context700 = ("A feedback and improvement loop allows brands to continually gather insights, make necessary improvements, and demonstrate responsiveness to customer needs, strengthening relationships.")
    
    context_group_1 = (
        context1 + "\n\n" + context2 + "\n\n" + context3 + "\n\n" + context4 + "\n\n" + context5 + "\n\n" +
        context6 + "\n\n" + context7 + "\n\n" + context8 + "\n\n" + context9 + "\n\n" + context10 + "\n\n" +    
        context11 + "\n\n" + context12 + "\n\n" + context13 + "\n\n" + context14 + "\n\n" + context15 + "\n\n" +
        context16 + "\n\n" + context17 + "\n\n" + context18 + "\n\n" + context19 + "\n\n" + context20 + "\n\n" +
        context21 + "\n\n" + context22 + "\n\n" + context23 + "\n\n" + context24 + "\n\n" + context25 + "\n\n" +
        context26 + "\n\n" + context27 + "\n\n" + context28 + "\n\n" + context29 + "\n\n" + context30 + "\n\n" +
        context31 + "\n\n" + context32 + "\n\n" + context33 + "\n\n" + context34 + "\n\n" + context35 + "\n\n" +
        context36 + "\n\n" + context37 + "\n\n" + context38 + "\n\n" + context39 + "\n\n" + context40 
    )

    context_group_2 = (
        context41 + "\n\n" + context42 + "\n\n" + context43 + "\n\n" + context44 + "\n\n" + context45 + "\n\n" +
        context46 + "\n\n" + context47 + "\n\n" + context48 + "\n\n" + context49 + "\n\n" + context50 + "\n\n" +
        context51 + "\n\n" + context52 + "\n\n" + context53 + "\n\n" + context54 + "\n\n" + context55 + "\n\n" +
        context56 + "\n\n" + context57 + "\n\n" + context58 + "\n\n" + context59 + "\n\n" + context60 + "\n\n" +
        context61 + "\n\n" + context62 + "\n\n" + context63 + "\n\n" + context64 + "\n\n" + context65 + "\n\n" +
        context66 + "\n\n" + context67 + "\n\n" + context68 + "\n\n" + context69 + "\n\n" + context70 + "\n\n" +
        context71 + "\n\n" + context72 + "\n\n" + context73 + "\n\n" + context74 + "\n\n" + context75 + "\n\n" +
        context76 + "\n\n" + context77 + "\n\n" + context78 + "\n\n" + context79 + "\n\n" + context80 
    )

    context_group_3 = (
        context81 + "\n\n" + context82 + "\n\n" + context83 + "\n\n" + context84 + "\n\n" + context85 + "\n\n" +
        context86 + "\n\n" + context87 + "\n\n" + context88 + "\n\n" + context89 + "\n\n" + context90 + "\n\n" +
        context91 + "\n\n" + context92 + "\n\n" + context93 + "\n\n" + context94 + "\n\n" + context95 + "\n\n" +
        context96 + "\n\n" + context97 + "\n\n" + context98 + "\n\n" + context99 + "\n\n" + context100 + "\n\n" +
        context101 + "\n\n" + context102 + "\n\n" + context103 + "\n\n" + context104 + "\n\n" + context105 + "\n\n" +
        context106 + "\n\n" + context107 + "\n\n" + context108 + "\n\n" + context109 + "\n\n" + context110 + "\n\n" +
        context111 + "\n\n" + context112 + "\n\n" + context113 + "\n\n" + context114 + "\n\n" + context115 + "\n\n" +
        context116 + "\n\n" + context117 + "\n\n" + context118 + "\n\n" + context119 + "\n\n" + context120 
    )    

    context_group_4 = (
        context121 + "\n\n" + context122 + "\n\n" + context123 + "\n\n" + context124 + "\n\n" + context125 + "\n\n" +
        context126 + "\n\n" + context127 + "\n\n" + context128 + "\n\n" + context129 + "\n\n" + context130 + "\n\n" +
        context131 + "\n\n" + context132 + "\n\n" + context133 + "\n\n" + context134 + "\n\n" + context135 + "\n\n" +
        context136 + "\n\n" + context137 + "\n\n" + context138 + "\n\n" + context139 + "\n\n" + context140 + "\n\n" +   
        context141 + "\n\n" + context142 + "\n\n" + context143 + "\n\n" + context144 + "\n\n" + context145 + "\n\n" +
        context146 + "\n\n" + context147 + "\n\n" + context148 + "\n\n" + context149 + "\n\n" + context150 + "\n\n" +
        context151 + "\n\n" + context152 + "\n\n" + context153 + "\n\n" + context154 + "\n\n" + context155 + "\n\n" +
        context156 + "\n\n" + context157 + "\n\n" + context158 + "\n\n" + context159 + "\n\n" + context160 
    )

    context_group_5 = (
        context161 + "\n\n" + context162 + "\n\n" + context163 + "\n\n" + context164 + "\n\n" + context165 + "\n\n" +
        context166 + "\n\n" + context167 + "\n\n" + context168 + "\n\n" + context169 + "\n\n" + context170 + "\n\n" +
        context171 + "\n\n" + context172 + "\n\n" + context173 + "\n\n" + context174 + "\n\n" + context175 + "\n\n" +
        context176 + "\n\n" + context177 + "\n\n" + context178 + "\n\n" + context179 + "\n\n" + context180 + "\n\n" +
        context181 + "\n\n" + context182 + "\n\n" + context183 + "\n\n" + context184 + "\n\n" + context185 + "\n\n" +
        context186 + "\n\n" + context187 + "\n\n" + context188 + "\n\n" + context189 + "\n\n" + context190 + "\n\n" +
        context191 + "\n\n" + context192 + "\n\n" + context193 + "\n\n" + context194 + "\n\n" + context195 + "\n\n" +
        context196 + "\n\n" + context197 + "\n\n" + context198 + "\n\n" + context199 + "\n\n" + context200
    )

    context_group_6 = (
        context201 + "\n\n" + context202 + "\n\n" + context203 + "\n\n" + context204 + "\n\n" + context205 + "\n\n" +
        context206 + "\n\n" + context207 + "\n\n" + context208 + "\n\n" + context209 + "\n\n" + context210 + "\n\n" +
        context211 + "\n\n" + context212 + "\n\n" + context213 + "\n\n" + context214 + "\n\n" + context215 + "\n\n" +
        context216 + "\n\n" + context217 + "\n\n" + context218 + "\n\n" + context219 + "\n\n" + context220 + "\n\n" +
        context221 + "\n\n" + context222 + "\n\n" + context223 + "\n\n" + context224 + "\n\n" + context225 + "\n\n" +
        context226 + "\n\n" + context227 + "\n\n" + context228 + "\n\n" + context229 + "\n\n" + context230 + "\n\n" +
        context231 + "\n\n" + context232 + "\n\n" + context233 + "\n\n" + context234 + "\n\n" + context235 + "\n\n" +
        context236 + "\n\n" + context237 + "\n\n" + context238 + "\n\n" + context239 + "\n\n" + context240 
    )

    context_group_7 = (
        context241 + "\n\n" + context242 + "\n\n" + context243 + "\n\n" + context244 + "\n\n" + context245 + "\n\n" +
        context246 + "\n\n" + context247 + "\n\n" + context248 + "\n\n" + context249 + "\n\n" + context250 + "\n\n" +
        context251 + "\n\n" + context252 + "\n\n" + context253 + "\n\n" + context254 + "\n\n" + context255 + "\n\n" +
        context256 + "\n\n" + context257 + "\n\n" + context258 + "\n\n" + context259 + "\n\n" + context260 + "\n\n" +
        context261 + "\n\n" + context262 + "\n\n" + context263 + "\n\n" + context264 + "\n\n" + context265 + "\n\n" +
        context266 + "\n\n" + context267 + "\n\n" + context268 + "\n\n" + context269 + "\n\n" + context270 + "\n\n" +
        context271 + "\n\n" + context272 + "\n\n" + context273 + "\n\n" + context274 + "\n\n" + context275 + "\n\n" +
        context276 + "\n\n" + context277 + "\n\n" + context278 + "\n\n" + context279 + "\n\n" + context280 
    )

    context_group_8 = (
        context281 + "\n\n" + context282 + "\n\n" + context283 + "\n\n" + context284 + "\n\n" + context285 + "\n\n" +
        context286 + "\n\n" + context287 + "\n\n" + context288 + "\n\n" + context289 + "\n\n" + context290 + "\n\n" +
        context291 + "\n\n" + context292 + "\n\n" + context293 + "\n\n" + context294 + "\n\n" + context295 + "\n\n" +
        context296 + "\n\n" + context297 + "\n\n" + context298 + "\n\n" + context299 + "\n\n" + context300 + "\n\n" +
        context301 + "\n\n" + context302 + "\n\n" + context303 + "\n\n" + context304 + "\n\n" + context305 + "\n\n" +
        context306 + "\n\n" + context307 + "\n\n" + context308 + "\n\n" + context309 + "\n\n" + context310 + "\n\n" +
        context311 + "\n\n" + context312 + "\n\n" + context313 + "\n\n" + context314 + "\n\n" + context315 + "\n\n" +
        context316 + "\n\n" + context317 + "\n\n" + context318 + "\n\n" + context319 + "\n\n" + context320
    )

    context_group_9 = (
        context321 + "\n\n" + context322 + "\n\n" + context323 + "\n\n" + context324 + "\n\n" + context325 + "\n\n" +
        context326 + "\n\n" + context327 + "\n\n" + context328 + "\n\n" + context329 + "\n\n" + context330 + "\n\n" +
        context331 + "\n\n" + context332 + "\n\n" + context333 + "\n\n" + context334 + "\n\n" + context335 + "\n\n" +
        context336 + "\n\n" + context337 + "\n\n" + context338 + "\n\n" + context339 + "\n\n" + context340 + "\n\n" +
        context341 + "\n\n" + context342 + "\n\n" + context343 + "\n\n" + context344 + "\n\n" + context345 + "\n\n" +
        context346 + "\n\n" + context347 + "\n\n" + context348 + "\n\n" + context349 + "\n\n" + context350 + "\n\n" +
        context351 + "\n\n" + context352 + "\n\n" + context353 + "\n\n" + context354 + "\n\n" + context355 + "\n\n" +
        context356 + "\n\n" + context357 + "\n\n" + context358 + "\n\n" + context359 + "\n\n" + context360 
    )

    context_group_10 = (
        context361 + "\n\n" + context362 + "\n\n" + context363 + "\n\n" + context364 + "\n\n" + context365 + "\n\n" +
        context366 + "\n\n" + context367 + "\n\n" + context368 + "\n\n" + context369 + "\n\n" + context370 + "\n\n" +
        context371 + "\n\n" + context372 + "\n\n" + context373 + "\n\n" + context374 + "\n\n" + context375 + "\n\n" +
        context376 + "\n\n" + context377 + "\n\n" + context378 + "\n\n" + context379 + "\n\n" + context380 + "\n\n" +
        context381 + "\n\n" + context382 + "\n\n" + context383 + "\n\n" + context384 + "\n\n" + context385 + "\n\n" +
        context386 + "\n\n" + context387 + "\n\n" + context388 + "\n\n" + context389 + "\n\n" + context390 + "\n\n" +
        context391 + "\n\n" + context392 + "\n\n" + context393 + "\n\n" + context394 + "\n\n" + context395 + "\n\n" +
        context396 + "\n\n" + context397 + "\n\n" + context398 + "\n\n" + context399 + "\n\n" + context400
    )

    context_group_11 = (
        context401 + "\n\n" + context402 + "\n\n" + context403 + "\n\n" + context404 + "\n\n" + context405 + "\n\n" +
        context406 + "\n\n" + context407 + "\n\n" + context408 + "\n\n" + context409 + "\n\n" + context410 + "\n\n" +
        context411 + "\n\n" + context412 + "\n\n" + context413 + "\n\n" + context414 + "\n\n" + context415 + "\n\n" +
        context416 + "\n\n" + context417 + "\n\n" + context418 + "\n\n" + context419 + "\n\n" + context420 + "\n\n" +
        context421 + "\n\n" + context422 + "\n\n" + context423 + "\n\n" + context424 + "\n\n" + context425 + "\n\n" +
        context426 + "\n\n" + context427 + "\n\n" + context428 + "\n\n" + context429 + "\n\n" + context430 + "\n\n" +
        context431 + "\n\n" + context432 + "\n\n" + context433 + "\n\n" + context434 + "\n\n" + context435 + "\n\n" +
        context436 + "\n\n" + context437 + "\n\n" + context438 + "\n\n" + context439 + "\n\n" + context440
    )

    context_group_12 = (
        context441 + "\n\n" + context442 + "\n\n" + context443 + "\n\n" + context444 + "\n\n" + context445 + "\n\n" +
        context446 + "\n\n" + context447 + "\n\n" + context448 + "\n\n" + context449 + "\n\n" + context450 + "\n\n" +
        context451 + "\n\n" + context452 + "\n\n" + context453 + "\n\n" + context454 + "\n\n" + context455 + "\n\n" +
        context456 + "\n\n" + context457 + "\n\n" + context458 + "\n\n" + context459 + "\n\n" + context460 + "\n\n" +
        context461 + "\n\n" + context462 + "\n\n" + context463 + "\n\n" + context464 + "\n\n" + context465 + "\n\n" +
        context466 + "\n\n" + context467 + "\n\n" + context468 + "\n\n" + context469 + "\n\n" + context470 + "\n\n" +
        context471 + "\n\n" + context472 + "\n\n" + context473 + "\n\n" + context474 + "\n\n" + context475 + "\n\n" +
        context476 + "\n\n" + context477 + "\n\n" + context478 + "\n\n" + context479 + "\n\n" + context480 
    )

    context_group_13 = (
        context481 + "\n\n" + context482 + "\n\n" + context483 + "\n\n" + context484 + "\n\n" + context485 + "\n\n" +
        context486 + "\n\n" + context487 + "\n\n" + context488 + "\n\n" + context489 + "\n\n" + context490 + "\n\n" +
        context491 + "\n\n" + context492 + "\n\n" + context493 + "\n\n" + context494 + "\n\n" + context495 + "\n\n" +
        context496 + "\n\n" + context497 + "\n\n" + context498 + "\n\n" + context499 + "\n\n" + context500 + "\n\n" +
        context501 + "\n\n" + context502 + "\n\n" + context503 + "\n\n" + context504 + "\n\n" + context505 + "\n\n" +
        context506 + "\n\n" + context507 + "\n\n" + context508 + "\n\n" + context509 + "\n\n" + context510 + "\n\n" +
        context511 + "\n\n" + context512 + "\n\n" + context513 + "\n\n" + context514 + "\n\n" + context515 + "\n\n" +
        context516 + "\n\n" + context517 + "\n\n" + context518 + "\n\n" + context519 + "\n\n" + context520
    )

    context_group_14 = (
        context521 + "\n\n" + context522 + "\n\n" + context523 + "\n\n" + context524 + "\n\n" + context525 + "\n\n" +
        context526 + "\n\n" + context527 + "\n\n" + context528 + "\n\n" + context529 + "\n\n" + context530 + "\n\n" +
        context531 + "\n\n" + context532 + "\n\n" + context533 + "\n\n" + context534 + "\n\n" + context535 + "\n\n" +
        context536 + "\n\n" + context537 + "\n\n" + context538 + "\n\n" + context539 + "\n\n" + context540 + "\n\n" +
        context541 + "\n\n" + context542 + "\n\n" + context543 + "\n\n" + context544 + "\n\n" + context545 + "\n\n" +
        context546 + "\n\n" + context547 + "\n\n" + context548 + "\n\n" + context549 + "\n\n" + context550 + "\n\n" +
        context551 + "\n\n" + context552 + "\n\n" + context553 + "\n\n" + context554 + "\n\n" + context555 + "\n\n" +
        context556 + "\n\n" + context557 + "\n\n" + context558 + "\n\n" + context559 + "\n\n" + context560
    )

    context_group_15 = (
        context561 + "\n\n" + context562 + "\n\n" + context563 + "\n\n" + context564 + "\n\n" + context565 + "\n\n" +
        context566 + "\n\n" + context567 + "\n\n" + context568 + "\n\n" + context569 + "\n\n" + context570 + "\n\n" +
        context571 + "\n\n" + context572 + "\n\n" + context573 + "\n\n" + context574 + "\n\n" + context575 + "\n\n" +
        context576 + "\n\n" + context577 + "\n\n" + context578 + "\n\n" + context579 + "\n\n" + context580 + "\n\n" +
        context581 + "\n\n" + context582 + "\n\n" + context583 + "\n\n" + context584 + "\n\n" + context585 + "\n\n" +
        context586 + "\n\n" + context587 + "\n\n" + context588 + "\n\n" + context589 + "\n\n" + context590 + "\n\n" +
        context591 + "\n\n" + context592 + "\n\n" + context593 + "\n\n" + context594 + "\n\n" + context595 + "\n\n" +
        context596 + "\n\n" + context597 + "\n\n" + context598 + "\n\n" + context599 + "\n\n" + context600
    )

    context_group_16 = (
        context601 + "\n\n" + context602 + "\n\n" + context603 + "\n\n" + context604 + "\n\n" + context605 + "\n\n" +
        context606 + "\n\n" + context607 + "\n\n" + context608 + "\n\n" + context609 + "\n\n" + context610 + "\n\n" +
        context611 + "\n\n" + context612 + "\n\n" + context613 + "\n\n" + context614 + "\n\n" + context615 + "\n\n" +
        context616 + "\n\n" + context617 + "\n\n" + context618 + "\n\n" + context619 + "\n\n" + context620 + "\n\n" +
        context621 + "\n\n" + context622 + "\n\n" + context623 + "\n\n" + context624 + "\n\n" + context625 + "\n\n" +
        context626 + "\n\n" + context627 + "\n\n" + context628 + "\n\n" + context629 + "\n\n" + context630 + "\n\n" +
        context631 + "\n\n" + context632 + "\n\n" + context633 + "\n\n" + context634 + "\n\n" + context635 + "\n\n" +
        context636 + "\n\n" + context637 + "\n\n" + context638 + "\n\n" + context639 + "\n\n" + context640
    )

    context_group_17 = (
        context641 + "\n\n" + context642 + "\n\n" + context643 + "\n\n" + context644 + "\n\n" + context645 + "\n\n" +
        context646 + "\n\n" + context647 + "\n\n" + context648 + "\n\n" + context649 + "\n\n" + context650 + "\n\n" +
        context651 + "\n\n" + context652 + "\n\n" + context653 + "\n\n" + context654 + "\n\n" + context655 + "\n\n" +
        context656 + "\n\n" + context657 + "\n\n" + context658 + "\n\n" + context659 + "\n\n" + context660 + "\n\n" +
        context661 + "\n\n" + context662 + "\n\n" + context663 + "\n\n" + context664 + "\n\n" + context665 + "\n\n" +
        context666 + "\n\n" + context667 + "\n\n" + context668 + "\n\n" + context669 + "\n\n" + context670 + "\n\n" +
        context671 + "\n\n" + context672 + "\n\n" + context673 + "\n\n" + context674 + "\n\n" + context675 + "\n\n" +
        context676 + "\n\n" + context677 + "\n\n" + context678 + "\n\n" + context679 + "\n\n" + context680 + "\n\n" +
        context681 + "\n\n" + context682 + "\n\n" + context683 + "\n\n" + context684 + "\n\n" + context685 + "\n\n" +
        context686 + "\n\n" + context687 + "\n\n" + context688 + "\n\n" + context689 + "\n\n" + context690 + "\n\n" +
        context691 + "\n\n" + context692 + "\n\n" + context693 + "\n\n" + context694 + "\n\n" + context695 + "\n\n" +
        context696 + "\n\n" + context697 + "\n\n" + context698 + "\n\n" + context699 + "\n\n" + context700
    )

    all_contexts = context_group_1 + "\n\n" + context_group_2 + "\n\n" + context_group_3 + "\n\n" + context_group_4 + "\n\n" + context_group_5 + "\n\n" + context_group_6 + "\n\n" + context_group_7 + "\n\n" + context_group_8 + "\n\n" + context_group_9 + "\n\n" + context_group_10 + "\n\n" + context_group_11 + "\n\n" + context_group_12 + "\n\n" + context_group_13 + "\n\n" + context_group_14 + "\n\n" + context_group_15 + "\n\n" + context_group_16 + "\n\n" + context_group_17 

    # Calculate cosine similarity
    all_contexts_list = all_contexts.split("\n")  # Split the string into a list of contexts
    all_texts = [user_input] + all_contexts_list  # Concatenate user input and contexts

    vectorizer = CountVectorizer().fit_transform(all_texts)
    vectors = vectorizer.toarray()
    cosine_similarities = cosine_similarity(vectors)

    # Find the most similar context
    most_similar_context_idx = cosine_similarities[0][1:].argmax()

    # Add error handling for index out of range
    if 0 <= most_similar_context_idx < len(all_contexts_list):
        most_similar_context = all_contexts_list[most_similar_context_idx]

    to_predict = []

    # Create a context with the user's question
    context = {
        "context": all_contexts,
        "qas": [{"question": user_input, "id": "user_question"}]
    }

    to_predict.append(context)

    answers, _ = model.predict(to_predict)

    # Extract the predicted answers and their confidence scores
    predicted_answers = answers[0]["answer"]
    confidences = answers[0].get("confidence", [1.0] * len(predicted_answers))

    # Set a confidence threshold (adjust as needed)
    confidence_threshold = 0.7

    best_f1_score = 0
    best_answer = None
    em = 0  # Initialize EM to 0

    # Iterate through predicted answers, calculating F1 and EM
    for answer, confidence in zip(predicted_answers, confidences):
        f1, em_candidate = compute_f1_em(user_input, answer)
        if answer.strip() and confidence > confidence_threshold and f1 > best_f1_score:
            best_f1_score = f1
            best_answer = answer
            em = max(em, em_candidate)  # Update EM based on the candidate EM

    if best_answer:
        # Print the user question and the most accurate answer
        print(f"User Question: {user_input}")
        print(f"DistilBERT+LSA Chatbot: {best_answer}")
        # Print F1 and EM scores
        print(f"F1 Score: {best_f1_score}")
        print(f"Exact Match (EM): {em}")
    elif most_similar_context:
        # If there is no match in cosine similarity, but there's a most similar context, use that as the answer
        print(f"User Question: {user_input}")
        print(f"DistilBERT+LSA Chatbot: {most_similar_context}")
    else:
        print("DistilBERT+LSA Chatbot: I couldn't find an answer to your question.")